# To facilitate processing GTEx expression data (in .gct format), transform to *.gctx first:

- use gct2gctx CLI to convert GTEx expression files from *.gct to *.gctx format to allow for faster subsetting using cid selection
    - https://clue.io/cmapPy/available_modules.html#pandasgexpress
    - https://clue.io/cmapPy/pandasGEXpress.html#module-cmapPy.pandasGEXpress.gct2gctx
    - command used:
    
    ```    
    gct2gctx -filename GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct -output_filepath GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gctx -v 
    ```
    
# Data processing

1. Parse sample metadata to determine which samples of interest you need (those from areas of interest)
    - Read in metadata from GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt
2. Parse expression data (feeding in cid --sample ids) using `cmapPy.pandasGEXpress.parse`
3. Issue of mismatch between list of cids and what is available in expression data
    - copy the list of mismatch cids that are output in the Exception caught from the parse function
    - read in expression data from the samples of interest
4. Select expression from gene of interest 

## 1. Select samples based on metadata

In [1]:
import numpy as np
import pandas as pd

In [2]:
metadf = pd.read_table('./data/raw/GTEx_v1.9/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt', sep='\t')

In [3]:
metadf.head()

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
0,GTEX-1117F-0003-SM-58Q7G,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GTEX-1117F-0003-SM-5DWSB,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GTEX-1117F-0003-SM-6WBT7,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GTEX-1117F-0011-R10a-SM-AHZ7F,NaN,"B1, A1",NaN,NaN,Brain,Brain - Frontal Cortex (BA9),0009834,1193.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GTEX-1117F-0011-R10b-SM-CYKQ8,NaN,"B1, A1",NaN,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# subject info can also be found in this table
subjid = pd.read_table('./data/raw/GTEx_v1.9/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt')

In [5]:
subjid.head()

,SUBJID,SEX,AGE,DTHHRDY
0,GTEX-1117F,2,60-69,4.0
1,GTEX-111CU,1,50-59,0.0
2,GTEX-111FC,1,60-69,1.0
3,GTEX-111VG,1,60-69,3.0
4,GTEX-111YS,1,60-69,0.0


### Number of donors
The `GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt` file shows there are 980 total donors. 

There is also agreement with this number from the metadata file `GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt` by counting the prefix of the SAMPID (which is == SUBJID).

In [6]:
subjid.SUBJID.nunique()

980

In [7]:
# donors are defined by the prefix of a SAMPID
metadf.SAMPID.str.split('-').str[:2].str.join('-').nunique()

980

In [8]:
metasubdjid = metadf.SAMPID.str.split('-').str[:2].str.join('-').unique()

In [9]:
intersection = np.intersect1d(subjid.SUBJID, metasubdjid)

In [10]:
intersection.shape

(980,)

### Samples of interest
There are 22951 total samples, 3326 of which are from the brain (and 164 from the substantia nigra).

In [11]:
metadf.shape

(22951, 63)

In [12]:
sample_ids = metadf.SAMPID.unique()

In [13]:
#sample_ids = df[df['SMTSD'].str.contains('Brain - Cortex')].SAMPID.unique()

In [14]:
sample_ids.shape

(22951,)

In [15]:
sample_ids[:5]

array(['GTEX-1117F-0003-SM-58Q7G', 'GTEX-1117F-0003-SM-5DWSB',
       'GTEX-1117F-0003-SM-6WBT7', 'GTEX-1117F-0011-R10a-SM-AHZ7F',
       'GTEX-1117F-0011-R10b-SM-CYKQ8'], dtype=object)

# 2. Parse expression data with cmapPy
- https://github.com/cmap/cmapPy
- https://clue.io/cmapPy/pandasGEXpress.html
- https://clue.io/cmapPy/_modules/cmapPy/pandasGEXpress/subset.html

In [16]:
from cmapPy.pandasGEXpress.parse import parse

## Extract expression of samples of interest from .gctx file using sample_ids identified above
The cell below returns an error because not all 22951 samples identified above are present in the expression matrix. Save the mismatch_ids that are output in the error message and subtract them from the sample_ids above in order to extract the data of interest.

In [17]:
%%time
gtex = parse(file_path='./data/raw/GTEx_v1.9/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gctx', 
             cid=list(sample_ids))

No handlers could be found for logger "cmap_logger"


Exception: parse_gctx check_id_validity some of the ids being used to subset the data are not present in the metadata for the file being parsed - mismatch_ids:  set(['GTEX-1B98T-1026-SM-79OL8', 'GTEX-ZT9W-0002-SM-53UB4', 'GTEX-1PBJJ-0004-SM-DJWWY', 'GTEX-R55F-0226-SM-48FCI', 'GTEX-145MN-5004-SM-AN41N', 'GTEX-1I1GS-0004-SM-DHP2C', 'GTEX-SN8G-0002-SM-4SVRA', 'GTEX-14LLW-0011-R10a-SM-62LEG', 'GTEX-1B932-0126-SM-7PC2M', 'GTEX-14H4A-0003-SM-5DU4K', 'GTEX-1NV88-0926-SM-DLT2P', 'GTEX-1E1VI-3126-SM-7IGNP', 'GTEX-145MI-0001-SM-7DRO7', 'GTEX-1477Z-0626-SM-CYKRO', 'GTEX-1399Q-0005-SM-5NQAG', 'GTEX-1CB4E-5001-SM-AN468', 'GTEX-13FHO-0011-R10b-SM-CYKPF', 'GTEX-OHPJ-0926-SM-2HMJ1', 'GTEX-YBZK-0001-SM-58Q92', 'K-562-SM-26GMQ', 'GTEX-TML8-0004-SM-325A2', 'GTEX-18D9U-0001-SM-DJWXR', 'GTEX-15FZZ-1226-SM-7KUFV', 'K-562-SM-4RTUY', 'GTEX-1ICG6-1126-SM-ADEIF', 'GTEX-14C5O-0011-R10a-SM-CYKSO', 'GTEX-1B933-0003-SM-7MD4Z', 'GTEX-ZLFU-0004-SM-7DRPW', 'GTEX-13FHP-0011-R10a-SM-AHZMM', 'GTEX-14JG1-0126-SM-CYKQB', '

In [18]:
# for all samples
mismatch_ids = pd.Series(['GTEX-1B98T-1026-SM-79OL8', 'GTEX-ZT9W-0002-SM-53UB4', 'GTEX-1PBJJ-0004-SM-DJWWY', 'GTEX-R55F-0226-SM-48FCI', 'GTEX-145MN-5004-SM-AN41N', 'GTEX-1I1GS-0004-SM-DHP2C', 'GTEX-SN8G-0002-SM-4SVRA', 'GTEX-14LLW-0011-R10a-SM-62LEG', 'GTEX-1B932-0126-SM-7PC2M', 'GTEX-14H4A-0003-SM-5DU4K', 'GTEX-1NV88-0926-SM-DLT2P', 'GTEX-1E1VI-3126-SM-7IGNP', 'GTEX-145MI-0001-SM-7DRO7', 'GTEX-1477Z-0626-SM-CYKRO', 'GTEX-1399Q-0005-SM-5NQAG', 'GTEX-1CB4E-5001-SM-AN468', 'GTEX-13FHO-0011-R10b-SM-CYKPF', 'GTEX-OHPJ-0926-SM-2HMJ1', 'GTEX-YBZK-0001-SM-58Q92', 'K-562-SM-26GMQ', 'GTEX-TML8-0004-SM-325A2', 'GTEX-18D9U-0001-SM-DJWXR', 'GTEX-15FZZ-1226-SM-7KUFV', 'K-562-SM-4RTUY', 'GTEX-1ICG6-1126-SM-ADEIF', 'GTEX-14C5O-0011-R10a-SM-CYKSO', 'GTEX-1B933-0003-SM-7MD4Z', 'GTEX-ZLFU-0004-SM-7DRPW', 'GTEX-13FHP-0011-R10a-SM-AHZMM', 'GTEX-14JG1-0126-SM-CYKQB', 'GTEX-15DCD-0001-SM-6RBJS', 'GTEX-N7MS-2526-SM-26GMA', 'GTEX-QVJO-0002-SM-2MRI6', 'GTEX-QEG4-0426-SM-5S2VQ', 'GTEX-XLM4-0003-SM-53IUZ', 'GTEX-N7MS-2526-SM-26GMR', 'GTEX-UTHO-3026-SM-CYKVK', 'GTEX-13G51-0226-SM-CYKUT', 'GTEX-OHPJ-0526-SM-EZ6XQ', 'GTEX-1I1GQ-0726-SM-ARZM4', 'GTEX-NL3H-0011-R3a-SM-2AXW9', 'GTEX-14E6E-0001-SM-5DU3J', 'GTEX-ZPIC-0004-SM-53UBP', 'GTEX-132NY-0001-SM-5GYA3', 'GTEX-OOBK-0004-SM-2BWYT', 'GTEX-S4UY-0426-SM-3K2AF', 'GTEX-Q2AG-0011-R7A-SM-2IZKA', 'GTEX-UPIC-1726-SM-4IHKG', 'GTEX-1HCUA-1826-SM-A8N87', 'GTEX-R55E-2526-SM-2TO6F', 'GTEX-13N11-0003-SM-5HJQF', 'GTEX-RNOR-0011-R6A-SM-2TWCF', 'GTEX-ZVP2-0002-SM-4YUXK', 'GTEX-13QBU-0003-SM-6WSD1', 'GTEX-1R46S-3026-SM-DTXFP', 'GTEX-13O1R-0002-SM-6WSB4', 'GTEX-1JK1U-5007-SM-AN487', 'GTEX-13O3P-0003-SM-5HJPJ', 'GTEX-139UW-0003-SM-5DU4H', 'GTEX-WVLH-0001-SM-5SOE6', 'GTEX-SNOS-0004-SM-325AE', 'GTEX-1CB4I-5004-SM-AN45V', 'GTEX-S32W-0004-SM-2XCE9', 'GTEX-1I4MK-0004-SM-DC69Q', 'GTEX-XPT6-0004-SM-4YUXL', 'GTEX-S3XE-0003-SM-6WBUO', 'GTEX-S32W-2226-SM-5S2S7', 'GTEX-1JMQJ-0003-SM-DE1N6', 'GTEX-PWOO-0826-SM-48TCL', 'GTEX-12WSF-0002-SM-5DWRS', 'K-562-SM-4TT1D', 'GTEX-13O21-0003-SM-5DU66', 'GTEX-14A5H-0004-SM-5DU5W', 'GTEX-1F6RS-3026-SM-7MXTQ', 'GTEX-W5WG-0003-SM-3UZF6', 'K-562-SM-5P9EQ', 'GTEX-13FTY-0004-SM-6WSBL', 'GTEX-XUZC-0002-SM-53IVH', 'GTEX-1GPI7-0002-SM-DINAX', 'GTEX-UJMC-0002-SM-4X626', 'GTEX-14BIM-3026-SM-CYKU4', 'GTEX-P44H-1126-SM-2XK3A', 'GTEX-11TT1-0001-SM-5DWT7', 'GTEX-SIU7-1826-SM-2VCU6', 'GTEX-PWOO-2226-SM-2S1P1', 'GTEX-1ICLY-5010-SM-AN47V', 'GTEX-OHPJ-0926-SM-2D45G', 'GTEX-P78B-1626-SM-2S1O1', 'GTEX-14ABY-0003-SM-7DRQD', 'GTEX-1F88E-0005-SM-A8N7N', 'GTEX-QVUS-0001-SM-3UZFP', 'K-562-SM-32PMP', 'GTEX-13N2G-3026-SM-CYKVX', 'GTEX-S4Q7-1626-SM-2XU8B', 'GTEX-YFCO-0001-SM-4YUWM', 'GTEX-169BO-0003-SM-6RBL1', 'GTEX-SIU7-1326-SM-4BRWW', 'GTEX-WQUQ-0002-SM-3PYK7', 'GTEX-T6MN-0011-R10A-SM-5S2SB', 'GTEX-11DXY-0004-SM-6WBTO', 'GTEX-WHPG-0002-SM-5URBH', 'GTEX-ZVE2-0002-SM-53UAO', 'GTEX-11ZUS-0003-SM-58Q8H', 'GTEX-14753-0003-SM-6WSBF', 'GTEX-PWOO-0004-SM-2XK17', 'K-562-SM-32PLC', 'GTEX-1477Z-0003-SM-5DU3M', 'GTEX-TMKS-0426-SM-3L284', 'GTEX-1IDJU-5004-SM-AN481', 'GTEX-1AX8Y-1526-SM-73KXD', 'K-562-SM-2HMJ2', 'GTEX-P4PQ-0003-SM-3UZFF', 'GTEX-13OVH-1026-SM-CYKS2', 'GTEX-111FC-0011-R10a-SM-CYKQ9', 'GTEX-PW2O-0004-SM-2XK1V', 'GTEX-14E6E-5001-SM-AN42D', 'GTEX-X585-0003-SM-3PYKR', 'GTEX-11GSP-0004-SM-6WBTQ', 'GTEX-12WSL-5016-SM-AN3XE', 'GTEX-OXRO-0126-SM-2M48E', 'GTEX-1CB4F-0826-SM-DLL3M', 'GTEX-1LSVX-0003-SM-DJWYB', 'GTEX-1C2JI-0003-SM-7MA6E', 'GTEX-1JKYR-0004-SM-DJWYJ', 'GTEX-17EVP-0726-SM-CYKU5', 'GTEX-R53T-0003-SM-5URBL', 'GTEX-15SB6-0002-SM-6RBJK', 'GTEX-QLQ7-1326-SM-4R1JY', 'GTEX-PWN1-0226-SM-5SI7O', 'GTEX-X5EB-0001-SM-3PZ7V', 'GTEX-QXCU-0004-SM-3USSG', 'GTEX-P4PP-0326-SM-2HML9', 'GTEX-Q2AH-1626-SM-2QU36', 'GTEX-S95S-0004-SM-5JK3U', 'GTEX-13OVJ-0001-SM-6WSAU', 'GTEX-13QBU-0003-SM-5HJRN_1', 'GTEX-1GMR2-0003-SM-7MA7E', 'GTEX-ZVZQ-0326-SM-59HLL', 'GTEX-PWOO-2426-SM-2IZJI', 'GTEX-QVJO-0126-SM-2TO62', 'GTEX-OXRN-0003-SM-3USRW', 'GTEX-P44H-2626-SM-2XV61', 'GTEX-ZPU1-0826-SM-7DHMN', 'K-562-SM-4WAWH', 'GTEX-U8T8-2026-SM-3G3TN', 'GTEX-UTHO-0003-SM-5SOE4', 'GTEX-QV44-2026-SM-2TO57', 'GTEX-YEC3-1826-101807-SM-EYYVR', 'GTEX-XAJ8-0001-SM-5URBF', 'GTEX-14LZ3-2926-SM-5YY8T', 'GTEX-1MCYP-0003-SM-DJWXA', 'K-562-SM-5HL4J', 'GTEX-13N1W-0011-R11a-SM-5MR3D', 'GTEX-14H4A-5001-SM-AN42I', 'GTEX-PWO3-0926-SM-2IZIY', 'GTEX-16GPK-0011-R10a-SM-AHZMF', 'GTEX-OHPK-0003-SM-2H2WX', 'GTEX-12126-0526-SM-CYKNY', 'GTEX-RU72-0011-R9A-SM-2TWBL', 'GTEX-14PN4-0326-SM-CYKOK', 'GTEX-PWCY-2026-SM-5QGQ9', 'GTEX-S341-1726-SM-3K2AK', 'GTEX-Q2AG-0011-R8A-SM-2IZK3', 'GTEX-WOFM-0426-SM-4OOSG', 'GTEX-15DDE-0011-R10a-SM-7KULA', 'GTEX-Z9EW-0002-SM-6UH37', 'GTEX-R45C-0526-SM-2TWC1', 'GTEX-WI4N-1326-SM-4X5ZN', 'GTEX-WFG7-0326-SM-5SI7L', 'GTEX-Q2AI-0006-SM-2IZII', 'GTEX-QCQG-1926-SM-2S1PI', 'GTEX-1AX8Y-2426-SM-73KXP', 'GTEX-WEY5-0003-SM-3G3SD', 'GTEX-S4Z8-1026-SM-2XUAL', 'GTEX-ZUA1-0001-SM-6WBSL', 'GTEX-12BJ1-0126-SM-5LU9P', 'GTEX-Y9LG-0003-SM-53IUW', 'K-562-SM-6LPHZ', 'GTEX-QESD-0001-SM-5URC9', 'GTEX-PX3G-0126-SM-2IZIC', 'GTEX-NPJ8-0011-R11A-SM-2QU5D', 'GTEX-ZXG5-0002-SM-6WBT1', 'GTEX-14PQA-0003-SM-6RBJN', 'GTEX-ZV68-0011-R10a-SM-AHZ7N', 'GTEX-1F75I-0004-SM-7MA7F', 'GTEX-RUSQ-0003-SM-2XV75', 'GTEX-SNMC-0004-SM-2XJZO', 'GTEX-OHPJ-1626-SM-2HMKO', 'GTEX-Y5LM-0002-SM-53IUF', 'GTEX-1HUB1-2826-SM-A8N81', 'GTEX-RN5K-0001-SM-DINCK', 'GTEX-11PRG-0004-SM-58Q9A', 'GTEX-11ZUS-0011-R10b-SM-5BC79', 'GTEX-146FH-0002-SM-5DU3S', 'K-562-SM-47JYY', 'GTEX-1RLM8-0626-SM-DLT2O', 'GTEX-14A5H-1826-SM-5NQAB', 'GTEX-139T4-0426-SM-DK2JW', 'GTEX-QESD-0001-SM-2MRI9', 'GTEX-1269C-2026-SM-DK2JN', 'GTEX-11WQK-3126-SM-5EGI2', 'GTEX-148VJ-0002-SM-5DU4I', 'GTEX-12WSA-0011-R10a-SM-AHZMV', 'GTEX-1I1GT-1426-SM-A8N8M', 'GTEX-U3ZN-0001-SM-5URBO', 'GTEX-14PK6-0003-SM-5GYF3', 'GTEX-PWCY-0001-SM-2IJGL', 'GTEX-ZYT6-0002-SM-6WBS2', 'GTEX-12WSK-5005-SM-AN3Y3', 'GTEX-1IKOE-5004-SM-AN48B', 'GTEX-13FLV-0004-SM-5HJRE', 'GTEX-Y8E5-0001-SM-4YUWN', 'GTEX-R55F-1326-SM-2TWBM', 'GTEX-ZDTT-0002-SM-6UH3D', 'GTEX-XUJ4-1926-SM-4BOOV', 'GTEX-S33H-0526-SM-5SI83', 'GTEX-1RLM8-0626-SM-DKTS6', 'GTEX-1H3NZ-2626-SM-9WPOJ', 'GTEX-PX3G-0526-SM-2IZIB', 'GTEX-NL4W-0011-R11A-SM-2IZHM', 'GTEX-1JMQL-2926-SM-CXZJI', 'GTEX-13113-0003-SM-5ANGI', 'GTEX-QV31-0005-SM-2TO5P', 'GTEX-1O97I-0004-SM-DE1KT', 'GTEX-1GMR2-5001-SM-AN476', 'GTEX-1399Q-0004-SM-7DRP9', 'GTEX-12WSB-0126-SM-59HJN', 'GTEX-OOBK-0004-SM-2XK1M', 'GTEX-16BQI-0003-SM-6RBLB', 'GTEX-OXRK-0526-SM-2M47L', 'GTEX-S4P3-0226-SM-2XUAX', 'GTEX-P4QS-0003-SM-4X5ZO', 'GTEX-WH7G-0004-SM-3UZEX', 'K-562-SM-3LK7S', 'GTEX-14DAQ-0326-SM-CYKPR', 'GTEX-13PVR-0003-SM-5GYDY', 'GTEX-RTLS-0326-SM-2TWAO', 'GTEX-QEG4-1226-SM-2QU2X', 'GTEX-1AYD5-5013-SM-AN45H', 'GTEX-1117F-0003-SM-5DWSB', 'GTEX-14PHX-0001-SM-7DLGQ', 'GTEX-TMMY-0003-SM-5SOES', 'GTEX-P4PQ-2126-SM-3NMCV', 'GTEX-Q2AH-0726-SM-2IZHZ', 'GTEX-13FLW-0011-R10b-SM-AHZ6I', 'GTEX-RM2N-0526-SM-2TO66', 'GTEX-OIZH-0005-SM-2AXUD', 'GTEX-12WSB-0726-SM-5N9GD', 'GTEX-13G51-0004-SM-5CV8L', 'GTEX-R55C-0003-SM-2XK34', 'GTEX-QMR6-1926-SM-3L285', 'GTEX-Z93S-0011-R10a-SM-CYKR2', 'GTEX-15G19-2926-SM-CYKUE', 'GTEX-QDVN-2126-SM-2IZIR', 'K-562-SM-4EDPU', 'GTEX-15SKB-0002-SM-6RBL4', 'GTEX-14BMV-0326-SM-CYKOP', 'GTEX-S4Z8-1526-SM-4AD5B', 'GTEX-RM2N-0002-SM-4X62E', 'GTEX-T5JC-0011-R11A-SM-5S2RX', 'GTEX-ZLWG-0001-SM-53UB3', 'GTEX-1PAR6-0008-SM-DMZ2D', 'GTEX-ZC5H-0003-SM-53UAR', 'GTEX-WZTO-1326-SM-CYKUQ-ELUATE', 'GTEX-P4PP-0226-SM-2M479', 'GTEX-12KS4-0003-SM-7DRPS', 'GTEX-ZGAY-1526-SM-4WWEO', 'GTEX-15G1A-1526-SM-7KUMM', 'GTEX-OXRO-0006-SM-2D44O', 'GTEX-12WSL-0003-SM-6WSBO', 'GTEX-OHPL-2626-SM-DO3X3', 'GTEX-PSDG-0726-SM-2M48T', 'GTEX-1QCLZ-0001-SM-DLIQ3', 'K-562-SM-5LU8G', 'GTEX-N7MS-0926-SM-2AXU3', 'GTEX-11OF3-1326-SM-DMZ32', 'GTEX-S7SE-0011-R8A-SM-2XV6X', 'GTEX-19HZE-0326-SM-7189K', 'GTEX-1J8Q3-2326-SM-A8N8Y', 'K-562-SM-7KFRB', 'GTEX-NPJ8-0007-SM-26GMF', 'GTEX-ZQG8-2226-SM-5DUWD', 'GTEX-NPJ8-0007-SM-26GMW', 'GTEX-1AX8Y-1826-SM-731AU', 'GTEX-13112-0004-SM-5DWRX', 'GTEX-1AX9J-0004-SM-7MA68', 'GTEX-1C64O-0003-SM-DJWXT', 'GTEX-PWN1-1726-SM-2QU37', 'GTEX-15TU5-0626-SM-7KUFJ', 'GTEX-12WSB-3226-SM-59HJP', 'GTEX-N7MS-0011-R3a-SM-5SI8H', 'GTEX-RM2N-1826-SM-2TO4K', 'GTEX-15SHU-0001-SM-6UH2J', 'GTEX-O5YW-0006-SM-2D45A', 'GTEX-O5YV-0326-SM-2AXWH', 'GTEX-ZXES-0001-SM-6WBSZ', 'GTEX-S341-1826-SM-2XU9V', 'GTEX-T5JC-0002-SM-5SOER', 'GTEX-RNOR-0326-SM-2TWB6', 'GTEX-1RAZA-0002-SM-DLIQ6', 'GTEX-13W3W-0004-SM-6WSCQ', 'GTEX-1OFPY-0004-SM-DJWX5', 'GTEX-X15G-0002-SM-7DRQ4', 'GTEX-OIZG-0003-SM-2BWYZ', 'GTEX-OHPN-0011-R8A-SM-2I5FJ', 'GTEX-1HSMQ-0826-SM-A8N8H', 'GTEX-QEG4-0626-SM-2M487', 'GTEX-12WS9-0011-R5b-SM-5P9EV', 'GTEX-UPK5-0002-SM-3G3SL', 'GTEX-1J8QM-0004-SM-G8OXQ', 'GTEX-15CHQ-0002-SM-6RBKS', 'GTEX-16XZZ-0001-SM-6UH1U', 'GTEX-QMRM-1726-SM-2TO4I', 'K-562-SM-9HWP8', 'GTEX-RVPU-0011-R8A-SM-2XU94', 'GTEX-1IKOE-5010-SM-AN489', 'GTEX-12WS9-0011-R6a-SM-5P9EW', 'GTEX-S7SF-0526-SM-2XUAI', 'GTEX-147F4-0011-R10b-SM-CYKQO', 'GTEX-OIZH-2626-SM-2HMJM', 'GTEX-ZV7C-0526-SM-51MRB', 'GTEX-1HSGN-0001-SM-G8OXV', 'GTEX-QDT8-0006-SM-2XV5I', 'GTEX-13OVK-0001-SM-6WSCS', 'K-562-SM-4LVKX', 'GTEX-1211K-5019-SM-AN3X7', 'GTEX-RUSQ-0006-SM-2TWBZ', 'GTEX-1313W-0004-SM-5DU5K', 'GTEX-XOTO-0011-R10B-SM-EZ6LM', 'GTEX-13FTZ-0004-SM-6WSAY', 'GTEX-1122O-0004-SM-58Q7O', 'K-562-SM-9HWP5', 'GTEX-16NFA-1326-SM-6M47K', 'GTEX-1HCUA-5004-SM-AN471', 'K-562-SM-9HWP6', 'GTEX-QVJO-0326-SM-2TO5B', 'K-562-SM-4B66B', 'GTEX-17F9E-0003-SM-6RBKR', 'GTEX-15TU5-2726-SM-6M47I', 'K-562-SM-6M461', 'GTEX-14BIN-0011-R10a-SM-CYKQL', 'GTEX-1H1DF-1326-SM-9WPPR', 'GTEX-12WSF-0002-SM-6WSCI', 'GTEX-ZLV1-1426-SM-4WWES', 'GTEX-11ONC-2326-SM-DOKK8', 'GTEX-QMRM-0726-SM-2M49B', 'GTEX-ZYWO-0626-SM-5E45K', 'GTEX-1GN73-0002-SM-7MA7C', 'GTEX-R53T-0003-SM-2RXEX', 'GTEX-XLM4-0011-R11A-SM-5S2SY', 'GTEX-S33H-0526-SM-2XCBK', 'GTEX-NL4W-0009-SM-2XK1I', 'GTEX-14PK6-5019-SM-AN432', 'K-562-SM-9HWPX', 'GTEX-QDVN-2426-SM-2QU5A', 'GTEX-QVUS-0001-SM-2XK2W', 'GTEX-16NFA-1026-SM-6M47O', 'GTEX-13FTW-5013-SM-AN3ZM', 'GTEX-14DAQ-0004-SM-7DROH', 'GTEX-QXCU-0004-SM-5URDS', 'GTEX-OXRO-0003-SM-3USSA', 'GTEX-144FL-0003-SM-6RBKQ', 'GTEX-1314G-5001-SM-AN3XN', 'GTEX-SIU8-0002-SM-325HY', 'GTEX-QVJO-1726-SM-2S1R3', 'GTEX-12WS9-0426-SM-59HJW', 'GTEX-13QJ3-0002-SM-6WSD6', 'GTEX-RVPV-0003-SM-3USSI', 'GTEX-17HG3-1526-SM-7MGWA', 'GTEX-QEG5-1126-SM-2M49J', 'GTEX-139UC-0011-R7a-SM-5IJCV', 'GTEX-11WQK-2126-SM-5EQME', 'GTEX-S7SF-0003-SM-5JK43', 'GTEX-WXYG-0001-SM-3PYKG', 'GTEX-1R9PN-0004-SM-DLIQC', 'GTEX-PWCY-0526-SM-5PNU8', 'GTEX-P44H-0326-SM-2XV4H', 'GTEX-1RNTQ-0003-SM-DE1IU', 'GTEX-13NZB-0004-SM-6WSBX', 'GTEX-RWS6-0004-SM-5JK3Z', 'GTEX-15DYW-2226-SM-6LPJ8', 'GTEX-S7SE-0926-SM-2XV63', 'GTEX-P4PP-0005-SM-2D45V', 'GTEX-18A66-0002-SM-DINAV', 'GTEX-R55F-1326-SM-5S2V4', 'GTEX-SUCS-0001-SM-3USRI', 'GTEX-YECK-0002-SM-6UH15', 'GTEX-1F75B-0011-R11a-SM-9OSXB', 'GTEX-QMR6-1926-SM-7DRQ1', 'GTEX-1ICLY-5004-SM-AN47W', 'GTEX-ZA64-0003-SM-4YUXC', 'GTEX-13OVG-0003-SM-5GYCC', 'GTEX-RVPV-0011-R5A-SM-2XUAC', 'GTEX-1A8FM-0004-SM-7MD4G', 'GTEX-PSDG-0226-SM-2M48X', 'GTEX-YJ89-0526-SM-CYKQA', 'GTEX-SNMC-1426-SM-2XV6L', 'GTEX-131XF-5004-SM-79OMG', 'GTEX-ZT9W-0226-SM-4YCCZ', 'GTEX-SIU7-0526-SM-2XV5V', 'GTEX-ZTX8-0004-SM-53UBH', 'GTEX-15ER7-0011-R10a-SM-AHZNC', 'GTEX-WHSE-1726-SM-3PYKB', 'GTEX-13W3W-0004-SM-5GYAC', 'GTEX-UTHO-0426-SM-3GAED', 'GTEX-RNOR-0926-SM-2TWCZ', 'GTEX-U3ZN-0001-SM-3UZF1', 'GTEX-S4Z8-0426-SM-3K2AH', 'GTEX-PWCY-1826-SM-2S1OK', 'GTEX-15SHV-2426-SM-6PAMF', 'GTEX-PSDG-1126-SM-2QU4R', 'GTEX-PLZ6-0003-SM-3UZFH', 'GTEX-S32W-0626-SM-5S2TC', 'GTEX-14XAO-0003-SM-7DLF1', 'GTEX-13OVI-2726-SM-5KM56', 'GTEX-XMD1-0626-SM-CYKP5', 'GTEX-Y8E4-0001-SM-4YUWB', 'GTEX-WHSB-0004-SM-3G3SZ', 'GTEX-17F98-0003-SM-6RBKG', 'GTEX-QDVN-2126-SM-2I3FR', 'GTEX-T5JC-0726-SM-5S2SP', 'GTEX-1I1CD-3126-SM-A8NA3', 'GTEX-1C2JI-2026-SM-7IGML', 'GTEX-15SHV-5019-SM-AN447', 'GTEX-T5JC-0002-SM-4X62I', 'GTEX-16XZY-5016-SM-AN457', 'GTEX-11EMC-0004-SM-6WBUA', 'GTEX-13FTY-0004-SM-5DU5P', 'GTEX-QEG4-0226-SM-2QU52', 'GTEX-VUSG-0002-SM-3G3SC', 'GTEX-SSA3-0001-SM-325HL', 'GTEX-S341-0001-SM-2XK2K', 'GTEX-13O61-5004-SM-AN3ZO', 'GTEX-TMKS-0002-SM-3G3S6', 'GTEX-RM2N-0006-SM-2TO4R', 'GTEX-13OW5-0126-SM-5MR3A', 'GTEX-ZV7C-1326-SM-5BC7G', 'GTEX-TKQ1-0002-SM-3G3TX', 'GTEX-SJXC-0726-SM-2XV6M', 'GTEX-11P82-1226-SM-5HL5P', 'GTEX-15CHS-0011-R3a-SM-6EU1E', 'GTEX-11ZVC-0004-SM-5DWTH', 'GTEX-13S86-0003-SM-5DU4X', 'GTEX-147GR-1226-SM-CYKOA', 'GTEX-NPJ7-0011-R11A-SM-2IZHX', 'GTEX-13NZB-5019-SM-AN3ZE', 'GTEX-12WS9-2926-SM-5EGL8', 'GTEX-U3ZM-0004-SM-5URC4', 'GTEX-1QP6S-0001-SM-DCXR3', 'GTEX-Y5LM-0002-SM-4YUV1', 'GTEX-RVPU-2226-SM-2XU9A', 'GTEX-P78B-0626-SM-2S1O5', 'GTEX-13OVJ-0011-R3b-SM-5P9H6', 'GTEX-1CB4E-0001-SM-DIND6', 'GTEX-P44H-0011-R11A-SM-2XV6S', 'K-562-SM-4MVNA', 'GTEX-XOTO-0011-R11B-SM-4B64O', 'GTEX-RVPU-0011-R3A-SM-2XUA9', 'GTEX-1H1DE-0002-SM-AG5YS', 'GTEX-15DCZ-1226-SM-6871P', 'GTEX-1LSNM-0004-SM-DE1KE', 'GTEX-15DCD-0011-R5a-SM-69LPC', 'GTEX-QV31-1026-SM-4R1K3', 'GTEX-19HZE-0126-SM-73KX7', 'GTEX-RTLS-0626-SM-2TWAT', 'GTEX-16YQH-0004-SM-6UH22', 'GTEX-1LVAO-0003-SM-DE1KQ', 'GTEX-1AX9J-5007-SM-AN464', 'GTEX-Q2AG-1026-SM-2HMJX', 'GTEX-12ZZY-0004-SM-5DWRV', 'GTEX-S4UY-0126-SM-3K2BB', 'GTEX-W5WG-0003-SM-3G3SH', 'GTEX-1ICG6-5001-SM-AN47I', 'GTEX-13111-0003-SM-5DWRJ', 'GTEX-12WSN-0004-SM-5DWRZ', 'GTEX-ZYVF-2126-SM-5E44N', 'GTEX-11GS4-0004-SM-6WBTL', 'GTEX-1IDJE-5001-SM-AN47L', 'K-562-SM-7DUEE', 'GTEX-145MI-0011-R7b-SM-5Q5AZ', 'K-562-SM-7MKF2', 'GTEX-1IKK5-1126-SM-AHZ2W', 'GTEX-18A7A-0002-SM-6RBLJ', 'GTEX-15DCZ-2226-SM-69LOB', 'GTEX-145MG-0002-SM-7DROJ', 'GTEX-12ZZY-0004-SM-6WSCC', 'GTEX-14A5I-0004-SM-7DRO1', 'GTEX-18D9B-5001-SM-AN45C', 'GTEX-P44H-0011-R1A-SM-2XV5Z', 'GTEX-1PBJI-1426-SM-EYYXG', 'K-562-SM-2S1RI', 'GTEX-1JK1U-5019-SM-AN488', 'GTEX-13SLW-1226-SM-CYKW6', 'GTEX-R55D-0626-SM-2TWBB', 'GTEX-P44H-0626-SM-2XV4S', 'GTEX-15SDE-0001-SM-6RBLS', 'GTEX-QVUS-0011-R4A-SM-2TWBU', 'GTEX-13X6J-0011-R10a-SM-AHZN5', 'GTEX-RWS6-0004-SM-2RXEY', 'GTEX-16MTA-0003-SM-6UH2R', 'GTEX-145MH-0001-SM-7DRP2', 'GTEX-S7SE-0726-SM-2XV5T', 'GTEX-QEG5-0626-SM-2QU47', 'GTEX-13NZ9-0004-SM-5DU3T', 'GTEX-WXYG-0001-SM-3PZ82', 'GTEX-XOTO-0008-SM-4GICE', 'GTEX-QDT8-0926-SM-325HN', 'GTEX-13FHP-0004-SM-6WSAX', 'GTEX-QV31-1126-SM-4R1K4', 'GTEX-1MJIX-0002-SM-DE1KW', 'GTEX-14DAR-5016-SM-AN42E', 'GTEX-ZYFC-0001-SM-5DWS7', 'GTEX-ZA64-0003-SM-6UH38', 'GTEX-13OVI-0003-SM-5HJQK', 'GTEX-14E6D-0326-SM-CYKP3', 'GTEX-ZTTD-0326-SM-57WFW', 'GTEX-1I1CD-0004-SM-AG5ZU', 'GTEX-1QCLY-0002-SM-DE1IV', 'GTEX-R55F-0011-R6A-SM-2TWCP', 'GTEX-NPJ7-0011-R2a-SM-2AXWC', 'GTEX-13113-1026-SM-5LZWM', 'GTEX-QLQW-0001-SM-2XK23', 'GTEX-UPIC-1026-SM-4IHLT', 'GTEX-R55F-0326-SM-7DRPK', 'GTEX-1A8FM-0004-SM-7J39J', 'GTEX-QMRM-0726-SM-2I5G8', 'GTEX-14E7W-0001-SM-5CV9I', 'GTEX-XUJ4-1826-SM-4BOOU', 'GTEX-1CB4J-0011-R7a-SM-9WG5O', 'GTEX-11EQ8-0001-SM-58Q82', 'GTEX-RVPV-0011-R11A-SM-2XU98', 'GTEX-133LE-0001-SM-7DRPC', 'GTEX-XUJ4-0001-SM-4YUW2', 'GTEX-ZDXO-0011-R10a-SM-AHZ7L', 'GTEX-1CB4J-0011-R3b-SM-9WG5Q', 'GTEX-WOFL-0003-SM-3PYKF', 'GTEX-16GPK-0526-SM-CYKU9-INPUT', 'GTEX-YECK-2626-101825-SM-EZ6LP', 'GTEX-ZEX8-0001-SM-6UH17', 'GTEX-11WQC-0011-R10a-SM-CYKQY', 'GTEX-13O3Q-0003-SM-5GYBP', 'GTEX-UPIC-0004-SM-3G3TC', 'GTEX-ZYWO-2726-SM-5NQ6N', 'GTEX-15UF7-0003-SM-6UH2Y', 'GTEX-SJXC-1126-SM-4DM6Y', 'GTEX-148VI-0002-SM-5GYBT', 'GTEX-1192W-0826-SM-5EGHE', 'GTEX-PLZ4-0926-SM-2QU4F', 'GTEX-PSDG-0426-SM-2QU46', 'GTEX-11NV4-0001-SM-58Q86', 'GTEX-WFON-2126-SM-5S2T9', 'GTEX-1AYCT-5001-SM-AN45K', 'GTEX-OXRN-0226-SM-2D44T', 'GTEX-145LV-5019-SM-AN41K', 'GTEX-13OVK-0001-SM-5HJQJ', 'K-562-SM-2TF4E', 'GTEX-11OF3-1326-SM-DLIPK', 'GTEX-12WSA-0011-R10b-SM-CYKPQ', 'GTEX-1477Z-0003-SM-6WSBA', 'GTEX-O5YW-0126-SM-2D45C', 'GTEX-1EWIQ-0002-SM-7MA78', 'GTEX-RM2N-0426-SM-2TO75', 'GTEX-NPJ8-1626-SM-2AXTZ', 'GTEX-145MO-1426-SM-5Q5BS', 'K-562-SM-5J1MB', 'GTEX-P4PP-0004-SM-2XJZY', 'GTEX-PX3G-0004-SM-5JK53', 'GTEX-OXRL-0326-SM-447BG', 'GTEX-133LE-0001-SM-5DWRY', 'GTEX-POMQ-0003-SM-2H2X7', 'GTEX-1F6RS-0004-SM-7MA74', 'GTEX-XMK1-0003-SM-53IVD', 'GTEX-1LC47-1526-SM-CY8IB', 'GTEX-RVPU-0011-R11A-SM-2XCAF', 'GTEX-XGQ4-0001-SM-53IUN', 'GTEX-13FTZ-0011-R10b-SM-CYKOZ', 'GTEX-QVJO-2526-SM-4R1KB', 'GTEX-QCQG-1826-SM-2QU2R', 'GTEX-QLQW-0001-SM-2MRID', 'GTEX-YEC3-0726-SM-5IFIW', 'GTEX-12WSB-0326-SM-5GCP4', 'GTEX-13N2G-0011-R10a-SM-4RGKC-humanPFCa', 'GTEX-13N2G-0011-R10a-SM-4RGKC-humanPFCb', 'GTEX-RVPU-0011-R1A-SM-2XCAI', 'GTEX-XUYS-0001-SM-53IVC', 'GTEX-14A6H-5019-SM-AN424', 'GTEX-1AX9K-5022-SM-AN45F', 'GTEX-1B8KZ-0003-SM-7MD4V', 'GTEX-OIZG-1326-SM-2AXUO', 'GTEX-15UF6-0001-SM-6RBL5', 'GTEX-ZG7Y-0001-SM-4YUVR', 'GTEX-1B98T-0926-SM-79ONY', 'GTEX-X8HC-0003-SM-3UZEU', 'GTEX-1117F-0011-R10a-SM-AHZ7F', 'GTEX-NPJ7-0009-SM-3USRM', 'GTEX-13RTJ-0003-SM-6WSCV', 'GTEX-147GR-0003-SM-5DU4P', 'GTEX-14C5O-0011-R10a-SM-AHZN3', 'GTEX-NL3H-0011-R10A-SM-2IZHY', 'GTEX-P4QT-0003-SM-5JK2P', 'GTEX-13CIG-0004-SM-6WSC9', 'GTEX-NL4W-0011-R6a-SM-2AXVS', 'GTEX-ZLV1-0003-SM-4YUVF', 'GTEX-ZQG8-0003-SM-6WBSG', 'GTEX-15CHQ-0011-R7b-SM-6AJAC', 'GTEX-17KNJ-0526-SM-7LTA8', 'GTEX-XYKS-0003-SM-5JK4D', 'GTEX-XPVG-0003-SM-6UH19', 'GTEX-1R9PM-0001-SM-DE1JJ', 'GTEX-17GQL-0003-SM-6UH25', 'GTEX-WZTO-1326-SM-CYKUQ-SUP', 'GTEX-RU72-0011-R1A-SM-2TWD2', 'GTEX-11EQ8-0126-SM-5EQ54', 'GTEX-XBEW-0001-SM-7DLG7', 'GTEX-ZYWO-1126-SM-5U3SH', 'GTEX-U3ZH-0326-SM-51MS7', 'GTEX-11P82-1426-SM-5BC5L', 'GTEX-OHPM-0326-SM-2HMKT', 'GTEX-183WM-0004-SM-6RBK2', 'GTEX-13FTY-0004-SM-5HJQG', 'GTEX-1HFI7-5004-SM-AN46U', 'GTEX-OHPN-0011-R5A-SM-2I5FF', 'GTEX-P44H-0011-R4A-SM-2XV64', 'GTEX-14PJO-0011-R10a-SM-CYKSN', 'GTEX-13VXU-0011-R10a-SM-AHZ6R', 'GTEX-X4XY-0002-SM-58Q91', 'GTEX-ZE9C-3126-SM-EZ6XV', 'GTEX-OHPJ-0726-SM-2YUNM', 'GTEX-N7MS-0011-R3a-SM-2HMKD', 'GTEX-U3ZG-0002-SM-5SOEN', 'GTEX-PWO3-0004-SM-4X622', 'GTEX-1LSNL-0226-SM-EVYCG', 'GTEX-QVJO-0011-R2A-SM-2TO61', 'K-562-SM-2TWBO', 'GTEX-13X6J-0002-SM-6WSAI', 'GTEX-1H2FU-0011-R11a-SM-CE6SB', 'GTEX-1KD5A-0001-SM-DE1KI', 'GTEX-Q2AG-0526-SM-2QU4W', 'GTEX-WY7C-0002-SM-5URBW', 'GTEX-ZT9W-2426-SM-57WFJ', 'GTEX-XLM4-0726-SM-5S2SH', 'GTEX-QCQG-0003-SM-5JK44', 'GTEX-1C6VQ-1426-SM-73KTW', 'GTEX-QLQ7-1026-SM-447BB', 'GTEX-18D9B-5010-SM-AN45B', 'GTEX-R55C-0526-SM-2TO6D', 'GTEX-15CHS-1426-SM-6871W', 'GTEX-1AX8Y-1226-SM-72D6Z', 'GTEX-1CAMQ-0003-SM-7MA6C', 'GTEX-15CHQ-0011-R10a-SM-AHZN1', 'K-562-SM-4VBPT', 'GTEX-XLM4-2326-SM-4TT2D', 'GTEX-QESD-0001-SM-2XK21', 'GTEX-Q2AG-0003-SM-4X61P', 'GTEX-145MG-0002-SM-5HJQB', 'GTEX-1F5PK-5001-SM-AN46O', 'GTEX-X585-0003-SM-5URCW', 'GTEX-ZQG8-1626-SM-5HL6H', 'GTEX-1128S-0001-SM-5DWTV', 'GTEX-RVPU-0011-R7A-SM-2XU9D', 'GTEX-1LC47-0002-SM-DJWWJ', 'GTEX-YJ89-0002-SM-4YUVL', 'GTEX-14E6D-0011-R10a-SM-AHZ76', 'GTEX-14LLW-0011-R10a-SM-AHZNO', 'K-562-SM-5LZU1', 'GTEX-1399R-5010-SM-AN3YF', 'GTEX-NL3H-0011-R2a-SM-2I3GG', 'GTEX-17F97-2726-SM-7IGQ4', 'GTEX-WFON-0003-SM-3G3T9', 'GTEX-14C38-5010-SM-AN428', 'GTEX-1LG7Z-0002-SM-DE1KD', 'GTEX-11DXW-0003-SM-58Q7S', 'GTEX-QVUS-0011-R10A-SM-2TWCC', 'GTEX-1B98T-1326-SM-7EPI7', 'GTEX-13NYB-0004-SM-6WSC1', 'GTEX-S32W-2326-SM-5S2S6', 'GTEX-17GQL-5004-SM-AN453', 'GTEX-XXEK-2226-SM-5S2SA', 'GTEX-13X6K-0003-SM-5DU4M', 'GTEX-12WS9-1426-SM-5FQT3', 'GTEX-117XS-0011-R10a-SM-AHZ7G', 'K-562-SM-5Q59U', 'GTEX-1117F-0003-SM-6WBT7', 'GTEX-QCQG-0003-SM-2XK19', 'K-562-SM-2TC6R', 'GTEX-11EQ9-0003-SM-5DWSP', 'GTEX-1HSMQ-5001-SM-AN47P', 'GTEX-S4Z8-0826-SM-4AD4J', 'GTEX-111CU-0003-SM-6WBUD', 'GTEX-PLZ5-0003-SM-2XK1R', 'GTEX-Y3IK-0004-SM-53UBR', 'GTEX-R3RS-0001-SM-2RXF4', 'GTEX-ZAKK-0001-SM-DK2JI', 'GTEX-YEC3-0626-SM-4YCFE', 'GTEX-13OVL-0003-SM-5HJRX', 'GTEX-OIZH-0003-SM-5JK41', 'GTEX-11UD2-1626-SM-5EQM3', 'GTEX-R55G-0003-SM-2XK1C', 'GTEX-S95S-0004-SM-4X613', 'GTEX-11ZUS-0003-SM-6WBU3', 'GTEX-144GL-0426-SM-DLIOC', 'GTEX-S7PM-0926-SM-CLTGC', 'GTEX-13SLW-0326-SM-CYKUV', 'GTEX-QLQ7-1926-SM-2QU3Q', 'GTEX-131XF-0004-SM-5DU3L', 'GTEX-1J1OQ-0002-SM-G8OXM', 'GTEX-RNOR-0011-R7A-SM-2TWCV', 'GTEX-RN64-0326-SM-2TO5K', 'GTEX-1MJK2-0004-SM-DJWY4', 'K-562-SM-4GICD', 'GTEX-PWN1-1626-SM-2QU4N', 'GTEX-12WSA-0001-SM-5ANGH', 'GTEX-S7SF-1826-SM-3K2AD', 'GTEX-XMD3-0003-SM-7DRQ7', 'GTEX-R55F-0326-SM-3G3TF', 'GTEX-13O3O-0003-SM-5HJQ9_1', 'GTEX-WRHU-0003-SM-3PZ7D', 'GTEX-ZEX8-0001-SM-4YUX4', 'GTEX-148VI-0002-SM-7DROU', 'GTEX-148VJ-0826-SM-CYKPD', 'GTEX-PVOW-0126-SM-2XV68', 'GTEX-QXCU-0226-SM-2TO52', 'GTEX-S341-0006-SM-2XUA1', 'GTEX-XMD2-0001-SM-4YUVJ', 'GTEX-139T8-0004-SM-5HJPE', 'GTEX-S4Q7-0126-SM-4AD5P', 'GTEX-NL3H-0009-SM-2XK1L', 'GTEX-1CAV2-1626-SM-7IGQ6', 'GTEX-13W46-0003-SM-5DU47', 'K-562-SM-72D4W', 'GTEX-POYW-1226-SM-5LU5O', 'K-562-SM-4OOR8', 'GTEX-111VG-0004-SM-5DWSX', 'GTEX-Q2AI-1726-SM-2QU4Z', 'GTEX-15SHV-5001-SM-AN446', 'GTEX-1H3VE-0426-SM-9WPOM', 'GTEX-14BIL-1126-SM-CYKT7', 'GTEX-POMQ-0003-SM-4X5ZM', 'GTEX-PW2O-1826-SM-2S1OA', 'GTEX-ZAKK-0001-SM-DLIO8', 'GTEX-1KANC-0004-SM-DE1KF', 'GTEX-ZLFU-1326-SM-4WWET', 'GTEX-NPJ8-0126-SM-2QU56', 'GTEX-PWCY-0005-SM-2I3E2', 'GTEX-131XF-5001-SM-AN3XR', 'GTEX-OXRL-0326-SM-2D44H', 'GTEX-OHPL-2626-SM-2AXUT', 'GTEX-P4QS-0526-SM-2D43T', 'GTEX-17MF6-0002-SM-6RBK9', 'GTEX-ZYWO-0526-SM-5E45I', 'GTEX-1J8Q2-0002-SM-G8OXT', 'GTEX-ZLV1-1326-SM-4WWER', 'GTEX-14JIY-0004-SM-5HX9Y', 'GTEX-145MF-0004-SM-5DU4B', 'GTEX-N7MT-1226-SM-2TC6K', 'GTEX-1K2DA-0002-SM-AYROF', 'GTEX-ZYWO-1726-SM-5E44T', 'GTEX-14BMV-0826-SM-CYKTX-SUP', 'GTEX-145MO-0001-SM-5HJRJ', 'GTEX-XMD1-0011-R10A-SM-CYKRE', 'K-562-SM-5KM2I', 'GTEX-1269C-2026-SM-DLIOA', 'GTEX-POYW-1226-SM-2XV5C', 'GTEX-OXRO-0011-R10a-SM-AHZ7U', 'GTEX-1S82Y-0004-SM-DJWXO', 'GTEX-12WSB-0011-R6b-SM-5GU6J', 'GTEX-QV31-1426-SM-2TO6Z', 'GTEX-1OKEX-0003-SM-DE1IK', 'GTEX-1F7RK-0004-SM-7MA6R', 'GTEX-NPJ8-0004-SM-5JK3D', 'GTEX-Q2AG-0011-R4A-SM-2IZK4', 'GTEX-1OJC3-0003-SM-DJWXZ', 'GTEX-S7SE-0011-R11A-SM-2XV71', 'GTEX-REY6-0004-SM-3UZFS', 'GTEX-11NV4-0001-SM-6WBTT', 'GTEX-ZDTT-2126-SM-5S2OJ', 'GTEX-15DCD-1426-SM-CYKTW', 'GTEX-139UC-2926-SM-5J1N6', 'GTEX-ZYWO-0426-SM-5SI9A', 'GTEX-1EN7A-2026-SM-EZ6Y5', 'GTEX-16XZZ-0526-SM-CYKUB-SUP', 'GTEX-1EH9U-0001-SM-7MA6P', 'GTEX-ZDTT-2126-SM-5S2O7', 'GTEX-XUJ4-2426-SM-4BOO3', 'GTEX-QESD-0426-SM-4R1JZ', 'GTEX-1CB4F-0826-SM-DK2JX', 'GTEX-PWOO-0926-SM-2IZI2', 'GTEX-S7SE-0004-SM-5JK4E', 'GTEX-WHSE-1726-SM-3PZ7X', 'GTEX-Q2AI-0326-SM-2IZI9', 'GTEX-1GF9W-1726-SM-9JGHY', 'GTEX-RVPU-0011-R10A-SM-2XCAH', 'GTEX-S33H-2226-SM-2XU99', 'GTEX-POMQ-2326-SM-2QU32', 'GTEX-1GTWX-0001-SM-7J3A5', 'GTEX-X4LF-0001-SM-3PYKO', 'GTEX-11ZUS-0011-R10b-SM-CYKOY', 'GTEX-1LVAM-0004-SM-DE1IQ', 'GTEX-1IDJE-0001-SM-DCCZG', 'GTEX-15CHS-0726-SM-686ZJ', 'GTEX-1LBAC-0004-SM-DJWYD', 'GTEX-WHSE-0526-SM-5HL79', 'GTEX-RVPV-0926-SM-47JXU', 'GTEX-O5YV-0004-SM-4X5ZE', 'GTEX-1JMPZ-2226-SM-CE6TE', 'GTEX-144FL-1126-SM-DLIPL', 'GTEX-11TTK-0002-SM-58Q8B', 'GTEX-S32W-0001-SM-3USSK', 'GTEX-R55F-1426-SM-2TF53', 'GTEX-OHPN-2726-SM-2AXWJ', 'GTEX-OHPL-0526-SM-2AXUW', 'GTEX-1HBPN-1326-SM-A8N9Q', 'GTEX-1GN1V-0002-SM-7J3B5', 'GTEX-XPVG-0003-SM-53IV7', 'GTEX-XXEK-0004-SM-5S2TO', 'GTEX-12WSA-0005-SM-5O9BM', 'GTEX-1R9K4-0002-SM-DE1JP', 'GTEX-1MJK2-0004-SM-DE1NJ', 'GTEX-1H2FU-0001-SM-AG5YK', 'GTEX-18465-0002-SM-DINCG', 'GTEX-R55F-1526-SM-2TF4X', 'GTEX-13OVG-5019-SM-AN3ZI', 'GTEX-15CHS-0826-SM-6ETZW', 'GTEX-S7SF-0006-SM-3K2B6', 'GTEX-RNOR-0826-SM-2TWBE', 'GTEX-QEG5-0326-SM-2QU3C', 'GTEX-1AX8Y-1026-SM-73KWL', 'GTEX-ZV7C-1326-SM-51MSH', 'GTEX-RU72-0326-SM-2TWCE', 'GTEX-U412-0001-SM-4X627', 'GTEX-11ILO-2226-SM-5A5L1', 'GTEX-1HSEH-0626-SM-A8NA1', 'GTEX-OHPK-0526-SM-2HMJB', 'GTEX-13CF3-0004-SM-5HJRC', 'GTEX-O5YW-0526-SM-2M47W', 'GTEX-QMR6-0011-R10a-SM-AHZ7X', 'GTEX-1I19N-0002-SM-AG61I', 'GTEX-14PJ5-5001-SM-AN42L', 'GTEX-14PJ3-0004-SM-7DROK', 'GTEX-11NUK-0004-SM-6WBTU', 'GTEX-Q2AH-0004-SM-2XK1W', 'GTEX-13X6I-0011-R1b-SM-DG7EQ-HP3_A', 'GTEX-SJXC-0003-SM-325AA', 'GTEX-S4P3-0006-SM-3K2AW', 'GTEX-RU72-2626-SM-4TT75', 'GTEX-13OVG-0003-SM-5HJR9', 'GTEX-TMKS-0005-SM-33OV7', 'GTEX-OHPN-0011-R2A-SM-2IZJ3', 'K-562-SM-9HWQG', 'GTEX-YEC3-0526-SM-5IFHN', 'GTEX-15CHS-0011-R3a-SM-6AJAE', 'GTEX-OIZF-0006-SM-2AXUB', 'GTEX-R55G-0826-SM-2TWCO', 'GTEX-11WQK-1226-SM-CYKV5', 'GTEX-WYBS-0126-SM-CLTFZ', 'GTEX-131XH-0003-SM-7DRPA', 'K-562-SM-5NQ6M', 'GTEX-NPJ8-0004-SM-3UZFD', 'GTEX-1LNCM-0004-SM-DJWYR', 'GTEX-YEC3-0004-SM-53IVL', 'GTEX-Y8DK-0002-SM-6UH1E', 'GTEX-14ICL-0004-SM-7DRNX', 'GTEX-1HSEH-0008-SM-DMZ2H', 'GTEX-1JMQJ-3026-SM-D5OVS', 'GTEX-WH7G-0004-SM-3PZ86', 'GTEX-QV31-0826-SM-4R1KR', 'GTEX-12WSC-0001-SM-5DWRP', 'K-562-SM-9HWQ9', 'K-562-SM-9HWQ7', 'GTEX-1MJK3-0003-SM-DJWWQ', 'GTEX-PWOO-2326-SM-2QU4C', 'GTEX-147F4-0003-SM-5GYA1', 'GTEX-P4QT-0003-SM-3USRQ', 'GTEX-111VG-0004-SM-58Q85', 'GTEX-QVUS-0011-R9A-SM-2TWAU', 'GTEX-S7SF-0003-SM-2XK2H', 'GTEX-13O3O-0003-SM-6WSCO', 'GTEX-OHPJ-0526-SM-2D45B', 'GTEX-OIZF-0626-SM-2I5GT', 'GTEX-11DXX-2326-SM-5Q5A2', 'GTEX-14A6H-1026-SM-5SIAZ', 'GTEX-11ZTS-0001-SM-58Q8K', 'GTEX-ZTPG-0003-SM-6WBSO', 'GTEX-139TT-0004-SM-5HJQ1', 'GTEX-RU72-1226-SM-2TWBD', 'GTEX-1F48J-0003-SM-7MA75', 'GTEX-ZDXO-0326-SM-CYKQS', 'GTEX-R55C-1826-SM-2TO5C', 'GTEX-QDT8-0426-SM-2XV4J', 'GTEX-12WSK-0003-SM-6WSBW', 'GTEX-PWOO-0926-SM-2I3EC', 'GTEX-XUJ4-0008-SM-4BOQI', 'GTEX-N7MS-0011-R6a-SM-2AXWD', 'GTEX-U4B1-0001-SM-3USQC', 'GTEX-13X6K-0003-SM-6WSAL', 'GTEX-1RAZQ-0001-SM-DE1JA', 'GTEX-OHPM-2626-SM-2HMJL', 'K-562-SM-7MXT1', 'GTEX-1F6RS-0004-SM-7J3AR', 'GTEX-S341-2026-SM-2XU8A', 'GTEX-13VXU-2726-SM-CYKUK', 'GTEX-WFG8-2326-SM-5S2UE', 'GTEX-Q2AI-1426-SM-2QU2Y', 'GTEX-11ILO-1526-SM-5A5KZ', 'GTEX-P4PQ-0003-SM-2XK1P', 'GTEX-OIZH-0126-SM-2D44Z', 'GTEX-1I1CD-2026-SM-A8NA4', 'GTEX-XBED-0004-SM-3USR4', 'GTEX-XOT4-0001-SM-4YUX9', 'GTEX-14E1K-5004-SM-AN425', 'GTEX-PWN1-0006-SM-2IZJQ', 'GTEX-QEG5-1126-SM-2I5GH', 'K-562-SM-CKZN7', 'GTEX-13SLW-0011-R10a-SM-AHZ71', 'GTEX-15G1A-0001-SM-7J38P', 'GTEX-13NYS-0426-SM-5MR4X', 'GTEX-16GPK-0526-SM-CYKU9-ELUATE', 'GTEX-POYW-1226-SM-CLTGA', 'GTEX-SIU8-0006-SM-2XV6N', 'GTEX-1PPH6-2726-SM-DKTRV', 'GTEX-1H4P4-0001-SM-AG5Y5', 'GTEX-R55G-0003-SM-2MRIE', 'K-562-SM-4BONS', 'GTEX-12ZZW-0926-SM-CYKRH', 'GTEX-11DZ1-0002-SM-58Q7R', 'GTEX-17KNJ-2226-SM-731CG', 'GTEX-14A6H-5004-SM-AN421', 'GTEX-1F88F-0003-SM-7MA6U', 'GTEX-17EVP-0011-R10a-SM-CYKU1', 'GTEX-SIU7-1826-SM-2XV6Z', 'GTEX-PX3G-0626-SM-EZ6LW', 'GTEX-145MH-0011-R10a-SM-AHZN7', 'GTEX-13X6I-0002-SM-5DU4A', 'GTEX-18QFQ-0003-SM-DLIOZ', 'GTEX-RNOR-0011-R1A-SM-2TF5D', 'GTEX-NPJ7-0009-SM-2XJZQ', 'GTEX-144GN-0226-SM-G8OXN', 'GTEX-1R9K5-0002-SM-DLIP5', 'GTEX-13OW7-0011-R10a-SM-AHZN6', 'GTEX-PVOW-1026-SM-5O9B9', 'GTEX-14LZ3-0011-R6a-SM-69LQM', 'GTEX-13OW7-0011-R10a-SM-CYKQU', 'GTEX-113JC-0001-SM-6WBTD', 'GTEX-X8HC-0003-SM-58Q8Y', 'GTEX-ZYVF-0004-SM-5DU5Y', 'GTEX-14BIM-0826-SM-5Q5BM', 'GTEX-1GZHY-0001-SM-7J3AC', 'GTEX-1N2DW-0003-SM-DJWYX', 'GTEX-OXRK-0226-SM-2YUN1', 'GTEX-PLZ5-0006-SM-5S2W5', 'GTEX-QLQ7-1726-SM-2M48G', 'GTEX-UPK5-0326-SM-5SI81', 'GTEX-1QPFJ-0004-SM-DCXR4', 'GTEX-14PJ4-0004-SM-5GYE3', 'GTEX-RM2N-0002-SM-2XK37', 'GTEX-1F6IF-0004-SM-7MA6V', 'GTEX-17HGU-0001-SM-6RBK5', 'GTEX-S3XE-1626-SM-3K2AJ', 'GTEX-ZVE1-0001-SM-6WBSP', 'GTEX-1R46S-0003-SM-DLIQ7', 'GTEX-OIZG-0726-SM-2I5GS', 'GTEX-13O3Q-0003-SM-5HJQV', 'GTEX-X4EP-0002-SM-5URCA', 'GTEX-15SHV-0002-SM-6RBL7', 'GTEX-13O1R-0002-SM-5HJP9', 'GTEX-1IDJC-0003-SM-DCD1A', 'GTEX-S4UY-0626-SM-3K2AO', 'GTEX-XUJ4-1026-SM-4BOPB', 'GTEX-14ASI-0001-SM-7DROX', 'GTEX-11ILO-0002-SM-5CV9V', 'GTEX-16NPX-0002-SM-6RBK7', 'K-562-SM-5CVL5', 'GTEX-X4EP-0011-R7B-SM-4PQZW', 'GTEX-POYW-1226-SM-2XK2C', 'GTEX-139UC-0011-R6b-SM-5K7Z1', 'GTEX-ZPU1-0426-SM-72D74', 'K-562-SM-7189B', 'GTEX-YBZK-0001-SM-DLIPP', 'GTEX-WHSE-3026-SM-CYKVL', 'GTEX-QDVJ-1826-SM-2QU2U', 'GTEX-X4EP-0011-R2B-SM-3P625', 'GTEX-PWO3-1626-SM-2IZJM', 'GTEX-139YR-0004-SM-6WSC7', 'GTEX-11DZ1-0002-SM-6WBTG', 'GTEX-ZAB5-1926-SM-5HL7O', 'GTEX-ZPIC-0004-SM-6WBST', 'GTEX-P4QR-0008-SM-48TE2', 'GTEX-ZAJG-0001-SM-4YUUW', 'GTEX-S32W-0726-SM-5S2T3', 'GTEX-ZU9S-0001-SM-5DU5V', 'GTEX-RUSQ-1926-SM-2XUA4', 'GTEX-1EX96-0926-SM-7MGXL', 'GTEX-13FXS-3126-SM-CYKQ2', 'GTEX-1F88F-0003-SM-7J3AI', 'GTEX-SJXC-0426-SM-2XV53', 'GTEX-S4Z8-1126-SM-4AD5C', 'GTEX-QEG4-1826-SM-4R1JN', 'GTEX-OIZF-0004-SM-3USRU', 'GTEX-WYBS-0126-SM-7DRP7', 'GTEX-12ZZY-0004-SM-5ANFV', 'GTEX-13YAN-1326-SM-DK2JK', 'GTEX-T8EM-0001-SM-5SOEA', 'GTEX-14ICK-5007-SM-AN42K', 'GTEX-S3XE-0003-SM-2XK2I', 'GTEX-13QJC-0003-SM-6WSAQ', 'GTEX-WQUQ-0002-SM-4X62H', 'GTEX-12WS9-0326-SM-59HJV', 'GTEX-REY6-1026-SM-2TO6G', 'GTEX-1F75W-0126-SM-9KNVT', 'K-562-SM-4SOHT', 'GTEX-Q2AG-1026-SM-2IZK1', 'GTEX-1IGQW-0002-SM-G8OXH', 'GTEX-1S82Z-0003-SM-DE1J2', 'GTEX-1IDJD-0826-SM-A8N9H', 'GTEX-O5YW-0003-SM-4X5ZF', 'GTEX-QEG5-0003-SM-2MRI7', 'GTEX-18464-0004-SM-DJWXS', 'GTEX-QVJO-0011-R10A-SM-2TO6Y', 'K-562-SM-6PAL1', 'GTEX-T6MN-0002-SM-5S2TI', 'GTEX-TKQ2-0002-SM-3USRK', 'GTEX-ZVE1-0001-SM-4YUX8', 'GTEX-1OZHM-0003-SM-DJWYU', 'GTEX-17KNJ-2226-SM-7LG6W', 'GTEX-1445S-1326-SM-DLIOE', 'GTEX-NPJ8-2626-SM-2D43I', 'GTEX-OHPJ-1026-SM-2HMLE', 'GTEX-17GQL-1126-SM-7KFSN', 'GTEX-S4UY-0003-SM-2VCTU', 'GTEX-XOTO-2926-SM-4B65G', 'GTEX-14C39-0011-R10b-SM-5YYBE', 'GTEX-13OW6-0011-R1a-SM-DG7EP-HP2_A', 'GTEX-13OW6-0011-R1a-SM-DG7EP-HP2_B', 'GTEX-1IDJI-1226-SM-A8N9J', 'GTEX-S32W-0004-SM-2XUAW', 'GTEX-13X6I-0011-R1b-SM-DG7EQ-HP3_B_united', 'GTEX-15DCD-0011-R11a-SM-6AJAF', 'GTEX-ZZ64-0004-SM-5DWSA', 'GTEX-PX3G-1826-SM-2QU3P', 'GTEX-145ME-1326-SM-EZ6Y4', 'GTEX-RNOR-0011-R4A-SM-2TWCS', 'GTEX-1F75I-5004-SM-AN46I', 'GTEX-S4Z8-0003-SM-5JK3I', 'GTEX-U3ZM-0004-SM-3USQ6', 'GTEX-WHSE-0926-SM-CYKVI-INPUT', 'K-562-SM-5IJDX', 'GTEX-RUSQ-1726-SM-2TWCJ', 'GTEX-13RTJ-0003-SM-5GY9V', 'GTEX-RNOR-0005-SM-2TWBH', 'GTEX-ZVZP-2026-SM-57WB3', 'GTEX-ZZPT-0011-R10b-SM-CYKQ7', 'GTEX-P4PQ-0003-SM-5JK3Q', 'GTEX-11DYG-0004-SM-58Q7K', 'GTEX-OHPN-0011-R9A-SM-4DXUH', 'GTEX-XMK1-0003-SM-5URB9', 'GTEX-1LGRB-0003-SM-DE1N8', 'GTEX-WL46-1226-SM-4OOS4', 'GTEX-U3ZH-0003-SM-3UZF3', 'GTEX-16NFA-0526-SM-7KULM', 'GTEX-U3ZN-0001-SM-3G3TS', 'GTEX-S4Z8-0326-SM-2XUAS', 'GTEX-RVPU-0001-SM-4X62C', 'GTEX-15SHU-0011-R10a-SM-AHZNA', 'GTEX-PLZ5-0726-SM-2M48O', 'GTEX-1I6K7-5004-SM-AN47E', 'GTEX-131XG-0003-SM-7DRPV', 'GTEX-13IVO-0011-R10a-SM-AHZMS', 'GTEX-WVLH-0011-R10A-SM-CYKW4-INPUT', 'GTEX-1OJC4-0001-SM-DE1IE', 'GTEX-1EWIQ-0002-SM-7J3AN', 'GTEX-14PJN-5016-SM-AN42P', 'GTEX-S4P3-1826-SM-2XU8N', 'GTEX-X3Y1-0002-SM-3USRG', 'GTEX-Y3I4-0004-SM-6UH1C', 'GTEX-1HSMP-1926-SM-A8N8A', 'GTEX-1KANB-0003-SM-DJWWC', 'GTEX-Q734-0001-SM-3UZFB', 'GTEX-11GSO-0003-SM-6WBTR', 'GTEX-1OFPY-0326-SM-E76OI', 'GTEX-1HSKV-0003-SM-AG5ZV', 'GTEX-ZT9X-0002-SM-6WBS9', 'GTEX-1128S-0001-SM-58Q96', 'GTEX-13OVH-0003-SM-5CV8F', 'GTEX-15RIG-0003-SM-6UH2M', 'GTEX-13113-0003-SM-5DWRH', 'GTEX-TKQ1-0002-SM-6WBRZ', 'GTEX-13QJ3-0002-SM-5HJRS_1', 'GTEX-1JMPY-0003-SM-DE1KK', 'GTEX-18465-0002-SM-DJWXW', 'GTEX-R55F-0126-SM-48FCK', 'GTEX-OIZI-0426-SM-2XV52', 'GTEX-RNOR-0011-R5A-SM-2TWD3', 'GTEX-X4LF-0001-SM-3USSN', 'GTEX-R55G-0003-SM-5URBQ', 'GTEX-1QCLY-0002-SM-DLIQA', 'GTEX-O5YV-0006-SM-2AXVG', 'GTEX-QV31-0426-SM-447C4', 'GTEX-11PRG-0011-R10b-SM-CYKQG', 'GTEX-WHSB-0004-SM-5URDE', 'GTEX-13SLX-1026-SM-CYKVQ', 'GTEX-1F88E-2126-SM-DLL3I', 'GTEX-1H1DF-0526-SM-9JGG5', 'GTEX-P44H-0011-R5A-SM-2XV6B', 'GTEX-1B98T-0526-SM-7EPHN', 'GTEX-13O21-0003-SM-6WSB5', 'GTEX-OXRK-0326-SM-2M47J', 'GTEX-145MH-0326-SM-CYKUI', 'GTEX-14PHW-0002-SM-7DROQ', 'GTEX-1KANB-0003-SM-DE1KX', 'GTEX-QXCU-0326-SM-2TO68', 'GTEX-YFC4-0126-101855-SM-5CVLZ', 'GTEX-ZV68-2326-SM-CYKSC', 'GTEX-ZVT4-0004-SM-4YUVU', 'GTEX-1F75A-0001-SM-7J3AT', 'GTEX-12WSB-0001-SM-7DRQC', 'GTEX-WZTO-0002-SM-5URDU', 'GTEX-13VXU-0004-SM-6WSAS', 'GTEX-OXRL-0126-SM-2M47I', 'GTEX-14PHY-0002-SM-7DLFV', 'GTEX-QEG5-0526-SM-2S1PC', 'K-562-SM-7EPG4', 'GTEX-1314G-0826-SM-5BC6L', 'GTEX-144GN-5001-SM-AN3ZY', 'GTEX-SNMC-0004-SM-3259Z', 'GTEX-16YQH-0926-SM-CYKU2', 'GTEX-X261-0011-R10B-SM-CYKUO', 'GTEX-NPJ8-2526-SM-2AXTX', 'GTEX-ZDXO-0001-SM-53UBX', 'GTEX-13OVG-5004-SM-AN3ZG', 'GTEX-Z93S-1326-SM-CYKV7', 'GTEX-13FHO-1726-SM-5IFGM', 'K-562-SM-48TE3', 'GTEX-13FXS-0011-R10a-SM-AHZ6T', 'GTEX-1N2EF-0002-SM-DJWYQ', 'GTEX-VUSG-2526-SM-5S2S9', 'GTEX-XUJ4-1626-SM-4BONV', 'GTEX-T6MO-0001-SM-5URCH', 'GTEX-NL4W-0011-R8a-SM-2AXVI', 'K-562-SM-4T8CH', 'GTEX-QEG4-0726-SM-4R1JL', 'GTEX-1GMRU-0004-SM-7J38O', 'GTEX-ZVZO-0001-SM-5DWS4', 'GTEX-QV44-1526-SM-4R1KI', 'GTEX-ZT9X-0002-SM-53UB8', 'GTEX-NL3H-0011-R7a-SM-2AXVL', 'GTEX-14BIM-0002-SM-5DU5L', 'GTEX-XOTO-0626-SM-EYYVN', 'GTEX-N7MS-2425-SM-2D43D', 'GTEX-1QP9N-0004-SM-DLIP4', 'GTEX-S32W-0626-SM-2XUA5', 'GTEX-UPIC-1426-SM-4IHLQ', 'GTEX-R53T-0005-SM-2TO74', 'GTEX-139UC-0726-SM-DK2JS', 'GTEX-1269C-0011-R4b-SM-5P9ES', 'GTEX-WRHK-0003-SM-3PZ7S', 'GTEX-16NFA-2526-SM-7KUFT', 'GTEX-15CHS-0011-R7a-SM-69LQL', 'GTEX-13113-5007-SM-793CD', 'GTEX-14LLW-2826-SM-62LE1', 'GTEX-QVJO-0002-SM-3USRR', 'GTEX-145MH-0001-SM-5HJR6', 'GTEX-WVJS-0003-SM-DINCI', 'GTEX-WWYW-0004-SM-3PYKH', 'GTEX-1F5PK-5019-SM-AN46Q', 'GTEX-OXRO-0011-R9A-SM-5S2RR', 'GTEX-Q2AI-0003-SM-3UZFI', 'GTEX-WFON-0326-SM-5S2TP', 'GTEX-PLZ5-2026-SM-2M48B', 'GTEX-RVPU-2326-SM-2XU8D', 'GTEX-1AX9I-0002-SM-7MD4H', 'K-562-SM-5DUV6', 'GTEX-11ILO-0006-SM-5LZWE', 'GTEX-NL3H-0011-R5a-SM-2AXVW', 'GTEX-11O72-1726-SM-DLIP2', 'GTEX-QMRM-0002-SM-5URC1', 'GTEX-OHPN-0011-R4A-SM-2IZJ5', 'GTEX-T6MN-0003-SM-3USRL', 'GTEX-14LZ3-1826-SM-5YY8Y', 'GTEX-OXRK-0426-SM-2M47N', 'GTEX-S4Z8-1026-SM-3K2AI', 'GTEX-14PKV-0002-SM-6RBLM', 'GTEX-OHPN-0011-R9A-SM-2YUMN', 'GTEX-13X6H-0001-SM-5HJR3', 'GTEX-12696-0003-SM-5DWTK', 'GTEX-YJ8O-1026-SM-DMZ3B', 'GTEX-RNOR-0726-SM-2TWBF', 'GTEX-UPIC-0926-SM-4IHLV', 'GTEX-1HCU7-0003-SM-AG5YC', 'GTEX-PWCY-0001-SM-5JK3J', 'GTEX-111CU-0003-SM-58Q95', 'GTEX-13X6J-0002-SM-5HJRQ_1', 'GTEX-T5JW-0002-SM-3UZEZ', 'GTEX-Q2AG-0003-SM-2IJGG', 'GTEX-S7SE-0011-R4A-SM-2XV6H', 'GTEX-117XS-3026-SM-CYKOR', 'GTEX-P44H-0426-SM-2XV5L', 'GTEX-ZDTS-1326-SM-6M462', 'GTEX-13FTZ-0004-SM-5HJPA', 'GTEX-N7MT-0011-R6a-SM-2IZIT', 'GTEX-OXRO-0003-SM-2H2X6', 'GTEX-1HT8W-2926-SM-A8NA9', 'GTEX-12ZZX-2926-SM-CYKV2', 'GTEX-POMQ-0003-SM-5JK46', 'GTEX-PWOO-0426-SM-48TZF', 'GTEX-1GN1W-5001-SM-AN46R', 'GTEX-R55F-1226-SM-2TF59', 'GTEX-17EVQ-0011-R10a-SM-AHZM3', 'GTEX-1IDJU-5016-SM-AN47Z', 'GTEX-11LCK-0002-SM-5DWRA', 'GTEX-11GSO-0003-SM-58Q84', 'GTEX-1HSMQ-5010-SM-AN47Q', 'GTEX-QEL4-0526-SM-2TWAR', 'GTEX-P44G-2526-SM-2D44S', 'GTEX-1CAMQ-0003-SM-7J39C', 'GTEX-13NZ8-0004-SM-5DU4L', 'GTEX-13FLV-0926-SM-CYKRL', 'GTEX-1O97I-0004-SM-DJWYL', 'GTEX-QDT8-0526-SM-2XV5Q', 'GTEX-SIU7-1426-SM-4BRWT', 'GTEX-1399Q-0004-SM-5DWRI', 'GTEX-1B98T-3126-SM-73KV2', 'GTEX-PLZ4-0003-SM-2IJGB', 'GTEX-1192X-0011-R10a-SM-CYKSL', 'GTEX-12ZZZ-0002-SM-5ANGD', 'GTEX-12126-0001-SM-5DWTE', 'GTEX-14LZ3-0011-R10a-SM-AHZNP', 'GTEX-1CB4J-0004-SM-7MA67', 'GTEX-1IKJJ-0004-SM-G8OX2', 'GTEX-Y111-0001-SM-53IVB', 'GTEX-QV44-1126-SM-4R1KO', 'GTEX-1LC46-5004-SM-AN494', 'GTEX-12WSG-0001-SM-5DWRF', 'GTEX-ZG7Y-0001-SM-53UAX', 'GTEX-1JN76-0004-SM-DJWYP', 'GTEX-OHPK-0003-SM-2XJZU', 'GTEX-14PQA-1126-SM-CYKTQ', 'GTEX-PWOO-0526-SM-2S1Q3', 'GTEX-Y114-0004-SM-53IUE', 'GTEX-RVPU-0005-SM-2XU97', 'GTEX-R3RS-0726-SM-2TO5R', 'GTEX-XQ3S-0002-SM-53IVU', 'GTEX-SIU7-1726-SM-2XCDZ', 'GTEX-P4QR-0004-SM-2XK1O', 'K-562-SM-5IJB7', 'GTEX-QDT8-0006-SM-5SI8N', 'GTEX-S7SF-0526-SM-3K2BC', 'GTEX-1H1ZS-0002-SM-AG5YJ', 'GTEX-1NUQO-0003-SM-DJWXG', 'GTEX-13QJ3-0002-SM-5DU5T', 'GTEX-15UKP-0003-SM-6RBJ5', 'GTEX-1JKYN-0004-SM-ANILP', 'GTEX-RU72-0004-SM-2RXF6', 'GTEX-SIU7-0426-SM-2XV6U', 'GTEX-15DDE-0001-SM-7DLFL', 'GTEX-RU1J-0126-SM-2TWBN', 'GTEX-YB5K-0426-SM-5LUA4', 'GTEX-11ILO-0426-SM-5HL4V', 'GTEX-11TUW-0004-SM-6WBU1', 'GTEX-VUSG-0006-SM-5S2TA', 'GTEX-16XZZ-0526-SM-CYKUB-INPUT', 'GTEX-V955-0001-SM-3G3SX', 'K-562-SM-43V9A', 'GTEX-1QP29-0004-SM-DE1JL', 'GTEX-1J8EW-2926-SM-A8N9A', 'GTEX-1IDJD-0726-SM-A8N9G', 'GTEX-139UC-1426-SM-5IJGH', 'GTEX-11ILO-2626-SM-DLIPH', 'GTEX-R45C-0003-SM-2RXEW', 'GTEX-T6MN-0226-SM-5S2SO', 'GTEX-S32W-2526-SM-2XU8Y', 'GTEX-15CHS-0001-SM-7DLFY', 'GTEX-QVJO-0011-R9A-SM-2TO5M', 'GTEX-1KANA-0003-SM-DE1N5', 'GTEX-1HR9M-0003-SM-DHP29', 'GTEX-13OW6-0002-SM-5HJPU', 'GTEX-S32W-0326-SM-5S2T4', 'GTEX-P4PP-0004-SM-2H2WW', 'GTEX-P4QR-0006-SM-2M49O', 'K-562-SM-6LLH2', 'GTEX-S3XE-0003-SM-3UZFM', 'GTEX-16NFA-1626-SM-7KUES', 'GTEX-ZTTD-0002-SM-4YUV5', 'GTEX-1HSEH-1226-SM-A96TX', 'GTEX-1CB4J-0011-R5a-SM-9WG5Y', 'GTEX-ZYY3-0002-SM-58Q9G', 'GTEX-RNOR-0126-SM-2TWAZ', 'GTEX-14C39-2026-SM-664OG', 'GTEX-QMRM-1626-SM-4R1KV', 'GTEX-145ME-0526-SM-5QGQV', 'GTEX-14BMV-0002-SM-5CV8E', 'GTEX-ZYWO-2526-SM-5GIDM', 'GTEX-147GR-0011-R10b-SM-CYKQJ', 'GTEX-X5EB-0001-SM-58Q75', 'GTEX-REY6-0004-SM-5URD2', 'GTEX-NPJ8-0011-R5a-SM-2AXW3', 'GTEX-1CB4E-0001-SM-DLIOY', 'GTEX-13N11-1226-SM-5KM4D', 'GTEX-OXRO-1726-SM-2D45U', 'GTEX-VUSG-0426-SM-5S2TB', 'GTEX-1EH9U-0001-SM-7J3AD', 'GTEX-1CAV2-0003-SM-DIND8', 'K-562-SM-48FEU', 'GTEX-TKQ2-0002-SM-325AD', 'GTEX-144GM-0226-SM-DLL3K', 'GTEX-1B97J-0003-SM-7J39S', 'GTEX-S95S-1426-SM-2XV5A', 'GTEX-Y8LW-0004-SM-53IVS', 'GTEX-14C38-5004-SM-AN42A', 'GTEX-147JS-0003-SM-5DU46', 'GTEX-ZP4G-0002-SM-6WBSI', 'GTEX-RU72-0011-R10A-SM-2TF6D', 'GTEX-ZQG8-0003-SM-53UC1', 'GTEX-14PQA-0526-SM-CYKUF', 'GTEX-OOBK-0526-SM-2D43Q', 'GTEX-SSA3-0001-SM-3USRH', 'GTEX-QXCU-0006-SM-2TO4H', 'GTEX-1F6I4-0004-SM-7J3AG', 'GTEX-OHPN-0004-SM-5JK2V', 'GTEX-13PVQ-0003-SM-7DRPY', 'GTEX-1IDJE-5016-SM-AN47N', 'GTEX-132QS-0003-SM-7DROZ', 'GTEX-1HSMQ-0004-SM-AG5ZW', 'GTEX-1I1GT-0226-SM-A8N8N', 'GTEX-14JIY-0004-SM-7DRNU', 'GTEX-OHPJ-1626-SM-2D458', 'GTEX-NL3G-2526-SM-4SOIE', 'GTEX-WRHU-0003-SM-CLTFY', 'GTEX-XMD1-0004-SM-4YUWX_1', 'GTEX-1399U-0004-SM-5HJQ5', 'GTEX-ZVZP-0003-SM-53UB7', 'GTEX-14XAO-0003-SM-6RBJA', 'GTEX-12ZZZ-0002-SM-6WSCB', 'GTEX-Q2AI-0526-SM-2IZI8', 'GTEX-12WSM-0011-R10a-SM-CYKRA', 'GTEX-12WSG-5016-SM-AN3XC', 'GTEX-PWO3-0005-SM-2I5FS', 'GTEX-S4Q7-0006-SM-3K2AT', 'GTEX-145LV-2426-SM-E9WQW', 'GTEX-14AS3-5016-SM-AN41Y', 'GTEX-14DAR-0004-SM-5CV9S', 'GTEX-1399T-2526-SM-DLIPI', 'GTEX-Z93S-0011-R10a-SM-AHZNS', 'GTEX-S32W-0006-SM-2XU8J', 'GTEX-13NZB-5001-SM-AN3ZC', 'GTEX-1K9T9-5010-SM-AN48G', 'GTEX-144GM-5016-SM-AN41A', 'GTEX-S7PM-0926-SM-5JK36', 'GTEX-QV44-0003-SM-2XK2A', 'GTEX-ZYT6-1026-SM-731CL', 'GTEX-XLM4-0003-SM-4YUW8', 'GTEX-RVPU-0011-R2A-SM-2XU9N', 'GTEX-13G51-0426-SM-CYKUU', 'GTEX-X5EB-0001-SM-5URCX', 'GTEX-X15G-0002-SM-3USR6', 'GTEX-13OVH-0003-SM-6WSCP', 'GTEX-14DAQ-0011-R10a-SM-AHZNB', 'GTEX-1HGF4-0003-SM-AG5YE', 'GTEX-S7SF-0226-SM-2XUAJ', 'GTEX-RNOR-0126-SM-2TF57', 'GTEX-1R46S-0003-SM-DE1JG', 'GTEX-1JMQI-0002-SM-DE1NG', 'GTEX-NPJ7-0009-SM-5JK4M', 'GTEX-NPJ8-1826-SM-2QU3L', 'K-562-SM-4IHK7', 'GTEX-NL4W-0011-R3a-SM-2AXWB', 'GTEX-1HCUA-5016-SM-AN472', 'GTEX-XUZC-0002-SM-5JK3R', 'GTEX-XK95-0726-SM-DMZ3A', 'GTEX-RU1J-0003-SM-2XV76', 'GTEX-X15G-0002-SM-3PZ7J', 'GTEX-1R9JW-0005-SM-DLIPE', 'GTEX-XMD2-0001-SM-7DLGS', 'GTEX-X4EP-0011-R9B-SM-4QASI', 'GTEX-15SZO-0011-R10a-SM-AHZN9', 'GTEX-NFK9-0626-SM-2AXU6', 'GTEX-13PL6-0002-SM-5GYAR', 'GTEX-P44H-0226-SM-2XV67', 'K-562-SM-4SOLN', 'GTEX-S3LF-0001-SM-DINCJ', 'GTEX-UPIC-0726-SM-3GADW', 'GTEX-POYW-1226-SM-5LZWQ', 'GTEX-11UD1-0001-SM-6WBTY', 'GTEX-WVJS-2726-SM-4MVNU', 'GTEX-14PJ6-5004-SM-AN42W', 'GTEX-U4B1-0001-SM-5URC2', 'K-562-SM-E9EYO', 'K-562-SM-E9EYN', 'K-562-SM-E9EYP', 'K-562-SM-E9EYV', 'K-562-SM-E9EYU', 'K-562-SM-E9EYT', 'K-562-SM-E9EYZ', 'GTEX-Y114-0004-SM-6UH1A', 'GTEX-13RTJ-0003-SM-5HJQ6', 'GTEX-P78B-0004-SM-5JK48', 'GTEX-1R7EV-0002-SM-DE1JB', 'GTEX-ZYWO-2326-SM-5GZY4', 'GTEX-15DCD-0011-R3b-SM-69LOY', 'GTEX-132NY-0001-SM-7DRPZ', 'GTEX-13X6I-0002-SM-6WSAR', 'GTEX-1HCVE-3126-SM-A8N8E', 'GTEX-NL3H-0009-SM-4X5ZJ', 'GTEX-ZV7C-0002-SM-53UC4', 'GTEX-11VI4-0003-SM-5DWTX', 'GTEX-ZV6S-0002-SM-53UC5', 'GTEX-NPJ7-0011-R8a-SM-2AXVB', 'GTEX-ZVZP-0003-SM-4YUWJ', 'GTEX-15DCE-0003-SM-6RBJM', 'GTEX-OXRN-1526-SM-2D44J', 'GTEX-XUJ4-1126-SM-4BOPC', 'GTEX-UPIC-1826-SM-4IHKC', 'GTEX-1KXAM-5013-SM-AN492', 'GTEX-OOBK-1626-SM-2D43R', 'GTEX-O5YV-2026-SM-26GMK', 'GTEX-PVOW-2526-SM-2XV6F', 'K-562-SM-9HWOY', 'GTEX-NPJ7-0011-R10A-SM-2IZHU', 'GTEX-T5JC-0002-SM-325A9', 'K-562-SM-9HWOP', 'K-562-SM-9HWOQ', 'K-562-SM-9HWOR', 'GTEX-ZAB5-0004-SM-4YUWZ', 'GTEX-14JFF-0002-SM-DLIOQ', 'GTEX-12WSN-2326-SM-5DUVG', 'K-562-SM-9HWOK', 'GTEX-1GF9U-5019-SM-AN42N', 'GTEX-OIZH-1826-SM-2QU54', 'GTEX-RWS6-2326-SM-2XU8O', 'GTEX-16NPV-0011-R10a-SM-CYKT6', 'K-562-SM-9HWOD', 'GTEX-1LC47-0002-SM-DE1NI', 'GTEX-WFON-0005-SM-5S2RV', 'GTEX-OHPM-0003-SM-2XK1K', 'GTEX-15F5U-0926-SM-6LPKB', 'GTEX-1K9T9-1226-SM-EVR4R', 'GTEX-1399R-1826-SM-E9IIE', 'GTEX-1GL5R-0002-SM-7J3A9', 'GTEX-S33H-0626-SM-2XUAP', 'K-562-SM-2I3E7', 'K-562-SM-9HWO3', 'GTEX-P78B-0004-SM-2XK12', 'GTEX-RM2N-1426-SM-2TO53', 'GTEX-145MG-0226-SM-CYKOM', 'GTEX-ZEX8-0001-SM-53UBO', 'GTEX-OXRP-0526-SM-2D441', 'GTEX-1AX8Y-0126-SM-731FE', 'GTEX-1QP67-0004-SM-DE1JC', 'GTEX-16NFA-0826-SM-6LPJD', 'GTEX-QEL4-0326-SM-2TWAX', 'GTEX-15CHS-0526-SM-6EU2D', 'GTEX-17MFQ-1326-SM-7KFRL', 'GTEX-RU72-0926-SM-2TWAP', 'GTEX-RU1J-0002-SM-2RXF5', 'GTEX-POMQ-0003-SM-2IJGF', 'GTEX-12ZZZ-0002-SM-5DWRU', 'GTEX-S4P3-0006-SM-2XU8F', 'GTEX-1EKGG-0004-SM-DLIOP', 'GTEX-14PJN-0002-SM-5GYEC', 'GTEX-1R7EU-0003-SM-DLIQ4', 'GTEX-NPJ8-0626-SM-48TBO', 'GTEX-139YR-5019-SM-AN3YT', 'GTEX-TSE9-0002-SM-5URBA', 'GTEX-WOFM-0003-SM-3PYK9', 'GTEX-X88G-0003-SM-53IVK', 'GTEX-T5JC-0526-SM-5S2SD', 'GTEX-11EM3-0004-SM-6WBUK', 'GTEX-11P7K-0002-SM-6WBUI', 'GTEX-14XAO-5016-SM-AN43S', 'K-562-SM-33HC7', 'GTEX-R53T-0526-SM-2TO76', 'GTEX-S4P3-0004-SM-2XK2G', 'GTEX-ZLWG-0001-SM-6WBS7', 'GTEX-15SKB-0002-SM-6UH2I', 'GTEX-15DCZ-2426-SM-6871N', 'GTEX-11I78-0002-SM-6WBU2', 'GTEX-1I1GV-0004-SM-AG5YW', 'GTEX-XYKS-0003-SM-53IUK', 'GTEX-QEG4-1826-SM-447AK', 'GTEX-1NT2E-0826-SM-DMZ2L', 'GTEX-PW2O-1926-SM-2QU39', 'GTEX-PWN1-0006-SM-2S1NG', 'GTEX-ZPCL-0001-SM-53UAU', 'GTEX-P78B-0005-SM-2M49N', 'GTEX-1R9PN-0004-SM-DE1JI', 'GTEX-14LLW-0002-SM-7DRNS', 'GTEX-16NFA-0726-SM-6LPJC', 'GTEX-14PKV-0002-SM-7DLG9', 'GTEX-17HII-0004-SM-6UH1Z', 'GTEX-OIZH-0003-SM-3USSF', 'GTEX-SIU7-0006-SM-2XCE6', 'GTEX-13PDP-2926-SM-5N9E8', 'GTEX-117XS-0001-SM-6WBTA', 'GTEX-1J8QM-1526-SM-A8N9C', 'GTEX-OHPL-0326-SM-2AXV1', 'GTEX-OHPM-0326-SM-2AXUU', 'GTEX-1AX8Y-2226-SM-72D51', 'GTEX-1192W-0001-SM-5DWSN', 'GTEX-139UC-0011-R3b-SM-5K7VZ', 'GTEX-PWCY-0526-SM-2IZIG', 'GTEX-S341-1826-SM-3K2AB', 'GTEX-14PJO-0526-SM-CYKOD', 'GTEX-14PN4-0001-SM-7DRP6', 'GTEX-11DXW-0003-SM-5DWSM', 'GTEX-1F88E-2126-SM-DK2JJ', 'GTEX-OIZH-0226-SM-2AXUJ', 'GTEX-14JG6-5013-SM-AN42G', 'GTEX-X4XY-0002-SM-3PZ7Q', 'GTEX-QVJO-1926-SM-2TO5L', 'GTEX-ZQUD-1726-SM-51MRQ', 'GTEX-16XZY-0002-SM-6UH26', 'GTEX-1QPFJ-1826-SM-EWRMS', 'GTEX-11ZU8-0001-SM-DLIPQ', 'GTEX-ZP4G-2426-SM-57WEY', 'GTEX-S7SF-0001-SM-3K2BE', 'GTEX-14C39-0004-SM-5CV93', 'GTEX-11NUK-0004-SM-5DWSZ', 'GTEX-R55E-0003-SM-2XK28', 'GTEX-1OKEX-2026-SM-E8VNL', 'K-562-SM-4RGL3', 'GTEX-17KNJ-5004-SM-AN44T', 'GTEX-S3XE-0426-SM-3K2AC', 'GTEX-PW2O-1526-SM-2IZJD', 'GTEX-Y8DK-0002-SM-4YUVY', 'GTEX-NPJ8-0011-R10A-SM-2QU5B', 'GTEX-R55E-0003-SM-3UZFL', 'GTEX-131XW-0002-SM-5GYA5', 'GTEX-NL3G-0010-SM-7DRPG', 'GTEX-P4QS-0005-SM-2D447', 'GTEX-13113-5001-SM-AN3XM', 'GTEX-139UC-0726-SM-DLIOD', 'GTEX-12WSF-0002-SM-5IX76', 'GTEX-QDVN-1526-SM-2IZJP', 'GTEX-QCQG-2126-SM-2S1P8', 'GTEX-S7SE-0011-R1A-SM-2XV6O', 'GTEX-12WSD-0011-R8b-SM-9QEGI', 'GTEX-QMRM-0002-SM-3UZFK', 'GTEX-1A32A-0001-SM-7J3A1', 'GTEX-XOTO-0011-R8A-SM-4B65J', 'GTEX-11DXZ-0003-SM-5DWSR', 'GTEX-P44G-0926-SM-325AM', 'GTEX-RNOR-0011-R1A-SM-2TWD5', 'GTEX-12ZZX-0011-R10a-SM-AHZMR', 'GTEX-POYW-0006-SM-2XV6I', 'GTEX-NL3H-0011-R2a-SM-2AXWF', 'GTEX-NL4W-0011-R10a-SM-AHZ7W', 'GTEX-11DXZ-0003-SM-CLTG6', 'GTEX-1EU9M-0003-SM-7J3AO', 'GTEX-13N1W-0002-SM-DO3XO', 'GTEX-1B8KE-5016-SM-AN462', 'GTEX-S7SE-0004-SM-325HK', 'GTEX-14PHX-0001-SM-5GYEB', 'GTEX-UPK5-0002-SM-4SVR1', 'GTEX-OHPL-1626-SM-2AXUX', 'GTEX-13JUV-0001-SM-6WSB6', 'GTEX-1RAZQ-0001-SM-DLIQ1', 'GTEX-NPJ8-2626-SM-26GMZ', 'GTEX-QEG5-0226-SM-2I5GI', 'GTEX-12WSH-0001-SM-5DWRM', 'GTEX-NPJ8-2626-SM-26GMI', 'GTEX-1S5VW-0001-SM-DJWXN', 'GTEX-146FQ-0003-SM-5GYCE', 'GTEX-XOTO-0426-SM-4B66A', 'GTEX-14ABY-1126-SM-CYKOT', 'GTEX-11TT1-0001-SM-6WBTZ', 'GTEX-1PIGE-0001-SM-DE1IG', 'GTEX-S33H-2326-SM-2XU8P', 'GTEX-U3ZM-0004-SM-3G3SF', 'GTEX-OHPL-0003-SM-5JK38', 'GTEX-QEL4-0826-SM-DMRPX', 'GTEX-11DXW-1126-SM-CYKSD', 'GTEX-VJWN-0003-SM-3G3T5', 'GTEX-16Z82-0004-SM-6UH1S', 'GTEX-13SLX-0011-R10a-SM-AHZ6P', 'GTEX-13OVJ-0001-SM-5HJR8', 'GTEX-1NT2E-0826-SM-DO3XN', 'GTEX-Y8E4-0001-SM-53IV3', 'GTEX-1NSGN-0001-SM-DE1JW', 'GTEX-1J8Q3-2226-SM-A8N8X', 'GTEX-QVUS-0011-R3A-SM-2TWCR', 'GTEX-1QEPI-0001-SM-DLIPF', 'GTEX-N7MT-0126-SM-2D43B', 'GTEX-13VXT-0003-SM-5GY9Y', 'GTEX-QLQW-0005-SM-2QU4U', 'GTEX-13N2G-0011-R5a-SM-5MR33', 'GTEX-17MFQ-0003-SM-6UH24', 'GTEX-1B98T-1126-SM-7EPHO', 'GTEX-YEC4-1026-SM-5P9FW', 'GTEX-NPJ8-0007-SM-2D43F', 'GTEX-15TU5-0004-SM-6RBKZ', 'GTEX-Z9EW-0002-SM-53UAK', 'GTEX-ZYFD-1526-SM-5NQ7T', 'GTEX-15CHC-0001-SM-6RBJF', 'GTEX-RVPU-0011-R6A-SM-2XCAC', 'GTEX-16GPK-0011-R10b-SM-CYKU7', 'GTEX-14JFF-0011-R10a-SM-AHZNL', 'GTEX-XOTO-0006-SM-4B64T', 'GTEX-14PJ6-0001-SM-5GYEE', 'GTEX-11O72-1726-SM-DMZ2Y', 'GTEX-SJXC-0003-SM-5JK2U', 'GTEX-OXRK-0004-SM-3USRO', 'GTEX-1JMQL-0004-SM-DE1KB', 'GTEX-1LSNM-0004-SM-DJWY7', 'GTEX-RN5K-0001-SM-DLIOT', 'GTEX-11I78-0002-SM-58Q8G', 'GTEX-13113-1726-SM-5GCOO', 'GTEX-QVJO-1626-SM-2TO5E', 'K-562-SM-4DM4W', 'GTEX-ZQG8-0003-SM-4YUWH', 'GTEX-1FIGZ-0005-SM-A8N7O', 'GTEX-S3XE-0006-SM-3K2AA', 'GTEX-V1D1-0001-SM-3G3TH', 'GTEX-1I6K6-1226-SM-A8N8P', 'GTEX-WZTO-0002-SM-3PZ84', 'GTEX-13QBU-5004-SM-AN41B', 'GTEX-15F5U-1926-SM-DK2JV', 'GTEX-SIU7-0002-SM-2XK2N', 'GTEX-16GPK-1126-SM-CYKSS', 'GTEX-NFK9-1026-SM-2AXU9', 'GTEX-1HBPN-1926-SM-A8N9S', 'GTEX-16YQH-0011-R10b-SM-CYKU8', 'GTEX-N7MT-0009-SM-5JK4X', 'GTEX-QV31-0726-SM-2TO6J', 'GTEX-131YS-0003-SM-5DU4F', 'GTEX-16XZZ-0926-SM-CYKP1', 'GTEX-OHPM-0126-SM-2QU3F', 'GTEX-ZUA1-0001-SM-53UBK', 'GTEX-PWN1-0004-SM-4X62D', 'GTEX-131XF-0326-SM-5DUVR', 'GTEX-S7SF-2226-SM-3K2BG', 'GTEX-P44G-0526-SM-DMZ38', 'GTEX-X5EB-1526-SM-4E3JC', 'GTEX-1JKYN-0004-SM-AYROE', 'GTEX-NL4W-0011-R2a-SM-2AXV4', 'GTEX-1H1E6-0001-SM-AG5YO', 'GTEX-OXRN-1226-SM-2D44Q', 'GTEX-XPVG-0003-SM-4YUWK', 'GTEX-1MCQQ-0002-SM-DE1K2', 'K-562-SM-2XCE8', 'GTEX-N7MS-0009-SM-3USRT', 'GTEX-14C5O-0002-SM-7DRQF', 'GTEX-13PL7-0003-SM-5GYAN', 'GTEX-1KXAM-0004-SM-DJWX8', 'GTEX-1IY9M-0004-SM-G8OXJ', 'GTEX-NPJ8-0004-SM-2XK1F', 'GTEX-18D9B-1026-SM-7KFS8', 'GTEX-12WSG-2826-SM-5FQUS', 'GTEX-S4Z8-1526-SM-D4P48', 'GTEX-1S831-0002-SM-DJWYA', 'GTEX-1HSKV-0004-SM-DHP2F', 'GTEX-15CHS-0226-SM-6871L', 'GTEX-16NGA-0004-SM-6RBKH', 'GTEX-SNMC-0004-SM-325HH', 'GTEX-12WS9-1526-SM-5FQT4', 'GTEX-ZDXO-0001-SM-4YUWF', 'GTEX-Z93T-0002-SM-5DWTP', 'GTEX-1C4CL-0003-SM-7MA6F', 'GTEX-ZLV1-0003-SM-53UAT', 'GTEX-147F3-5001-SM-AN41M', 'GTEX-UPIC-0526-SM-4WWCQ', 'GTEX-14A6H-2826-SM-5QGPM', 'GTEX-NFK9-0426-SM-2QU4B', 'GTEX-1RAZR-0003-SM-DLIQL', 'GTEX-UJMC-0002-SM-3G3TU', 'GTEX-S3XE-2026-SM-3K2B5', 'GTEX-12WSE-0002-SM-6WSCA', 'GTEX-13QBU-5016-SM-AN41D', 'GTEX-1445S-1326-SM-DK2JU', 'GTEX-R55E-0126-SM-2TO6U', 'GTEX-SE5C-0526-SM-2XV6Y', 'GTEX-XUJ4-2626-SM-4BOQ3', 'GTEX-RVPV-1226-SM-2XUAG', 'GTEX-T6MN-2026-SM-5S2T6', 'GTEX-1212Z-0004-SM-7DRQE', 'GTEX-Y111-0001-SM-6UH1B', 'GTEX-1F7RK-0004-SM-7J3AF', 'GTEX-PLZ4-0226-SM-2M482', 'GTEX-13D11-0003-SM-5JVBH', 'GTEX-1C475-0003-SM-7MA7G', 'GTEX-RVPU-0011-R6A-SM-2XU9I', 'GTEX-15SB6-5001-SM-AN44H', 'GTEX-SUCS-0001-SM-325HM', 'GTEX-QMRM-0002-SM-2MRIF', 'GTEX-16NGA-0004-SM-6UH29', 'GTEX-OHPN-0011-R7A-SM-2IZJ8', 'GTEX-111FC-1426-SM-5N9C7', 'GTEX-N7MT-1126-SM-2YUNQ', 'GTEX-NFK9-0004-SM-2BWYG', 'GTEX-O5YV-1826-SM-2QU4A', 'GTEX-OHPK-0003-SM-3USRN', 'GTEX-14A6H-0003-SM-7DROY', 'GTEX-QV44-0526-SM-2TO51', 'GTEX-13CF2-0004-SM-5HJPZ', 'GTEX-YEC3-2526-SM-5CVLS', 'GTEX-REY6-0826-SM-2TO4M', 'GTEX-ZZPT-0011-R10a-SM-AHZO1', 'GTEX-ZWKS-0002-SM-5JVBF', 'GTEX-1M5QR-5007-SM-AN49D', 'GTEX-RVPU-0005-SM-2TF6L', 'GTEX-OHPN-0011-R3A-SM-2I5FC', 'GTEX-S7SE-0005-SM-2XV5X', 'GTEX-13FHP-0726-SM-CYKPV', 'GTEX-OOBJ-0006-SM-2D44R', 'GTEX-PWCY-0005-SM-2IZHR', 'GTEX-1F75A-0001-SM-7MA76', 'GTEX-TMKS-0002-SM-DINCQ', 'GTEX-QDVN-0003-SM-2IJGQ', 'GTEX-1A3MX-0002-SM-7MA69', 'GTEX-R55D-0426-SM-2TWB1', 'GTEX-13OW8-2426-SM-5L3ID', 'GTEX-PLZ6-0626-SM-5K7WJ', 'GTEX-R55F-0005-SM-2TF4W', 'GTEX-11ZTS-0001-SM-6WBU5', 'GTEX-QV44-0826-SM-2TO6M', 'GTEX-13O3Q-0011-R1b-SM-4W9GN-hHP3b', 'GTEX-1GZ4I-2126-SM-9JGG4', 'GTEX-13OVH-3026-SM-CYKS1', 'GTEX-12WSB-0011-R7b-SM-5GU6V', 'GTEX-X585-0003-SM-7FG6K', 'GTEX-13IVO-0011-R11b-SM-5LZXQ', 'GTEX-1AX9J-0004-SM-7J38U', 'GTEX-13NYC-0004-SM-6WSB9', 'GTEX-1OKEX-0003-SM-DJWXH', 'GTEX-ZPU1-2826-SM-57WEU', 'GTEX-T2YK-0003-SM-325HR', 'GTEX-13FHO-0003-SM-6WSAZ', 'GTEX-P4QR-0004-SM-2H2X5', 'GTEX-13CZV-0011-R10b-SM-CYKS6', 'GTEX-XPT6-0004-SM-5URDL', 'GTEX-POMQ-0006-SM-33HBY', 'GTEX-PLZ6-1526-SM-2QU3D', 'GTEX-T6MO-0001-SM-325I3', 'GTEX-SIU7-0001-SM-3NMAW', 'GTEX-QDVN-0726-SM-2IZIP', 'GTEX-OIZF-0004-SM-5JK4C', 'GTEX-111FC-0001-SM-58Q7U', 'GTEX-13N11-0003-SM-5JVB5', 'GTEX-15EOM-1926-SM-69LQ2', 'GTEX-ZF28-0002-SM-6UH3K', 'GTEX-RVPU-0011-R4A-SM-2TF6X', 'GTEX-X4XX-0726-SM-4QARS', 'GTEX-SIU8-0326-SM-2XV4V', 'GTEX-RWS6-0005-SM-2XU9T', 'GTEX-T6MN-0926-SM-5S2S3', 'GTEX-1RNSC-0011-R11b-SM-EXUTG', 'GTEX-XUJ4-1326-SM-4BOQ9', 'GTEX-P44H-0011-R3A-SM-2XV6J', 'GTEX-OHPL-0526-SM-2HMIX', 'GTEX-139TU-0926-SM-DLIOG', 'GTEX-ZT9X-0002-SM-4YUX6', 'GTEX-14BIL-0011-R10a-SM-CYKOB', 'GTEX-16NFA-1526-SM-7KULS', 'GTEX-1B97J-0003-SM-7MD4P', 'K-562-SM-2M47Q', 'GTEX-PLZ4-0226-SM-5SI7M', 'GTEX-RVPU-0011-R11A-SM-2XU9W', 'GTEX-1JMPZ-0004-SM-DJWX4', 'GTEX-139UC-3026-SM-5IJE4', 'GTEX-N7MS-0007-SM-26GME', 'GTEX-13OVJ-1026-SM-5IFGI', 'GTEX-113JC-0001-SM-58Q7M', 'GTEX-OHPJ-0726-SM-2M47S', 'K-562-SM-79OM9', 'GTEX-ZVP2-0002-SM-6WBSN', 'GTEX-N7MS-0007-SM-26GMV', 'GTEX-QCQG-0006-SM-5SI8M', 'GTEX-11ONC-1326-SM-DLIPJ', 'GTEX-145MG-0011-R10a-SM-AHZ7D', 'GTEX-TSE9-0002-SM-3G3TB', 'GTEX-OHPK-0006-SM-2D45H', 'GTEX-QESD-0626-SM-2M497', 'GTEX-ZVE2-0002-SM-4YUV6', 'GTEX-1LVAN-0004-SM-AYROD', 'GTEX-11NV4-0001-SM-5DWSY', 'GTEX-N7MT-0009-SM-3UZEY', 'GTEX-OOBJ-0003-SM-4X61D', 'GTEX-1QW4Y-0003-SM-DLIQP', 'GTEX-1MCC2-5017-SM-AN49G', 'GTEX-P4PP-1626-SM-2D43X', 'GTEX-OIZG-0005-SM-2HMJC', 'GTEX-1KWVE-0003-SM-DJWWN', 'GTEX-17F98-0003-SM-6UH28', 'GTEX-13IVO-0004-SM-5HJQQ', 'GTEX-NPJ7-1726-SM-2QU3I', 'GTEX-Y5V5-0002-SM-53IUG', 'GTEX-P78B-0004-SM-2MRIA', 'GTEX-RVPU-0011-R3A-SM-2XCAE', 'GTEX-13VXT-5007-SM-AN412', 'GTEX-QLQ7-0426-SM-4R1JU', 'GTEX-RNOR-0011-R3A-SM-2TF5J', 'GTEX-SJXC-0326-SM-2XV4O', 'GTEX-QVUS-0001-SM-5URDT', 'GTEX-1GF9W-0004-SM-7J3AL', 'GTEX-1B932-5004-SM-AN45R', 'GTEX-S7SF-0003-SM-2VCU4', 'GTEX-1B996-0003-SM-7J38R', 'GTEX-13O3O-0003-SM-5CV8I', 'GTEX-QXCU-0626-SM-2TO5N', 'GTEX-N7MS-2526-SM-2D43A', 'GTEX-RU1J-0002-SM-2XK2V', 'GTEX-14PJ5-0001-SM-7DROP', 'GTEX-13111-1826-SM-5GCP1', 'GTEX-1OFPY-0004-SM-DE1JV', 'GTEX-13113-0426-SM-5GCOD', 'GTEX-QEL4-0926-SM-2TWBY', 'GTEX-R55D-0926-SM-325I1', 'GTEX-13CF2-0004-SM-6WSB1', 'GTEX-S7SF-0426-SM-3K2B7', 'GTEX-S7SE-0011-R10A-SM-2XV6P', 'GTEX-14ASI-0526-SM-CYKNX', 'GTEX-QLQ7-0003-SM-2MRIB', 'GTEX-16MT9-0004-SM-6UH2T', 'GTEX-P4PQ-0003-SM-2H2WY', 'GTEX-ZLFU-0004-SM-4YUV4', 'GTEX-U8XE-0004-SM-3G3SU', 'GTEX-WCDI-0001-SM-3PZ7A', 'GTEX-1AX8Y-0926-SM-731EI', 'GTEX-15TU5-0426-SM-6PALT', 'GTEX-U412-0001-SM-3G3S1', 'GTEX-OHPM-0003-SM-2BWYB', 'GTEX-1JJE9-0003-SM-DE1KN', 'GTEX-T5JC-2026-SM-5S2TJ', 'GTEX-14PII-0004-SM-5GYE5', 'GTEX-S3XE-1926-SM-2XU8H', 'GTEX-131XF-1926-SM-5DUW4', 'GTEX-132QS-0003-SM-5DU49', 'GTEX-15ER7-3126-SM-CYKUH', 'GTEX-S4Q7-1426-SM-3K2B9', 'GTEX-1CB4J-2926-SM-7PBZ2', 'GTEX-P4QR-0326-SM-2S1NU', 'GTEX-ZV7C-1126-SM-5E45O', 'GTEX-U3ZG-0002-SM-4YUWL_1', 'GTEX-1C6WA-0003-SM-7J39H', 'GTEX-13U4I-0004-SM-5DU3P', 'GTEX-PLZ6-0426-SM-2I5FG', 'GTEX-13OW7-0001-SM-6WSD2', 'GTEX-NL3H-0011-R11A-SM-2IZHV', 'GTEX-1H3VY-0005-SM-A8N7Q', 'GTEX-PLZ5-0003-SM-5JK3C', 'GTEX-TMMY-0003-SM-4X62G', 'GTEX-12WSC-0001-SM-5ANGC', 'GTEX-R55F-1426-SM-2TWAS', 'GTEX-11TUW-0011-R3a-SM-DO92U', 'GTEX-TMMY-0926-SM-4WKH4', 'GTEX-1GF9U-0004-SM-7J3A8', 'K-562-SM-4U82S', 'GTEX-1NV8Z-0003-SM-DJWX2', 'GTEX-16MT9-0004-SM-6RBLG', 'GTEX-R45C-0926-SM-2TWCY', 'GTEX-R55C-0626-SM-2TO5I', 'GTEX-X261-0002-SM-5URDP', 'GTEX-UTHO-0011-R10a-SM-AHZNR', 'GTEX-OHPN-0011-R6A-SM-2IZJ7', 'GTEX-VUSH-0002-SM-3PZ7N', 'GTEX-13N2G-0011-R10a-SM-4RGKC-S8', 'GTEX-1L5NE-5001-SM-AN48J', 'GTEX-13N2G-0011-R10a-SM-4RGKC-S7', 'GTEX-1LBAC-0004-SM-DE1KR', 'GTEX-YB5E-0003-SM-53IUH', 'GTEX-1PBJJ-0004-SM-DE1II', 'GTEX-OHPM-0006-SM-2AXV8', 'GTEX-ZV68-0001-SM-53UBB', 'GTEX-Y3I4-0004-SM-4YUVP', 'GTEX-OOBJ-0003-SM-2XK2P', 'GTEX-RUSQ-1826-SM-2XU9Q', 'GTEX-13CZU-0003-SM-5HJRM_1', 'GTEX-1HCU6-0004-SM-AG5YG', 'GTEX-146FQ-1226-SM-5QGQ4', 'GTEX-13X6H-5001-SM-AN414', 'GTEX-X4LF-0001-SM-3PZ79', 'GTEX-11EQ9-0003-SM-6WBTK', 'GTEX-P4QT-1626-SM-2M47E', 'GTEX-1I19N-2326-SM-CE6SU', 'GTEX-1HCU7-5010-SM-AN477', 'GTEX-WFON-0003-SM-5URD4', 'GTEX-XUJ4-0226-SM-4BOP8', 'GTEX-13PVR-5016-SM-AN417', 'GTEX-Q2AG-0011-R6A-SM-2IZK8', 'GTEX-17EVQ-1526-SM-CYKTA', 'GTEX-ZG7Y-0001-SM-7FG68', 'GTEX-1H2FU-1926-SM-9WPO7', 'GTEX-1CB4I-0002-SM-DINB1', 'GTEX-1ICLY-2126-SM-A8N9E', 'GTEX-15EOM-0004-SM-6RBKT', 'GTEX-1NSGN-0001-SM-DJWYW', 'GTEX-117YW-2926-SM-5GIEQ', 'GTEX-WHSB-0326-SM-5GICB', 'GTEX-ZPU1-0826-SM-73KXI', 'GTEX-OXRO-0326-SM-2I5EE', 'GTEX-1E2YA-0002-SM-DIND7', 'GTEX-13CF3-0826-SM-5LZYZ', 'GTEX-1R9K4-0002-SM-DLIQI', 'GTEX-UPJH-0002-SM-5SOE9', 'GTEX-17KNJ-0003-SM-6UH1P', 'GTEX-ZVT4-0004-SM-6WBSR', 'GTEX-P4PP-0226-SM-2S1NN', 'GTEX-P78B-0726-SM-2M48A', 'GTEX-PLZ4-1326-SM-2QU2W', 'GTEX-1K9T9-5004-SM-AN48E', 'GTEX-SIU8-0002-SM-325AJ', 'GTEX-OHPJ-1326-SM-3MJHG', 'GTEX-Q2AG-0011-R2A-SM-2IZKB', 'GTEX-T5JC-0005-SM-5S2SQ', 'GTEX-13FHO-0003-SM-5DU53', 'GTEX-XUW1-0002-SM-53IVO', 'GTEX-12WSM-0001-SM-5DWS1', 'GTEX-RN64-0126-SM-2TO55', 'GTEX-RU72-3026-SM-2TF6C', 'GTEX-11ZUS-0126-SM-CYKR8', 'GTEX-ZQUD-1526-SM-51MRE', 'GTEX-ZYWO-0926-SM-5E43J', 'GTEX-QLQW-0001-SM-3USS3', 'GTEX-R3RS-0426-SM-2TO5J', 'GTEX-11ZVC-0004-SM-6WBU9', 'GTEX-YJ89-0011-R10a-SM-CYKPS', 'GTEX-R55D-0926-SM-325AK', 'GTEX-11EM3-0004-SM-58Q9C', 'GTEX-W5X1-0002-SM-5URBS', 'GTEX-11DXX-0004-SM-6WBT9', 'GTEX-12WS9-0001-SM-7DRQ3', 'GTEX-1QMI2-0004-SM-DLIQH', 'GTEX-NPJ8-0004-SM-CLTG5', 'GTEX-RN64-0526-SM-2TO4V', 'GTEX-RVPV-0226-SM-2XU9L', 'GTEX-19HZE-3026-SM-718BR', 'GTEX-1AMEY-0004-SM-7J39X', 'GTEX-ZF29-0004-SM-4YUX5', 'GTEX-145MN-5001-SM-AN41L', 'GTEX-1IDJF-5007-SM-AN47X', 'GTEX-YB5E-0003-SM-4YUVA', 'GTEX-P44H-2426-SM-2XV73', 'GTEX-1I4MK-0004-SM-DCCZN', 'GTEX-145ME-5001-SM-AN41C', 'GTEX-1MCQQ-0002-SM-DJWXD', 'GTEX-OXRK-0004-SM-2XJZV', 'GTEX-13111-0003-SM-5ANG9', 'GTEX-14C5O-1326-SM-CYKQM', 'GTEX-OHPN-0011-R9A-SM-2IZJL', 'GTEX-1LG7Y-0001-SM-DE1KH', 'GTEX-OHPK-1826-SM-2M47R', 'GTEX-13FLW-0003-SM-5HJRA', 'GTEX-18D9U-5010-SM-AN44Y', 'GTEX-PLZ5-0003-SM-3UZFG', 'GTEX-S3XE-1826-SM-2XU8Z', 'GTEX-1F75B-0011-R1b-SM-9QEIP', 'GTEX-1HC8U-1026-SM-A8N9Z', 'GTEX-11DXW-0011-R10a-SM-AHZNX', 'GTEX-1QP66-0002-SM-DCXS5', 'GTEX-1I1GU-1526-SM-A8N84', 'GTEX-1399S-2326-SM-DO3X1', 'GTEX-13FXS-0004-SM-6WSB8', 'GTEX-13PL7-0003-SM-5HJRW', 'GTEX-P44G-0226-SM-2I3F3', 'GTEX-14A5H-0011-R10a-SM-AHZ78', 'GTEX-16YQH-0426-SM-CYKTD', 'GTEX-1HBPN-0006-SM-A8N7X', 'GTEX-13CZU-0003-SM-5DU3K', 'GTEX-RNOR-0001-SM-3UZFO', 'GTEX-ZVZO-0001-SM-DLIPR', 'GTEX-QEG5-0926-SM-2M49I', 'GTEX-R55D-0226-SM-2TWBG', 'GTEX-16XZY-0002-SM-6RBKE', 'GTEX-QESD-1426-SM-2QU4L', 'GTEX-1AX8Y-0002-SM-7J38S', 'GTEX-1E2YA-0002-SM-DLIOW', 'GTEX-11NSD-0126-SM-DLIPG', 'GTEX-U4B1-0001-SM-3G3SS', 'GTEX-1I1GU-0004-SM-DHP2B', 'K-562-SM-686YS', 'GTEX-RVPU-2426-SM-CYKV9', 'GTEX-15EU6-0003-SM-7DLFZ', 'GTEX-QDVN-0003-SM-3UZFJ', 'GTEX-XOTO-0226-SM-4B66H', 'GTEX-15RIG-0003-SM-6RBL9', 'K-562-SM-4YCCQ', 'GTEX-1QP29-0004-SM-DLIQG', 'GTEX-13X6J-0002-SM-5DU4W', 'GTEX-P44H-0006-SM-2XV4L', 'GTEX-QDVN-0326-SM-2IZIS', 'GTEX-ZPIC-0004-SM-4YUVS', 'GTEX-XBEW-0001-SM-53IUT', 'GTEX-11TTK-0002-SM-6WBTX', 'K-562-SM-2S1OR', 'GTEX-1JK1U-0001-SM-DE1IM', 'GTEX-1CB4H-0004-SM-7MA5Z', 'GTEX-ZXES-0001-SM-5DWS2', 'GTEX-P44H-0011-R8A-SM-2XV5U', 'GTEX-1CB4G-0002-SM-DLIOL', 'GTEX-ZF28-0011-R10a-SM-AHZMI', 'GTEX-ZF3C-2326-SM-5S2MZ', 'GTEX-ZAJG-0011-R10a-SM-AHZMG', 'GTEX-UPIC-0004-SM-5SOEF', 'GTEX-PWO3-0011-R10a-SM-AHZ7S', 'GTEX-1N2EE-0002-SM-DJWWO', 'GTEX-O5YV-0526-SM-26GMJ', 'GTEX-QEL4-0726-SM-2TWCQ', 'GTEX-RN64-0004-SM-2XK29', 'GTEX-ZAB5-1226-SM-5P9FY', 'GTEX-R55E-0011-R11A-SM-2TO6C', 'GTEX-ZYFD-0001-SM-6WBT3', 'GTEX-1H1E6-0226-SM-A8N7Y', 'GTEX-QVUS-2826-SM-CYKP8', 'GTEX-XUJ4-2026-SM-4BOOW', 'GTEX-YEC3-2526-SM-4YCDN', 'GTEX-ZAJG-2926-SM-5HL8P', 'GTEX-WZTO-0011-R10B-SM-CYKUP', 'GTEX-1GZ2Q-0002-SM-7J3AK', 'GTEX-1I1HK-0001-SM-DC69N', 'GTEX-1B932-1826-SM-73KTU', 'GTEX-RVPU-0011-R7A-SM-2XCAB', 'GTEX-TMMY-0926-SM-4VBQZ', 'GTEX-ZDXO-0001-SM-6UH3F', 'GTEX-1F5PL-0002-SM-7J3AU', 'GTEX-OXRK-0226-SM-2M485', 'GTEX-S7SE-0011-R6A-SM-2XV62', 'GTEX-PWO3-0011-R1A-SM-2IZIW', 'GTEX-Y111-0001-SM-4YUXE', 'GTEX-1R9JW-0005-SM-DE1JN', 'GTEX-Q2AH-0004-SM-4X62O', 'GTEX-11ILO-0008-SM-5QGR9', 'GTEX-1C64N-0003-SM-7MD4X', 'GTEX-R55D-0926-SM-4X61C', 'K-562-SM-3GAFC', 'GTEX-T5JW-0002-SM-3G3SN', 'GTEX-WL46-0002-SM-5SOE8', 'GTEX-S4Z8-1026-SM-CM2RE', 'GTEX-QMRM-0126-SM-4R1K9', 'GTEX-X8HC-0003-SM-3PZ7F', 'GTEX-1LB8K-0003-SM-DJWWI', 'GTEX-NL3H-0011-R8a-SM-2YUNH', 'GTEX-1I19N-2926-SM-A8N8C', 'GTEX-15EO6-0004-SM-6RBJH', 'GTEX-XUYS-0001-SM-4YUWO', 'GTEX-1HBPI-0003-SM-AG5Y4', 'GTEX-1A8G6-2526-SM-72D59', 'GTEX-ZLWG-0726-SM-5S2NG', 'GTEX-WYVS-1926-SM-4SOIC', 'GTEX-1C6VQ-0003-SM-7MA5K', 'GTEX-15TU5-0626-SM-CYKUD', 'GTEX-X4EO-0003-SM-3PYKZ', 'GTEX-1GN1V-0011-R11b-SM-A8N9N', 'GTEX-NPJ7-0011-R6a-SM-2AXVN', 'GTEX-131XF-2126-SM-5DUWS', 'GTEX-13O3O-3126-SM-CYKUY', 'GTEX-O5YV-0006-SM-2I5GX', 'GTEX-13YAN-1326-SM-DLIPO', 'GTEX-SIU8-0526-SM-2XV5O', 'GTEX-1497J-0004-SM-7DRPQ', 'GTEX-R55E-0003-SM-2MRIC', 'GTEX-Y8E5-0001-SM-DLIPY', 'GTEX-XLM4-1826-SM-5S2SC', 'GTEX-QVJO-0011-R1A-SM-2TO64', 'K-562-SM-5IFHC', 'GTEX-1B98T-2126-SM-79OOG', 'GTEX-QLQW-1426-SM-2TO4N', 'GTEX-1L5NE-5016-SM-AN48K', 'GTEX-OXRN-1326-SM-CYKP9', 'GTEX-139UC-0011-R5b-SM-5K7VY', 'GTEX-ZDTS-0926-SM-4WAXG', 'GTEX-SIU8-0126-SM-2XV58', 'GTEX-R55C-0003-SM-2RXF1', 'GTEX-ZF29-0004-SM-53UBU', 'GTEX-15DCE-5007-SM-AN458', 'GTEX-1A8G6-0004-SM-7J39W', 'GTEX-TML8-0004-SM-325HI', 'GTEX-OXRP-0326-SM-2I5ET', 'GTEX-OXRK-0426-SM-2YUMT', 'GTEX-13IVO-0004-SM-6WSBI', 'GTEX-YEC4-0526-SM-CKJTE', 'GTEX-14PII-0004-SM-7DROO', 'GTEX-PLZ4-0003-SM-3UZF9', 'GTEX-13IVO-0626-SM-CYKS8', 'GTEX-1HSMO-0003-SM-AG611', 'GTEX-1MJIX-0002-SM-DJWYZ', 'GTEX-13CZV-0003-SM-5DU45', 'GTEX-T5JC-0001-SM-5S2S4', 'GTEX-WI4N-1326-SM-3PYKI', 'GTEX-1GN2E-0003-SM-7J3AB', 'GTEX-1K9T9-0004-SM-DE1N9', 'GTEX-QV44-1226-SM-4R1KE', 'GTEX-S7SF-2226-SM-2XU93', 'GTEX-1GZ4H-0001-SM-7J3B8', 'GTEX-1IDJC-2126-SM-ACKWU', 'GTEX-1HCU7-2926-SM-A8N9U', 'GTEX-1MA7X-0003-SM-DJWWD', 'GTEX-1AX8Y-1326-SM-72D4Y', 'GTEX-1RNSC-1626-SM-EVR4U', 'GTEX-ZTPG-0003-SM-53UAI', 'GTEX-1IDJV-2326-SM-A8N8S', 'GTEX-XUJ4-0326-SM-4BOP9', 'GTEX-11NSD-0126-SM-DMZ2Z', 'GTEX-13N2G-0001-SM-6WSC5', 'GTEX-ZV68-0001-SM-4YUWU', 'GTEX-N7MS-0009-SM-2BWY4', 'GTEX-N7MT-0011-R7a-SM-2AXV9', 'GTEX-OHPM-1826-SM-2QU3Z', 'GTEX-SNOS-0004-SM-325HU', 'GTEX-ZYVF-0626-SM-5E43Q', 'GTEX-14PJ2-0002-SM-7DROD', 'GTEX-131XF-2026-SM-5DUWG', 'GTEX-RU1J-0002-SM-3USSC', 'GTEX-1LB8K-0003-SM-DE1NH', 'GTEX-N7MS-0011-R11A-SM-2IZJZ', 'GTEX-S4Z8-0003-SM-4X62P', 'GTEX-1IL2U-2926-SM-A8N8V', 'GTEX-1J8Q3-0001-SM-G8OXO', 'GTEX-1J8Q3-5007-SM-AN484', 'GTEX-ZT9W-2626-SM-57WFV', 'GTEX-1K2DU-0926-SM-DHXKB', 'GTEX-15CHQ-0002-SM-7DLEY', 'GTEX-SN8G-0002-SM-5JK4B', 'GTEX-PWCY-0001-SM-2XK14', 'GTEX-S7SE-0004-SM-3USS8', 'GTEX-PLZ4-0003-SM-2XJZZ', 'GTEX-13OVH-0003-SM-5HJPW_1', 'GTEX-P44G-0526-SM-2D44L', 'GTEX-YBZK-0001-SM-5DWTQ', 'GTEX-11DXX-0004-SM-58Q7I', 'GTEX-QLQ7-0526-SM-2M496', 'GTEX-RNOR-0011-R9A-SM-2TWB8', 'GTEX-14C38-0003-SM-7DRO3', 'GTEX-1B98T-0004-SM-7MA63', 'GTEX-15DCD-0011-R10a-SM-AHZN4', 'GTEX-ZV68-0626-SM-CYKQW', 'GTEX-VJYA-0003-SM-4X5ZU', 'GTEX-ZVT4-0004-SM-53UB1', 'GTEX-1IDJF-5004-SM-AN47Y', 'GTEX-14BIM-0011-R10a-SM-AHZ7A', 'GTEX-14C5O-0626-SM-CYKQH', 'GTEX-S32W-2326-SM-2XU9H', 'GTEX-13SLX-0003-SM-6WSAO', 'GTEX-PWCY-2026-SM-5Q5BF', 'GTEX-14E1K-5019-SM-AN426', 'GTEX-ZC5H-0003-SM-6UH3C', 'GTEX-12584-0004-SM-58Q8Q', 'GTEX-1F75I-0004-SM-DLIPB', 'GTEX-1IDJE-0002-SM-DHP13', 'GTEX-S95S-1626-SM-2XV5P', 'GTEX-1F75I-0004-SM-7OO5C', 'GTEX-1MJK2-2426-SM-EVYCS', 'GTEX-1GF9U-1326-SM-AHZ4I', 'GTEX-U3ZM-0426-SM-59HKR', 'GTEX-XLM4-0011-R10A-SM-5S2TN', 'GTEX-SIU7-1826-SM-2XCE2', 'GTEX-RVPU-0008-SM-3NM8B', 'GTEX-1HC8U-5019-SM-AN47M', 'GTEX-WVLH-0001-SM-3PZ7K', 'GTEX-R55F-0011-R8A-SM-2TF4F', 'GTEX-PX3G-0006-SM-2I3E4', 'K-562-SM-5K7ZB', 'GTEX-111YS-0004-SM-6WBTN', 'GTEX-13QJC-3226-SM-5ZZWI', 'GTEX-PVOW-1026-SM-5JK3V', 'GTEX-NFK9-0004-SM-5JK4Y', 'GTEX-13FHO-0003-SM-5HJQE', 'GTEX-OXRN-2426-SM-2M48I', 'GTEX-11TUW-0004-SM-58Q8F', 'GTEX-OOBK-2626-SM-2D43S', 'GTEX-WHPG-2426-SM-3NMBQ', 'GTEX-WWYW-0004-SM-5URDN', 'GTEX-1314G-0003-SM-5DU4J', 'GTEX-15CHQ-0726-SM-CYKOH', 'GTEX-15CHS-0926-SM-5YY94', 'GTEX-1N7R6-0003-SM-DJWYN', 'GTEX-14BMU-0003-SM-5CV98', 'GTEX-R55E-0526-SM-2TO56', 'GTEX-1CAMR-0004-SM-7MA7I', 'GTEX-1PPH6-2726-SM-DLT2K', 'GTEX-15TU5-1426-SM-CYKTT', 'GTEX-RU72-0011-R8A-SM-2TWCW', 'GTEX-11GS4-0004-SM-5DWSQ', 'GTEX-X4EP-0002-SM-3USRC', 'GTEX-178AV-0004-SM-6RBJU', 'GTEX-ZYW4-0002-SM-5DWSC', 'GTEX-11DXZ-2226-SM-5EGH2', 'GTEX-1GF9X-0003-SM-7MA6J', 'GTEX-YEC3-1326-SM-4WWEL', 'GTEX-XYKS-0003-SM-4YUVD', 'GTEX-ZVT3-0001-SM-53UAL', 'GTEX-ZZ64-0004-SM-58Q7F', 'GTEX-QEG4-0326-SM-2S1OS', 'GTEX-PLZ5-0003-SM-AXTS8', 'GTEX-1AX9J-5001-SM-AN463', 'GTEX-XGQ4-1726-SM-4RH4G', 'GTEX-WWTW-0426-SM-DMZ39', 'GTEX-12WSB-2226-SM-5GCOS', 'GTEX-QMRM-1426-SM-4R1K8', 'GTEX-19HZE-3126-SM-731BC', 'GTEX-QDVJ-1926-SM-2QU3T', 'GTEX-Y3I4-1726-SM-CKJTB', 'GTEX-P44H-0011-R6A-SM-2XV5S', 'GTEX-OXRP-2526-SM-2M47A', 'GTEX-14JG1-0002-SM-7DRNT', 'GTEX-SE5C-0003-SM-325AC', 'GTEX-NL3G-0010-SM-4YUVQ', 'GTEX-S95S-0004-SM-325HJ', 'GTEX-PWN1-0126-SM-2I3FK', 'GTEX-R55C-1726-SM-2TO69', 'GTEX-13RTK-0003-SM-5HJRF', 'GTEX-1N2DW-0003-SM-DE1KC', 'GTEX-15CHR-5001-SM-AN439', 'GTEX-1RQEC-0004-SM-DLIQ2', 'GTEX-11ZU8-0001-SM-58Q8J', 'GTEX-OXRK-0426-SM-3NB2E', 'GTEX-PWOO-2526-SM-2QU3N', 'GTEX-139TU-0002-SM-5HJR5', 'GTEX-ZAJG-0001-SM-53IUC', 'GTEX-YECK-1126-SM-4W1ZQ', 'GTEX-1R9PO-0004-SM-DLIQO', 'GTEX-1F88F-0011-R7a-SM-9QEI3', 'GTEX-14A5H-0004-SM-7DRO9', 'GTEX-NPJ8-1626-SM-2HMIY', 'GTEX-14BMV-0011-R10a-SM-AHZN8', 'GTEX-113JC-0001-SM-5DWSH', 'GTEX-1AX8Y-1126-SM-731AG', 'GTEX-Q2AI-0526-SM-2I3EJ', 'GTEX-OXRL-0003-SM-2XK1Q', 'GTEX-11TUW-2726-SM-5EQLC', 'K-562-SM-2AXWK', 'GTEX-14PJO-0011-R10a-SM-AHZNH', 'GTEX-145LV-5001-SM-AN41H', 'GTEX-VUSH-0002-SM-5SOEH', 'GTEX-QVUS-2926-SM-2TWDD', 'GTEX-RTLS-0004-SM-2XK2Y', 'GTEX-13O3Q-0011-R10a-SM-AHZ6Q', 'GTEX-1I1GT-0008-SM-EZ6XN', 'GTEX-WFJO-0003-SM-5URBP', 'GTEX-WHWD-0002-SM-3G3TZ', 'GTEX-1GF9X-0003-SM-7J3A6', 'K-562-SM-3GADY', 'GTEX-Q2AG-0005-SM-2HMK3', 'GTEX-XLM4-0426-SM-5S2T7', 'GTEX-14LZ3-0003-SM-5GYE9', 'GTEX-139T6-0826-SM-DO3X4', 'GTEX-16NFA-1126-SM-7KUE7', 'GTEX-XOTO-0002-SM-4YUWW', 'GTEX-1AX8Y-0002-SM-7MA62', 'GTEX-14BMV-0002-SM-7DRNQ', 'GTEX-OOBJ-2626-SM-2D44W', 'GTEX-11DYG-0004-SM-6WBTB', 'GTEX-147F3-0002-SM-5DU5B', 'GTEX-11WQK-0011-R10a-SM-CYKOX', 'GTEX-1QP9N-0004-SM-DCXS1', 'GTEX-XOTO-0002-SM-53IVT', 'GTEX-REY6-0626-SM-2TO4G', 'GTEX-1RMOY-0003-SM-DE1IT', 'GTEX-OXRP-0003-SM-3USRP', 'GTEX-1S82Z-0003-SM-DJWXY', 'GTEX-P44H-0011-R10A-SM-2XV6E', 'GTEX-QV31-1526-SM-2TO58', 'GTEX-R55G-0006-SM-2TO6P', 'GTEX-14LLW-0526-SM-CYKP4', 'GTEX-11ILO-0726-SM-5HL5I', 'GTEX-1LSNM-1126-SM-DIPG9', 'GTEX-1HSMP-0002-SM-AG612', 'GTEX-TMZS-0002-SM-3G3T6', 'GTEX-1F6RS-2926-SM-7MKG5', 'GTEX-OHPN-2626-SM-2IZJG', 'GTEX-13X6K-0003-SM-5HJPO', 'GTEX-QVUS-2026-SM-CYKPC', 'GTEX-139UC-3226-SM-5N9EZ', 'GTEX-15SHV-0002-SM-6UH2K', 'GTEX-Y5V6-0001-SM-6UH32', 'GTEX-OIZI-0726-SM-2XV6V', 'GTEX-OIZF-0004-SM-2XK1H', 'GTEX-11ILO-0226-SM-5N9D3', 'GTEX-OXRO-0003-SM-2XK2O', 'GTEX-XQ8I-0001-SM-5JK4O', 'GTEX-11ILO-1026-SM-5A5LP', 'GTEX-OIZF-1626-SM-2QU3W', 'GTEX-OIZI-1026-SM-5JK2X', 'GTEX-1QPFJ-0004-SM-DLIQN', 'GTEX-1MA7W-0826-SM-DKPPF', 'GTEX-QEG5-0003-SM-3USS5', 'GTEX-N7MT-0011-R1a-SM-5SI7S', 'GTEX-YJ89-0011-R10a-SM-AHZO2', 'GTEX-UPIC-0005-SM-3GACV', 'GTEX-15CHQ-0011-R1a-SM-6EU31', 'GTEX-17GQL-0003-SM-6RBKD', 'GTEX-ZF3C-0002-SM-53UBG', 'GTEX-QCQG-0006-SM-2IZJN', 'GTEX-NPJ8-0011-R7a-SM-2AXVP', 'GTEX-T5JC-0011-R6A-SM-5SI89', 'GTEX-UJHI-0001-SM-5SOEU', 'GTEX-OOBJ-0326-SM-2D452', 'GTEX-1AX8Z-2626-SM-731EP', 'GTEX-15RJE-0003-SM-6UH2G', 'GTEX-WZTO-0002-SM-3USRF', 'GTEX-16NFA-1226-SM-6M481', 'GTEX-11DXY-0004-SM-58Q7Z', 'GTEX-X4XX-0004-SM-3PZ7P', 'GTEX-1B98T-0826-SM-7DHME', 'GTEX-14C39-0004-SM-7DROE', 'GTEX-QEG4-0003-SM-2XK24', 'GTEX-12WSB-0008-SM-5TDC5', 'GTEX-1RNSC-0002-SM-DE1NY', 'GTEX-PLZ6-0426-SM-5IFK2', 'GTEX-QDVN-1926-SM-48TZ8', 'GTEX-11WQC-0011-R10a-SM-AHZ6D', 'GTEX-14C5O-1126-SM-CYKO3', 'GTEX-1399U-5001-SM-AN3YA', 'GTEX-SIU7-1626-SM-4BRUK', 'GTEX-PWOO-1726-SM-48TDC', 'GTEX-14C39-0011-R10a-SM-AHZ77', 'GTEX-1B97I-0004-SM-7J39N', 'GTEX-17HHY-0002-SM-6RBK6', 'GTEX-1GF9V-0004-SM-7J3AE', 'GTEX-S3XE-2026-SM-2XU9P', 'GTEX-QEL4-0826-SM-325HZ', 'GTEX-145ME-0426-SM-5S2QP', 'GTEX-RWS6-0326-SM-2XUAF', 'GTEX-YEC3-2426-SM-5IFJR', 'GTEX-1339X-0004-SM-5HJQH', 'GTEX-QDVN-0626-SM-2M49S', 'GTEX-OIZF-0426-SM-2AXUM', 'GTEX-P44H-0011-R10A-SM-CYKW8', 'GTEX-17EVP-0011-R10a-SM-AHZLY', 'GTEX-S4UY-0426-SM-2XU8M', 'GTEX-PVOW-0426-SM-5N9DW', 'GTEX-RWSA-0826-SM-2XUAA', 'GTEX-16NFA-2326-SM-7KUE5', 'GTEX-132QS-5016-SM-AN3Y8', 'GTEX-OXRN-0011-R10A-SM-2M49E', 'GTEX-1JMPZ-1726-SM-CNNPR', 'GTEX-15TU5-2926-SM-7KUML', 'GTEX-R55E-0006-SM-2TO6I', 'GTEX-NPJ8-0011-R1a-SM-2AXVF', 'GTEX-1NHNU-0004-SM-DE1K3', 'GTEX-POYW-0426-SM-2XV4T', 'GTEX-132AR-1526-SM-DLIPN', 'GTEX-PLZ4-1226-SM-2M48Q', 'K-562-SM-4KKZ9', 'GTEX-13D11-0003-SM-6WSCK', 'GTEX-XUJ4-0626-SM-4BOOO', 'GTEX-1GZ4H-0001-SM-7MA6N', 'GTEX-SIU8-0002-SM-5JK47', 'GTEX-12WSB-0001-SM-5DU3Y', 'GTEX-P44G-0926-SM-325I2', 'GTEX-ZQUD-0002-SM-6WBSF', 'GTEX-N7MT-0011-R10a-SM-AHZ7Z', 'GTEX-1A8G6-0426-SM-72D6Q', 'GTEX-XBEC-0002-SM-4YUVW_1', 'GTEX-1PDJ9-0003-SM-DJWWM', 'K-562-SM-4VDQQ', 'GTEX-14E6D-0011-R10a-SM-CYKPM', 'GTEX-OIZF-1826-SM-2QU4O', 'GTEX-SE5C-0003-SM-4X61V', 'GTEX-145LV-0926-SM-EZ6Y3', 'GTEX-12ZZX-0001-SM-5ANG4', 'GTEX-XAJ8-0001-SM-58Q94', 'GTEX-ZYY3-0002-SM-6WBUF', 'GTEX-13CF2-0004-SM-5DU48', 'K-562-SM-4TT6S', 'GTEX-PLZ5-0003-SM-2H2X9', 'GTEX-18A7A-0002-SM-6UH2A', 'GTEX-1H1CY-0002-SM-DHP2D', 'GTEX-1CAMS-0003-SM-7MA7J', 'GTEX-P4PP-0126-SM-2D442', 'GTEX-13RTK-0003-SM-6WSCZ', 'GTEX-TML8-0004-SM-4X614', 'GTEX-T6MN-0826-SM-5S2SK', 'GTEX-RVPU-0001-SM-2RXES', 'GTEX-S3XE-1726-SM-2XU9F', 'GTEX-XXEK-2426-SM-5S2S8', 'GTEX-15ER7-0002-SM-7DLFM', 'GTEX-ZPCL-0001-SM-4YUWG', 'GTEX-15DYW-0011-R10a-SM-CYKSJ', 'GTEX-11VI4-0003-SM-58Q81', 'GTEX-1HSMP-2126-SM-A8N8B', 'GTEX-1HCU9-0826-SM-A8N8I', 'K-562-SM-73KWI', 'GTEX-1GZ4I-0002-SM-7J3AA', 'GTEX-X4EP-0011-R10B-SM-CYKW5', 'GTEX-QCQG-1726-SM-2QU57', 'K-562-SM-2XV5K', 'GTEX-NL3H-0011-R8a-SM-2QU3M', 'GTEX-16AAH-0003-SM-6UH2V', 'GTEX-UPJH-0002-SM-3G3S2', 'GTEX-SNOS-0004-SM-4X61S', 'GTEX-T2IS-0002-SM-4X5ZR', 'GTEX-ZAK1-0726-SM-DMZ37', 'GTEX-13D11-5007-SM-AN3YX', 'GTEX-11EQ8-0001-SM-6WBTP', 'GTEX-S7SE-0526-SM-2XV4N', 'GTEX-1399T-0011-R10b-SM-AHZ6E', 'GTEX-14PJM-0926-SM-CYKSY', 'GTEX-145MN-0001-SM-7DROT', 'GTEX-15TU5-0004-SM-6UH2D', 'GTEX-1C475-0003-SM-7J3B4', 'GTEX-T8EM-0001-SM-3G3TL', 'K-562-SM-3P61Y', 'GTEX-11VI4-0003-SM-6WBUJ', 'GTEX-1S5ZU-0004-SM-DJWX6', 'GTEX-YEC3-1326-SM-5PNXV', 'GTEX-RU1J-0003-SM-3K2A8', 'GTEX-11GS4-2526-SM-CYKV3', 'GTEX-1A3MX-0002-SM-7J38V', 'GTEX-PWOO-0008-SM-48TDU', 'GTEX-QDVN-2126-SM-5SI7X', 'GTEX-NFK9-0226-SM-2AXU5', 'GTEX-O5YT-0010-SM-325AG', 'GTEX-TMMY-0003-SM-3G3TW', 'GTEX-1LC46-5001-SM-AN495', 'GTEX-13VXU-0326-SM-CYKUJ', 'GTEX-Q734-0526-SM-2IZI7', 'GTEX-15RJ7-0001-SM-6RBLA', 'GTEX-13113-0004-SM-DO3XJ', 'GTEX-WYJK-0001-SM-3PYKX', 'GTEX-1PDJ9-0003-SM-DE1IC', 'GTEX-X8HC-1426-SM-46MWC', 'GTEX-1R7EV-0002-SM-DLIQ9', 'GTEX-1M5QR-5013-SM-AN49C', 'GTEX-R55F-1226-SM-2TWB5', 'GTEX-1R9PO-0004-SM-DCXR6', 'GTEX-1F75B-0003-SM-7J3AW', 'K-562-SM-48U2C', 'GTEX-XBED-0004-SM-5URD9', 'GTEX-13NZA-0003-SM-5HJQ4', 'GTEX-Q2AG-0011-R5A-SM-2IZJU', 'GTEX-ZEX8-0426-SM-4WWEN', 'GTEX-S3XE-0006-SM-2XU9B', 'GTEX-1JKYR-0004-SM-DE1K8', 'GTEX-OOBJ-0326-SM-5SI84', 'GTEX-1IL2U-1426-SM-A8N8U', 'GTEX-ZZPU-0003-SM-58Q8V', 'GTEX-13X6I-0011-R10a-SM-AHZND', 'GTEX-1HCU9-0002-SM-DHP2A', 'GTEX-T6MN-0003-SM-325HW', 'GTEX-1JJ6O-0001-SM-DIND4', 'GTEX-SIU7-0426-SM-2XCDX', 'GTEX-U3ZH-0003-SM-3G3T1', 'GTEX-OIZG-0003-SM-5JK3W', 'GTEX-WWYW-0926-SM-CYKVP', 'GTEX-WFG7-0002-SM-4X61R', 'GTEX-ZTPG-0003-SM-4YUXJ', 'GTEX-S341-0001-SM-3USS9', 'GTEX-1HFI7-0003-SM-AG5YF', 'K-562-SM-3GILO', 'GTEX-13SLX-0003-SM-5HJPY', 'GTEX-ZTSS-0003-SM-53UAM', 'GTEX-1HCUA-1726-SM-A8N86', 'GTEX-PWN1-0226-SM-2QU2P', 'GTEX-WHSE-1726-SM-CLTG4', 'GTEX-WCDI-0001-SM-3PYKQ', 'GTEX-15D1Q-0526-SM-CYKT2', 'GTEX-ZQG8-1626-SM-CKJTC', 'GTEX-13O21-5028-SM-AN3ZD', 'GTEX-ZGAY-1226-SM-4YCF4', 'GTEX-16NFA-5001-SM-AN44L', 'GTEX-ZVZQ-0002-SM-53UBN', 'GTEX-PVOW-1026-SM-4SVQZ', 'GTEX-S7SE-0011-R7A-SM-2XV6D', 'GTEX-QLQ7-0726-SM-2M495', 'GTEX-1IDJU-0001-SM-G8OXB', 'GTEX-YEC3-2026-SM-4WWFL', 'GTEX-1IDJI-0002-SM-G8OXG', 'GTEX-1AX8Y-1426-SM-73KU6', 'GTEX-YEC3-1426-SM-5PNXW', 'GTEX-R55E-0011-R7A-SM-2TO6A', 'GTEX-1PPGY-0004-SM-DLIQ8', 'GTEX-14C38-2626-SM-5RQI4', 'GTEX-WWYW-0011-R10A-SM-CYKW7-SUP', 'GTEX-X62O-0226-SM-4E3JB', 'GTEX-13112-0004-SM-5ANFX', 'GTEX-1H1DG-3026-SM-A8N9V', 'GTEX-12WSB-0006-SM-5O998', 'GTEX-S4Z8-1726-SM-2XU8V', 'GTEX-1O9I2-0004-SM-DJWWK', 'GTEX-OHPL-0003-SM-4X5ZG', 'GTEX-RU1J-0226-SM-2TWBC', 'GTEX-17F96-0002-SM-6UH1K', 'GTEX-NL3G-0010-SM-53ITK', 'GTEX-17EUY-0003-SM-6RBKF', 'GTEX-WQUQ-0002-SM-3PZ78', 'GTEX-UPJH-0002-SM-4X62B', 'GTEX-1N2DV-0003-SM-DJWXC', 'GTEX-U412-0006-SM-3DB8J', 'GTEX-XUJ4-1226-SM-4BOPD', 'GTEX-1MCC2-0004-SM-DJWWS', 'GTEX-SNMC-1226-SM-2XV54', 'GTEX-NFK9-0526-SM-2QU51', 'GTEX-1AX8Z-0002-SM-7J39U', 'GTEX-1F75I-5013-SM-AN46G', 'GTEX-SSA3-0326-SM-47JWY', 'GTEX-139YR-0004-SM-5DU3U', 'K-562-SM-7MGVH', 'GTEX-15CHQ-0011-R8b-SM-6EU32', 'GTEX-P44G-0826-SM-2D44D', 'GTEX-15TU5-0126-SM-7KUM2', 'GTEX-139UW-0003-SM-5HJRP', 'GTEX-SNMC-0004-SM-5JK4Q', 'GTEX-PVOW-0011-R5A-SM-2XV6T', 'GTEX-WWYW-0011-R10A-SM-CYKW7-INPUT', 'GTEX-11EI6-0011-R10a-SM-CYKQP', 'GTEX-PX3G-0004-SM-2XK13', 'GTEX-11NSD-0005-SM-5LZZ5', 'GTEX-Q734-0001-SM-5JK3N', 'K-562-SM-5P9HH', 'GTEX-ZYWO-1326-SM-5SI8X', 'GTEX-WFG8-0004-SM-3G3TP', 'GTEX-WHPG-0002-SM-3USR1', 'GTEX-OXRL-2626-SM-2D44F', 'GTEX-13OW8-0002-SM-5HJQX_1', 'GTEX-1JMQI-0002-SM-DJWWH', 'GTEX-S3XE-1826-SM-3K2B4', 'GTEX-1399R-0003-SM-5HJRL_1', 'GTEX-15TU5-0011-R10b-SM-CYKU3', 'GTEX-OHPN-2526-SM-2IZJH', 'GTEX-1GMRU-0004-SM-7MA66', 'GTEX-SE5C-0626-SM-4IHLJ', 'GTEX-13FH7-0004-SM-5DU3O', 'GTEX-1B97I-0004-SM-7MD4K', 'GTEX-1L5NE-0002-SM-DE1ND', 'GTEX-Q734-2226-SM-2QU4H', 'GTEX-RU72-2926-SM-2TWB3', 'GTEX-XUJ4-0001-SM-5JK4T', 'GTEX-1H1DG-0001-SM-AG5YP', 'GTEX-XMD3-0003-SM-4YUX3', 'GTEX-PWOO-1026-SM-48TCN', 'GTEX-NPJ8-0011-R10A-SM-DIQCU-PFC2_A3', 'GTEX-NPJ8-0011-R10A-SM-DIQCU-PFC2_A2', 'GTEX-NPJ8-0011-R10A-SM-DIQCU-PFC2_A1', 'GTEX-VJYA-0003-SM-3G3S9', 'GTEX-14E7W-1226-SM-CYKOO', 'GTEX-14C38-0003-SM-5DU4G', 'GTEX-139TT-0004-SM-7DRNY', 'GTEX-OHPN-0011-R11A-SM-2IZJB', 'GTEX-OHPM-0003-SM-5JK4I', 'GTEX-P78B-0926-SM-2M48P', 'GTEX-13CZV-0003-SM-5HJPB', 'GTEX-ZPIC-1726-SM-57WF2', 'GTEX-15G19-0011-R10a-SM-AHZNN', 'GTEX-12ZZW-0003-SM-5DWRO', 'GTEX-1KWVE-0003-SM-DE1ID', 'GTEX-UPIC-1126-SM-4IHLO', 'GTEX-11TUW-0004-SM-7FG6E', 'GTEX-QLQ7-0326-SM-4R1JV', 'GTEX-X4EP-0011-R5A-SM-3P628', 'GTEX-Q2AH-0005-SM-2I3FI', 'GTEX-1C4CL-0003-SM-7J39F', 'GTEX-PW2O-0526-SM-2IZHN', 'GTEX-SE5C-0626-SM-3P5ZJ', 'GTEX-147F4-0926-SM-CYKOU', 'GTEX-PLZ4-0003-SM-5JK4H', 'GTEX-R3RS-1026-SM-2TO5T', 'GTEX-QV31-1526-SM-2S1QB', 'GTEX-15SB6-0002-SM-6UH1J', 'GTEX-VUSG-2626-SM-5S2TM', 'GTEX-QVJO-0226-SM-2S1R2', 'GTEX-1AX8Z-1526-SM-73KW1', 'GTEX-183FY-0003-SM-6UH1R', 'GTEX-13113-1926-SM-5LZWG', 'GTEX-ZXES-0526-SM-5E45D', 'GTEX-12WSD-0002-SM-5ANGA', 'GTEX-X4XX-0011-R7A-SM-46MWR', 'GTEX-11ZTS-0001-SM-5DWTD', 'GTEX-12WSB-0011-R5b-SM-5GU67', 'GTEX-14PHX-0006-SM-9MQJN', 'GTEX-1MUQO-0004-SM-DE1JZ', 'GTEX-RWSA-0003-SM-5JK4L', 'GTEX-12696-0326-SM-DLIOI', 'GTEX-1GZ4I-0002-SM-7MA6M', 'GTEX-1KAFJ-0003-SM-DE1KV', 'GTEX-Z93S-0002-SM-4YUVX', 'GTEX-15DCZ-0001-SM-6RBKW', 'GTEX-1GL5R-5001-SM-AN46E', 'GTEX-XBED-0004-SM-58Q93', 'GTEX-1CAMQ-0006-SM-A9SK3', 'GTEX-YFCO-0001-SM-53IV9', 'GTEX-14PJO-0003-SM-5GYDZ', 'GTEX-ZY6K-0002-SM-6WBT2', 'GTEX-169BO-0003-SM-6UH2W', 'GTEX-XUW1-0002-SM-5JK4P', 'GTEX-T5JC-2026-SM-32PMM', 'GTEX-V955-0001-SM-4X61T', 'GTEX-PW2O-0004-SM-5JK3M', 'GTEX-WVJS-0003-SM-DLIOR', 'K-562-SM-9JGFK', 'GTEX-PVOW-1026-SM-325HO', 'GTEX-1GZHY-1226-SM-9QEIY', 'GTEX-17F97-0002-SM-6RBLP', 'GTEX-18D9B-0004-SM-DINCD', 'GTEX-16NPV-0003-SM-6UH1Q', 'GTEX-14PJM-0011-R1a-SM-6EU2Z', 'GTEX-1B98T-1726-SM-731FB', 'GTEX-S7SE-0004-SM-2XK2B', 'GTEX-ZTTD-0526-SM-57WDV', 'GTEX-QDT8-0926-SM-325A6', 'GTEX-14JG6-0004-SM-5HX8A', 'GTEX-14XAO-5004-SM-AN43Q', 'GTEX-111FC-0001-SM-6WBTJ', 'GTEX-1NHNU-0004-SM-DJWXE', 'GTEX-19HZE-0011-R5a-SM-79OOU', 'GTEX-13X6J-0326-SM-CYKU6', 'GTEX-1AX8Y-5016-SM-AN45N', 'GTEX-1QCLZ-0003-SM-DHP2G', 'GTEX-Q2AH-1726-SM-3NB2B', 'GTEX-S3XE-1926-SM-3K2B3', 'GTEX-QESD-1626-SM-2QU58', 'GTEX-QVJO-0002-SM-2XK1B', 'GTEX-1HGF4-3026-SM-A8N89', 'GTEX-WHSE-0526-SM-4SOJ6', 'GTEX-1HCU6-2426-SM-A8N85', 'GTEX-ZZPU-0003-SM-6WBUC', 'GTEX-OXRN-0011-R10a-SM-AHZM9', 'GTEX-1399T-0004-SM-5HJPS_1', 'GTEX-14AS3-5007-SM-AN41Z', 'GTEX-145MI-0001-SM-5DU4V', 'GTEX-NFK9-0926-SM-2AXU8', 'GTEX-ZYWO-2126-SM-5GZXF', 'GTEX-QV31-0003-SM-5URDH', 'GTEX-146FR-0004-SM-6WSBD', 'GTEX-15SHU-0001-SM-6RBL6', 'GTEX-1GN1U-0002-SM-7MD51', 'GTEX-1HC8U-0003-SM-DHP2K', 'GTEX-ZF28-0726-SM-CYKR3', 'GTEX-S7SE-0011-R5A-SM-2XV6C', 'GTEX-RWSA-0126-SM-2XU9X', 'GTEX-15TU5-2226-SM-6LPI2', 'GTEX-QDVN-2326-SM-2QU3O', 'GTEX-ZAB4-0001-SM-4YUVZ', 'GTEX-S7SF-2026-SM-3K2AS', 'GTEX-ZGAY-0002-SM-4YUV3', 'GTEX-16XZZ-0001-SM-6RBK3', 'GTEX-OHPL-0003-SM-2BWY5', 'GTEX-PLZ5-0926-SM-4RGM9', 'GTEX-145ME-1426-SM-E9WQY', 'GTEX-XQ3S-0002-SM-4YUWE', 'GTEX-Y5V6-0001-SM-53IVM', 'GTEX-PLZ6-0003-SM-5JK4W', 'GTEX-145ME-5001-SM-7IGOL', 'GTEX-ZAJG-0526-SM-CYKPH', 'GTEX-ZLV1-0003-SM-7DRPD', 'GTEX-14BMU-0003-SM-7DRNV', 'GTEX-113IC-0001-SM-DLIPU', 'GTEX-ZVZP-0003-SM-6WBSB', 'GTEX-PLZ4-0726-SM-2QU59', 'GTEX-QEG5-1026-SM-4R1KT', 'GTEX-QEL4-0626-SM-2TWB7', 'K-562-SM-5N9DX', 'GTEX-NFK9-0004-SM-2XK1E', 'GTEX-UPIC-0226-SM-3GADO', 'GTEX-W5X1-0002-SM-3G3T8', 'GTEX-145MH-0001-SM-5GYC7', 'GTEX-15EO6-2826-SM-CYKSA', 'GTEX-RU72-3026-SM-5SI7Y', 'GTEX-13FTX-5011-SM-AN3Z6', 'GTEX-1212Z-0004-SM-5DWTI', 'GTEX-11ZU8-0001-SM-5DWTC', 'GTEX-1GL5R-0002-SM-7MA6L', 'GTEX-13N2G-0001-SM-5HJQA', 'GTEX-ZV6S-0002-SM-4YUWI', 'GTEX-N7MS-2425-SM-26GMU', 'GTEX-NL4W-0011-R9a-SM-2AXVA', 'GTEX-N7MT-1226-SM-26GMT', 'GTEX-WXYG-0001-SM-4X625', 'GTEX-14PJ6-0001-SM-7DLF2', 'GTEX-TMKS-0005-SM-5SI7W', 'GTEX-R3RS-0126-SM-2TO5G', 'GTEX-1CB4F-0005-SM-A8N7L', 'GTEX-N7MT-1226-SM-26GMC', 'GTEX-18A67-5004-SM-AN451', 'GTEX-P4QS-0326-SM-2D43U', 'GTEX-YEC4-0006-SM-53IVG', 'GTEX-1399T-2526-SM-DMZ31', 'GTEX-QLQW-0001-SM-5URCU', 'GTEX-1LH75-0726-SM-DIPG4', 'GTEX-11WQC-0626-SM-CYKQZ', 'GTEX-16XZZ-0526-SM-CYKUB-ELUATE', 'GTEX-1MCYP-0003-SM-DE1KP', 'GTEX-P44H-1126-SM-CLTFW', 'GTEX-14PKU-5016-SM-AN42Y', 'GTEX-1211K-0003-SM-CLTG2', 'GTEX-1B932-5007-SM-AN45T', 'GTEX-R45C-0006-SM-2TWCB', 'GTEX-ZY6K-0002-SM-5DWS5', 'GTEX-ZF29-0826-SM-5S2NP', 'GTEX-QCQG-0003-SM-4X61E', 'GTEX-1R7EU-0003-SM-DE1JE', 'GTEX-X8HC-0003-SM-5URDI', 'GTEX-Q2AG-0011-R9A-SM-2IZJS', 'GTEX-S32W-0001-SM-2XK2F', 'GTEX-1QP2A-0626-SM-DLT2L', 'GTEX-QEG4-1126-SM-2QU2V', 'GTEX-XUJ4-2526-SM-4BOO4', 'GTEX-RVPV-0003-SM-2RXEQ', 'GTEX-1497J-5007-SM-AN41X', 'GTEX-ZYFC-0001-SM-58Q7B', 'GTEX-13O3P-1226-SM-5J2O8', 'GTEX-13RTL-0011-R9a-SM-5Q5BR', 'GTEX-1192X-0004-SM-6WBTF', 'GTEX-QV44-0726-SM-4R1KP', 'GTEX-13X6H-5016-SM-AN415', 'GTEX-RWSA-0526-SM-2XU8U', 'GTEX-14C38-5007-SM-AN429', 'GTEX-RWS6-0004-SM-2XK2T', 'GTEX-12KS4-0003-SM-5ANGF', 'GTEX-OXRK-0626-SM-2D44M', 'GTEX-13PL6-0002-SM-5HJPV', 'K-562-SM-3NMAP', 'GTEX-145LS-2426-SM-EZ6XT', 'GTEX-15DCD-0011-R5a-SM-6EU1F', 'GTEX-Q2AH-0005-SM-5SI7G', 'GTEX-X4XY-0002-SM-3PYKN', 'K-562-SM-4AT3W', 'GTEX-Y8E4-0001-SM-6UH33', 'GTEX-15DCZ-0826-SM-6AJBB', 'GTEX-1KWVE-0005-SM-DIPE4', 'GTEX-OHPN-0004-SM-2BWYH', 'GTEX-RVPV-1726-SM-2TF5W', 'GTEX-S4UY-0626-SM-2XUAQ', 'GTEX-1IDJC-1726-SM-CMKG3', 'GTEX-P4QR-0006-SM-2I5GN', 'GTEX-1S5ZA-0004-SM-DJWYF', 'GTEX-P78B-0526-SM-2M48M', 'GTEX-Y114-0004-SM-4YUWP', 'GTEX-WFON-0001-SM-5S2SE', 'GTEX-OIZF-1026-SM-2AXUK', 'GTEX-178AV-0004-SM-6UH1M', 'GTEX-13NZA-0526-SM-CYKQV', 'GTEX-QEG5-0826-SM-2M49H', 'GTEX-1A8G7-0003-SM-7J38W', 'GTEX-14E7W-0011-R5b-SM-664NS', 'GTEX-13U4I-0004-SM-5HJPX', 'GTEX-WQUQ-0002-SM-6WBSD', 'GTEX-OOBK-0126-SM-2QU3Y', 'GTEX-PSDG-0004-SM-3USRY', 'GTEX-1LGRB-0003-SM-ANILR', 'GTEX-1M5QR-0003-SM-DJWWP', 'GTEX-S4P3-1826-SM-3K2AL', 'GTEX-QDT8-0526-SM-4LMJR', 'GTEX-1HBPH-0006-SM-A8N7W', 'GTEX-14JFF-0011-R11a-SM-5ZZV6', 'GTEX-WI4N-1326-SM-7DRPR', 'GTEX-14JFF-2926-SM-CYKNZ', 'GTEX-QV44-0006-SM-2TO6Q', 'GTEX-S32W-0006-SM-5S2SW', 'GTEX-13X6H-0001-SM-6WSCM', 'GTEX-SN8G-0002-SM-325HX', 'GTEX-RVPV-1726-SM-2XU8R', 'GTEX-OHPK-0526-SM-2D455', 'GTEX-15DDE-1126-SM-CYKRM', 'GTEX-OXRO-0326-SM-2D45W', 'GTEX-13D11-5019-SM-AN3YZ', 'GTEX-ZF2S-0002-SM-53UBA', 'GTEX-1IDJI-5013-SM-AN47F', 'GTEX-ZF2S-0002-SM-6UH39', 'GTEX-QEG5-1126-SM-5SI88', 'GTEX-OXRK-1826-SM-2D44B', 'GTEX-ZYWO-2626-SM-5E43V', 'GTEX-139TS-0011-R10a-SM-AHZ6H', 'GTEX-P4PP-1726-SM-2M47H', 'K-562-SM-5K7TT', 'GTEX-1122O-0004-SM-5DWSJ', 'GTEX-OHPM-1626-SM-2HMK4', 'GTEX-1S5ZU-0004-SM-DE1J3', 'GTEX-1AYD5-5001-SM-AN45G', 'GTEX-16Z82-0426-SM-EZ6Y6', 'GTEX-ZVT2-0003-SM-53UBL', 'GTEX-14ABY-0003-SM-5CV94', 'GTEX-144FL-2326-SM-5LUAC', 'GTEX-SE5C-0003-SM-6WBS1', 'GTEX-15CHC-5019-SM-AN438', 'GTEX-Q2AG-0626-SM-2QU4M', 'GTEX-1CB4H-0004-SM-7J38K', 'GTEX-12WSG-0001-SM-5ANGG', 'GTEX-RNOR-0001-SM-2RXF2', 'GTEX-UPIC-0326-SM-4IHJ9', 'GTEX-1GMR8-0003-SM-7MA65', 'GTEX-WFON-0426-SM-5S2T1', 'GTEX-15DDE-0011-R11b-SM-6LPIF', 'GTEX-14BIN-2226-SM-CYKPI', 'GTEX-RWS6-0226-SM-2XUAO', 'GTEX-13111-5001-SM-AN3XJ', 'GTEX-14PN3-5013-SM-AN435', 'GTEX-WFG8-2326-SM-3GILF', 'GTEX-14LLW-0002-SM-5HX99', 'GTEX-PWOO-0006-SM-2I3E3', 'GTEX-XMD1-2926-SM-4AT42', 'GTEX-11ILO-2526-SM-5A5LQ', 'K-562-SM-5GZZE', 'GTEX-15G1A-0001-SM-6RBJD', 'GTEX-15CHC-0001-SM-7DLH1', 'GTEX-1I1GT-0126-SM-E9WQZ', 'GTEX-17HG3-0002-SM-6RBKU', 'GTEX-16XZY-5001-SM-AN456', 'GTEX-13SLW-0003-SM-6WSAP', 'GTEX-145LV-1126-SM-5Q5C1', 'GTEX-RN64-0006-SM-2TO5F', 'GTEX-ZYT6-1026-SM-7LG6X', 'GTEX-13FHP-0004-SM-5DU3X', 'GTEX-14BIL-0011-R10a-SM-AHZ7B', 'GTEX-NL3H-0011-R1a-SM-2AXV7', 'GTEX-O5YV-0526-SM-2D7VZ', 'GTEX-PX3G-0004-SM-4X612', 'GTEX-RN64-0004-SM-2MRIG', 'GTEX-RNOR-2326-SM-2TWC7', 'GTEX-PWOO-0326-SM-48TDB', 'GTEX-15TU5-1426-SM-7KUN6', 'GTEX-PVOW-0426-SM-2XV5E', 'GTEX-WH7G-0004-SM-3PYKL', 'GTEX-S341-0226-SM-5S2VG', 'GTEX-ZVE1-0001-SM-53UAH', 'GTEX-1J8JJ-0004-SM-G8OXF', 'GTEX-R3RS-0001-SM-3USSD', 'GTEX-Q2AG-2826-SM-2IZJY', 'GTEX-14BIN-0004-SM-5CV9N', 'GTEX-17JCI-0004-SM-6UH23', 'GTEX-14BMV-0826-SM-CYKTX-ELUATE', 'GTEX-1B933-0003-SM-7J3A4', 'GTEX-11DXX-0004-SM-5DWSD', 'GTEX-OIZH-0526-SM-2AXUN', 'GTEX-13VXT-0003-SM-5HJR7', 'GTEX-1B932-0003-SM-7MD4Q', 'GTEX-S341-0001-SM-2VCTY', 'GTEX-XOT4-0001-SM-53IVJ', 'GTEX-QEL4-0004-SM-2RXF3', 'GTEX-WEY5-0003-SM-5URDM', 'GTEX-13NZ8-2126-SM-5L3EO', 'GTEX-TMMY-0926-SM-33HBG', 'GTEX-ZE7O-0002-SM-4YUXG', 'GTEX-11PRG-2226-SM-5GU5R', 'GTEX-1HCUA-0002-SM-AG5YA', 'GTEX-OIZI-1026-SM-4SVQ7', 'GTEX-1F75W-0004-SM-7MA6T', 'GTEX-OHPJ-0003-SM-2H2WV', 'GTEX-Y3I4-0004-SM-53IUR', 'GTEX-1HCU7-5004-SM-AN478', 'GTEX-1LG7Y-0001-SM-DJWYK', 'GTEX-13IVO-0011-R5a-SM-5L3CY', 'GTEX-1IDJD-0003-SM-DCCZJ', 'GTEX-145ME-5016-SM-7938P', 'GTEX-1JN76-0004-SM-DE1KU', 'GTEX-S32W-2126-SM-2XU8S', 'GTEX-WHWD-0002-SM-5SOEC', 'GTEX-15CHS-0011-R8a-SM-6EU1L', 'GTEX-1M4P7-0004-SM-DJWWT', 'GTEX-R55C-0005-SM-2TO5O', 'GTEX-ZDTT-0002-SM-4YUVO', 'GTEX-VUSG-0926-SM-5S2SV', 'GTEX-OHPN-0011-R3A-SM-2IZJ4', 'GTEX-14JG1-0011-R10a-SM-AHZNJ', 'K-562-SM-2D7VW', 'GTEX-UJHI-0001-SM-4X62F', 'K-562-SM-2D7VY', 'GTEX-1AX8Y-5001-SM-AN45M', 'GTEX-QEG5-0008-SM-4R1KL', 'GTEX-PVOW-0626-SM-2XV4R', 'GTEX-15CHS-0011-R5b-SM-69LPA', 'GTEX-WWYW-0011-R10A-SM-CYKW7-ELUATE', 'GTEX-15CHS-0011-R5b-SM-69LP3', 'GTEX-P4PQ-0526-SM-2D449', 'GTEX-ZVZQ-0002-SM-4YUWV', 'GTEX-XUJ4-0726-SM-4BOOP', 'GTEX-QLQ7-0126-SM-4R1JW', 'GTEX-11GSP-0004-SM-58Q83', 'GTEX-11P81-0002-SM-58Q88', 'GTEX-1HT8W-1526-SM-DKTQF', 'GTEX-12C56-0003-SM-5DU56', 'GTEX-1PIEJ-0003-SM-DJWWX', 'GTEX-148VJ-0002-SM-7DRO4', 'GTEX-14E1K-0003-SM-5CV9F', 'GTEX-132Q8-0004-SM-7DRQB', 'GTEX-X638-0004-SM-53IUI', 'GTEX-12WS9-0001-SM-5DU63', 'GTEX-111VG-0004-SM-6WBTS', 'GTEX-S95S-0005-SM-2XV5N', 'GTEX-UPIC-0826-SM-3GADQ', 'GTEX-12BJ1-0001-SM-5DU4Z', 'GTEX-1AX8Y-2326-SM-7189N', 'GTEX-1RAZR-0003-SM-DCXS4', 'GTEX-OIZF-1026-SM-7P8QV', 'GTEX-WFJO-0003-SM-3G3S3', 'GTEX-XUJ4-1726-SM-4BONW', 'GTEX-Q734-0001-SM-2IJGM', 'GTEX-ZAJG-0011-R10a-SM-CYKQ1', 'GTEX-113IC-0001-SM-5DWSI', 'GTEX-14E1K-0003-SM-7DROG', 'GTEX-U8XE-0004-SM-5URB8', 'GTEX-1H3NZ-0002-SM-AG5Z1', 'GTEX-1497J-5001-SM-AN41V', 'GTEX-15CHS-0011-R4b-SM-6EU1D', 'GTEX-1B932-0003-SM-7J39T', 'GTEX-QLQW-1126-SM-2TO72', 'GTEX-SIU7-0226-SM-4BRX2', 'GTEX-1B8KE-0004-SM-7J39M', 'GTEX-XGQ4-0001-SM-4YUVK_1', 'GTEX-1CB4G-0002-SM-DINCF', 'GTEX-ZYWO-2826-SM-5GIDY', 'GTEX-OXRK-0006-SM-2HML1', 'GTEX-12WS9-0008-SM-5TDC2', 'GTEX-QEG5-0726-SM-4R1JQ', 'GTEX-S4Z8-1426-SM-72D7I', 'GTEX-1A3MV-0002-SM-7MD4O', 'GTEX-1HSMP-1226-SM-A8N9X', 'GTEX-15UKP-0003-SM-6UH1I', 'GTEX-SJXC-0003-SM-325HP', 'GTEX-1B8SF-0003-SM-7J38X', 'GTEX-S4Q7-1626-SM-3K2AE', 'GTEX-1GZ2Q-0002-SM-7MA6O', 'GTEX-Y9LG-0003-SM-6UH3O', 'K-562-SM-4WWAK', 'GTEX-ZQG8-0008-SM-4E3J9', 'GTEX-RM2N-1626-SM-2TO4W', 'GTEX-X62O-0626-SM-EVR34', 'GTEX-TMKS-0002-SM-3G3T7', 'GTEX-RVPU-0011-R8A-SM-5SI8I', 'GTEX-U3ZH-0326-SM-4WWDX', 'GTEX-ZXG5-0002-SM-58Q77', 'GTEX-13U4I-0004-SM-6WSAN', 'GTEX-1GZ2Q-0011-R11a-SM-A8N7T', 'GTEX-QXCU-1726-SM-2TO54', 'GTEX-13FTX-0004-SM-5CV8K', 'GTEX-XQ3S-0002-SM-5JK42', 'GTEX-S7SF-0226-SM-3K2BI', 'GTEX-P4PQ-2626-SM-5SI7K', 'GTEX-WEY5-2026-SM-3GILE', 'GTEX-ZF2S-1026-SM-4WWB1', 'GTEX-15UKP-5010-SM-AN44O', 'GTEX-1BAJH-0003-SM-7MA6G', 'GTEX-P4PQ-0226-SM-2M477', 'GTEX-18D9U-5004-SM-AN44W', 'GTEX-N7MT-0011-R1a-SM-2TC6G', 'GTEX-14JG6-5001-SM-AN42H', 'GTEX-1ICLY-0626-SM-A8N9F', 'K-562-SM-66EY4', 'GTEX-XXEK-2626-SM-5S2T5', 'GTEX-13O3Q-2926-SM-CYKRS', 'GTEX-1PWST-1526-SM-E9IK5', 'GTEX-1N2DV-0003-SM-DE1K1', 'K-562-SM-5NQ9D', 'GTEX-R55E-0011-R1A-SM-2TO6L', 'GTEX-YECK-0002-SM-4YUVN', 'GTEX-NL4W-0009-SM-5JK3O', 'GTEX-Q2AH-0004-SM-2IJGI', 'GTEX-YF7O-0002-SM-4YUUX', 'GTEX-1HSMP-1526-SM-A8N9Y', 'GTEX-13OVK-5019-SM-AN3ZL', 'GTEX-XLM4-2426-SM-DO3WY', 'GTEX-OIZG-0226-SM-2AXUG', 'GTEX-ZXG5-0002-SM-5DWS3', 'GTEX-12BJ1-5001-SM-AN3X8', 'GTEX-1GN1W-0001-SM-7J3B1', 'GTEX-1POEN-0002-SM-DLIPZ', 'GTEX-WL46-0002-SM-3PYK6', 'GTEX-11EMC-0004-SM-58Q8T', 'GTEX-16YQH-0011-R10a-SM-AHZMD', 'GTEX-NL4W-0006-SM-2IZIU', 'K-562-SM-7KUKI', 'GTEX-13JUV-0011-R10a-SM-AHZ6O', 'GTEX-12ZZX-0001-SM-5DWRR', 'GTEX-ZYFG-0004-SM-6WBT5', 'GTEX-15SHW-1726-SM-6LPJ2', 'GTEX-145ME-0626-SM-5O99Q', 'GTEX-PVOW-2626-SM-2XV6W', 'GTEX-17HHE-0002-SM-6UH1N', 'GTEX-SIU7-0926-SM-4BRX1', 'GTEX-RNOR-0426-SM-2TWAN', 'K-562-SM-59HJ4', 'GTEX-OHPN-0004-SM-4X5ZI', 'GTEX-X4EP-0002-SM-3PZ7Y', 'GTEX-XOTO-0011-R7B-SM-4B64R', 'GTEX-14JIY-0011-R10a-SM-AHZ75', 'GTEX-12WSD-0011-R10b-SM-5GU79', 'GTEX-YEC3-2126-SM-5CVLR', 'GTEX-12WSH-0011-R10a-SM-CYKRI', 'GTEX-OXRK-0526-SM-2YUMW', 'GTEX-P4PQ-0126-SM-2M47B', 'GTEX-POMQ-0526-SM-2QU4T', 'GTEX-15DDE-0526-SM-CYKOF', 'GTEX-15RJE-0003-SM-6RBL3', 'GTEX-1211K-5001-SM-AN3X5', 'GTEX-YJ89-0002-SM-6UH1H', 'GTEX-1IKK5-1026-SM-AHZ2V', 'GTEX-ZPIC-1726-SM-CKJTG', 'GTEX-16NPV-0011-R10a-SM-AHZM1', 'GTEX-14PHW-0011-R10a-SM-AHZNG', 'K-562-SM-9MQJO', 'GTEX-QEL4-0826-SM-2TWC4', 'GTEX-13O61-0003-SM-5GYAM', 'GTEX-PVOW-2526-SM-CYKP7', 'GTEX-OIZI-1026-SM-2XK39', 'GTEX-11WQK-0003-SM-6WBUB', 'GTEX-1IGQW-2726-SM-A8N9K', 'GTEX-RTLS-0926-SM-2TWBR', 'GTEX-1N7R6-0003-SM-DE1K9', 'GTEX-OXRO-0011-R10A-SM-2D45I', 'GTEX-WI4N-1326-SM-3PZ7L', 'GTEX-ZYFD-1026-SM-5E43Z', 'GTEX-XLM4-0004-SM-5S2TF', 'GTEX-Q2AI-0003-SM-2XK1X', 'K-562-SM-2S1Q5', 'GTEX-16GPK-0003-SM-6RBLF', 'GTEX-1MA7X-0003-SM-DE1KY', 'GTEX-ZTSS-0003-SM-6WBSU', 'GTEX-OHPJ-1026-SM-2D45F', 'GTEX-1ICLY-0003-SM-DC69K', 'GTEX-XOTO-0011-R1B-SM-4B65C', 'GTEX-17F97-0002-SM-6UH2Z', 'GTEX-113IC-0001-SM-58Q97', 'GTEX-PLZ6-1426-SM-2QU4V', 'GTEX-1AX8Z-0326-SM-731BU', 'GTEX-NPJ8-1326-SM-2AXTY', 'GTEX-RVPU-0011-R9A-SM-2XUAM', 'GTEX-U412-0326-SM-5BC7E', 'GTEX-1MJK2-0011-R2b-SM-EXUS5', 'GTEX-12WS9-2826-SM-5FQU9', 'GTEX-17F9E-5001-SM-AN44R', 'GTEX-1CB4J-0011-R2a-SM-9WG5N', 'GTEX-NPJ7-0009-SM-2BWYS', 'GTEX-ZVT3-0001-SM-4YUV7', 'GTEX-111YS-0004-SM-5DWSS', 'GTEX-1399S-1226-SM-5J1O6', 'GTEX-Q2AG-0011-R3A-SM-2IZJT', 'GTEX-11LCK-0002-SM-6WBSW', 'GTEX-X4XX-0004-SM-5URCR', 'GTEX-U3ZH-0003-SM-3G3RZ', 'GTEX-17F96-0002-SM-6RBJL', 'GTEX-11PRG-0004-SM-6WBUN', 'GTEX-14E6C-3126-SM-664ON', 'GTEX-PWN1-0004-SM-2IJGN', 'GTEX-PLZ6-0426-SM-2M48R', 'GTEX-14JFF-0002-SM-DINCH', 'GTEX-P4QS-0003-SM-5JK4U', 'GTEX-XLM4-0226-SM-5S2TL', 'GTEX-WI4N-0926-SM-3LK7K', 'GTEX-POMQ-0326-SM-2M48U', 'GTEX-PWOO-2026-SM-48TDE', 'GTEX-P4QS-0003-SM-2XK1S', 'GTEX-13PLJ-0003-SM-5CV8G', 'GTEX-POYW-0526-SM-2XV5Y', 'GTEX-13VXT-0003-SM-6WSCL', 'GTEX-1B8KE-0004-SM-7MD4J', 'GTEX-1I1GP-0326-SM-DKTR8', 'GTEX-1PPH8-0003-SM-DLIPC', 'GTEX-RU72-1326-SM-2TWC9', 'K-562-SM-5L3FO', 'GTEX-17F96-0226-SM-7EPGV', 'GTEX-RNOR-0001-SM-2XK2U', 'GTEX-1QAET-1426-SM-DKTRZ', 'GTEX-1F52S-0004-SM-7MA7B', 'GTEX-WH7G-0004-SM-5URCL', 'GTEX-NPJ8-1526-SM-26GMH', 'GTEX-1J8Q3-5010-SM-AN486', 'GTEX-1GMR8-0003-SM-7J38N', 'GTEX-S4Q7-0001-SM-3UZFN', 'GTEX-117YX-0001-SM-5DU4O', 'GTEX-NPJ8-1526-SM-26GMY', 'GTEX-1GF9X-2326-SM-7PBZD', 'GTEX-ZGAY-0002-SM-6UH3J', 'GTEX-131XE-1626-SM-5K7VO', 'GTEX-1HSMO-0011-R8b-SM-CM2SW', 'GTEX-QEG5-0226-SM-2M49K', 'GTEX-R55F-1726-SM-2TF4R', 'GTEX-RU72-0126-SM-2TWBK', 'GTEX-R55C-0003-SM-5URCJ', 'GTEX-PX3G-0326-SM-2IZID', 'GTEX-Y8LW-0004-SM-4YUUY', 'GTEX-1I6K6-2226-SM-A8N8Q', 'GTEX-WEY5-0003-SM-3G3TE', 'GTEX-NPJ8-0011-R3a-SM-2AXWE', 'GTEX-P44G-0011-R11A-SM-2D45R', 'GTEX-QDVJ-1726-SM-2M492', 'GTEX-1RAZR-0826-SM-EYYWP', 'GTEX-S33H-0004-SM-2VCTX', 'GTEX-1RB15-0004-SM-DE1J7', 'GTEX-UTHO-0003-SM-4X62R', 'GTEX-13PDP-1026-SM-DK2JQ', 'GTEX-XV7Q-0004-SM-5JK3G', 'GTEX-16MTA-0003-SM-6RBLE', 'GTEX-1RAZA-0002-SM-DE1JD', 'GTEX-13CZV-0006-SM-5P9IN', 'GTEX-RN64-0006-SM-2TC6P', 'GTEX-NL4W-0011-R1a-SM-2AXVC', 'GTEX-OHPN-0004-SM-2XJZT', 'GTEX-POYW-1226-SM-4SVQN', 'GTEX-SIU7-1826-SM-2XK3C', 'GTEX-1399S-5019-SM-AN3YM', 'GTEX-ZAB4-0001-SM-53IUX', 'GTEX-Y5LM-0002-SM-6UH31', 'GTEX-SJXC-0526-SM-2XV57', 'GTEX-ZF3C-0002-SM-4YUXH', 'GTEX-N7MT-1126-SM-2QU53', 'GTEX-13FLV-0011-R10b-SM-AHZ6K', 'GTEX-16GPK-1226-SM-7KUE6', 'K-562-SM-5LZWR', 'GTEX-R55F-0008-SM-48FE6', 'GTEX-OHPL-0526-SM-4M1XS', 'GTEX-16A39-0004-SM-6RBLC', 'GTEX-12WS9-0726-SM-5FQT7', 'GTEX-WWYW-1326-SM-CYKVO', 'GTEX-1I1GS-3026-SM-ACKW4', 'GTEX-XAJ8-0001-SM-3USR2', 'GTEX-R55E-0011-R6A-SM-2TO6K', 'GTEX-T2YK-0003-SM-3USRJ', 'K-562-SM-4DXSD', 'GTEX-TMKS-0326-SM-3DB8A', 'GTEX-NL3G-0226-SM-4RGLU', 'GTEX-QEG4-0426-SM-2M49M', 'GTEX-S4Q7-0426-SM-3K2BJ', 'GTEX-ZY6K-1726-SM-5GZX8', 'GTEX-13QIC-0004-SM-5HJR4', 'GTEX-TKQ1-0002-SM-3G3SW', 'GTEX-REY6-0426-SM-2TO5S', 'GTEX-145MG-0002-SM-5DU3Z', 'GTEX-111CU-1926-SM-5GZYZ', 'GTEX-WYBS-0126-SM-4SVQC', 'GTEX-14ABY-1326-SM-CYKRP', 'GTEX-S4Z8-1426-SM-CM2U3', 'GTEX-1H2FU-0005-SM-A8N7R', 'GTEX-T6MN-0426-SM-5S2SU', 'GTEX-UPK5-0726-SM-4IHL7', 'GTEX-12ZZW-0011-R10a-SM-AHZMP', 'GTEX-16YQH-0004-SM-6RBKA', 'GTEX-QVUS-0001-SM-2RXET', 'GTEX-13JVG-0003-SM-5DU67', 'GTEX-1JN6P-0001-SM-DJWYE', 'GTEX-13NYB-0004-SM-5DU4R', 'GTEX-12WSL-0003-SM-5DWRL', 'GTEX-SSA3-0001-SM-5URCI', 'GTEX-ZG7Y-0001-SM-6UH3L', 'GTEX-PVOW-0011-R10a-SM-AHZMA', 'GTEX-SIU8-0926-SM-2XCDO', 'GTEX-16XZZ-2326-SM-6LPJK', 'GTEX-N7MS-0009-SM-5JK3E', 'GTEX-N7MT-0011-R4a-SM-2AXVR', 'GTEX-11DZ1-0002-SM-5DWSL', 'GTEX-1F5PL-0002-SM-7MA77', 'GTEX-QXCU-0126-SM-2TO4U', 'GTEX-PWOO-0006-SM-2IZHS', 'GTEX-16GPK-0526-SM-CYKU9-SUP', 'GTEX-YEC3-0004-SM-6UH35', 'GTEX-RNOR-0011-R1A-SM-5SI8F', 'GTEX-15CHS-0011-R9b-SM-6LLH8', 'GTEX-S4Z8-1926-SM-2XU8Q', 'GTEX-13NYC-0004-SM-5DU65', 'GTEX-11PRG-0011-R10a-SM-AHZML', 'GTEX-WH7G-0726-SM-5EGIX', 'GTEX-11EM3-0004-SM-5DWTY', 'GTEX-XMD1-0004-SM-7DLGN', 'GTEX-O5YV-1626-SM-2QU48', 'GTEX-PWO3-0004-SM-2XK16', 'GTEX-OHPJ-0626-SM-2M47U', 'K-562-SM-9HWOX', 'GTEX-XMD2-0001-SM-53IUM', 'GTEX-15DCZ-2526-SM-69LOC', 'GTEX-NPJ7-0011-R5a-SM-2AXVT', 'GTEX-1H11D-2326-SM-A8N9O', 'GTEX-1S5ZA-0004-SM-DE1J6', 'GTEX-15DCD-0011-R11a-SM-69LP4', 'GTEX-1A3MW-0826-SM-72D6C', 'GTEX-1IOXB-0003-SM-G8OXU', 'GTEX-14DAR-0004-SM-7DROF', 'GTEX-13OVI-5001-SM-AN3ZK', 'GTEX-13JVG-0003-SM-5HJQ2', 'GTEX-13X6I-0011-R10a-SM-CYKOE', 'GTEX-S4Z8-0426-SM-2XUA2', 'GTEX-PWO3-0011-R5A-SM-2IZIZ', 'GTEX-N7MS-0009-SM-2XK1D', 'GTEX-S3XE-0003-SM-2VCTV', 'GTEX-WEY5-2026-SM-5SI8D', 'GTEX-NL4W-0009-SM-3UZFE', 'GTEX-1MCQQ-2526-SM-E9TJZ', 'GTEX-131XE-5003-SM-AN3XU', 'GTEX-11EI6-0004-SM-7FG6D', 'GTEX-S4Z8-1726-SM-3K2AX', 'GTEX-15FZZ-1026-SM-DLL3J', 'K-562-SM-9HWOI', 'GTEX-1477Z-0011-R10b-SM-CYKRY', 'GTEX-OIZI-0008-SM-2XV77', 'GTEX-17F9E-0003-SM-6UH2B', 'GTEX-12WSE-0002-SM-5DWRT', 'GTEX-P4QS-0003-SM-2H2X1', 'K-562-SM-9HWOB', 'GTEX-14AS3-0003-SM-7DRO8', 'GTEX-OXRL-0526-SM-2D44C', 'GTEX-1F75B-0003-SM-7MA79', 'GTEX-S4Q7-0326-SM-3K2B1', 'GTEX-16BQI-5007-SM-AN44E', 'GTEX-11TT1-0001-SM-58Q8E', 'GTEX-11ILO-0526-SM-5N9DF', 'GTEX-14ICK-0004-SM-7DROB', 'GTEX-1GN1U-0011-R7a-SM-CE6RE', 'GTEX-1IDJC-0926-SM-A8N95', 'GTEX-1HSGN-2126-SM-A8N8G', 'GTEX-1H11D-5004-SM-AN46X', 'GTEX-XMD1-0004-SM-53IVF', 'GTEX-13OVG-0003-SM-6WSAV', 'GTEX-NPJ8-1226-SM-3MJHM', 'GTEX-111CU-0003-SM-5DWTR', 'GTEX-1OJC3-0003-SM-DE1JY', 'GTEX-ZDYS-0005-SM-6UH3G', 'GTEX-OHPK-2626-SM-2D457', 'K-562-SM-46MWI', 'GTEX-QDVJ-0003-SM-5URC5', 'GTEX-QMRM-0326-SM-4R1K5', 'GTEX-13PVQ-0126-SM-5SIB4', 'K-562-SM-5HL7A', 'GTEX-1C2JI-0003-SM-7J39E', 'GTEX-O5YV-0004-SM-5JK2N', 'GTEX-132AR-0002-SM-5DU51', 'GTEX-ZAK1-0726-SM-DMRPK', 'GTEX-OXRP-2326-SM-2M476', 'GTEX-1PIIG-2026-SM-DLT2G', 'GTEX-U3ZG-0002-SM-53IV8', 'GTEX-X62O-0001-SM-5URD5', 'GTEX-15RJ7-5016-SM-AN442', 'GTEX-XXEK-0926-SM-5S2T8', 'GTEX-13O61-5019-SM-AN3ZQ', 'GTEX-TMKS-0426-SM-3DB85', 'GTEX-1JMQL-2426-SM-CNNPW', 'K-562-SM-2I5EP', 'K-562-SM-5N9B7', 'GTEX-1S3DN-0001-SM-DJWXL', 'GTEX-1A32A-3026-SM-72D61', 'GTEX-12WSH-0001-SM-5ANG1', 'GTEX-QEL4-1126-SM-4R1JJ', 'GTEX-15DCZ-2826-SM-6EU2J', 'GTEX-139T8-0726-SM-5L3D2', 'K-562-SM-3MJHH', 'GTEX-ZDTS-1326-SM-4WKGX', 'GTEX-T5JC-0011-R10A-SM-5S2TK', 'GTEX-13NYC-0004-SM-5HJQD', 'GTEX-PVOW-0426-SM-2XCF8', 'GTEX-RVPU-2426-SM-2XCAR', 'GTEX-ZF29-2126-SM-4WWB8', 'GTEX-ZDXO-1326-SM-CYKQX', 'GTEX-17EVQ-0001-SM-6UH1V', 'GTEX-1PFEY-0003-SM-DJWWL', 'GTEX-14JG6-0004-SM-7DRNR', 'GTEX-117YX-1626-SM-5GZZG', 'K-562-SM-2TF5R', 'GTEX-RU72-2626-SM-4GIE1', 'GTEX-QEG5-0426-SM-2M49L', 'GTEX-QDVJ-0003-SM-3USRZ', 'GTEX-15D1Q-0011-R10a-SM-AHZNI', 'K-562-SM-4QAR5', 'GTEX-QVJO-0006-SM-2TO6V', 'GTEX-N7MS-0011-R2a-SM-2IZK7', 'GTEX-RU72-0011-R10A-SM-2TWD4', 'GTEX-16XZZ-1126-SM-CYKSR', 'GTEX-YEC3-0426-SM-4YCEP', 'GTEX-147F4-0003-SM-6WSBG', 'GTEX-R55D-0926-SM-7DRPI', 'GTEX-S32W-0826-SM-5S2SI', 'GTEX-14E7W-0001-SM-7DROS', 'K-562-SM-9HWO4', 'GTEX-14PN3-0004-SM-6RBLO', 'GTEX-OXRO-2026-SM-2M484', 'GTEX-139T8-0926-SM-EZ6XP', 'GTEX-1S82P-0002-SM-DJWXK', 'GTEX-15G19-0001-SM-6RBJJ', 'GTEX-NPJ7-0011-R10a-SM-AHZ7T', 'GTEX-R53T-1826-SM-2TO5Z', 'GTEX-1H3VE-0001-SM-AG5YN', 'GTEX-16NFA-0003-SM-6RBLD', 'GTEX-1NUQO-0003-SM-DE1KJ', 'GTEX-13112-0004-SM-6WSBQ', 'GTEX-1CAMS-0003-SM-7J3B7', 'GTEX-UPIC-0004-SM-3G3SB', 'GTEX-14JG1-1026-SM-CYKOG', 'GTEX-144GL-0426-SM-DK2JR', 'GTEX-WWYW-0004-SM-3USR5', 'GTEX-11ZTT-0003-SM-6WBU4', 'GTEX-PWCY-2026-SM-2S1NF', 'GTEX-R53T-0003-SM-2XK2X', 'GTEX-13PL7-0003-SM-6WSD3', 'GTEX-14LLW-0011-R11b-SM-62LEH', 'GTEX-15TU5-2626-SM-7KUKK', 'GTEX-13OVL-0003-SM-5CV8H', 'GTEX-13PVQ-0003-SM-5GYCU', 'GTEX-RUSQ-0001-SM-5JK34', 'GTEX-1A3MW-0004-SM-7J39Q', 'GTEX-WRHK-0003-SM-3PYK4', 'GTEX-OHPL-0626-SM-3MJGJ', 'K-562-SM-2TO4S', 'GTEX-1HR9M-0011-R3b-SM-ARL7B', 'GTEX-S32W-0004-SM-5S2SG', 'GTEX-PWO3-0004-SM-2IJGP', 'GTEX-QEL4-0826-SM-4X62N', 'GTEX-ZVTK-0002-SM-53UB2', 'GTEX-1IKJJ-2826-SM-A8N8T', 'GTEX-R55F-1326-SM-2TF5F', 'GTEX-POMQ-0006-SM-2I5F4', 'K-562-SM-51MQT', 'GTEX-RWSA-0626-SM-2XU9U', 'GTEX-OHPJ-0626-SM-2YUMY', 'GTEX-VUSG-0002-SM-3G3TD', 'GTEX-S33H-0004-SM-4SVPL', 'GTEX-WHSE-0011-R10a-SM-AHZO3', 'GTEX-1HSGN-2026-SM-A8N7S', 'GTEX-WH7G-0726-SM-5EQ6C', 'GTEX-1C64O-0003-SM-DINCB', 'GTEX-YEC3-0726-SM-4YCD1', 'GTEX-ZEX8-0526-SM-4WKH5', 'K-562-SM-2YUNN', 'GTEX-WK11-0002-SM-4X5ZT', 'GTEX-QCQG-0326-SM-2IZIH', 'GTEX-15EOM-0326-SM-6LLJH', 'GTEX-1EKGG-0526-SM-7P8SZ', 'GTEX-14BIM-0002-SM-7DROA', 'GTEX-S4Z8-1826-SM-2XU9S', 'GTEX-1B996-0003-SM-7MA64', 'GTEX-1192W-2326-SM-5N9BS', 'GTEX-1HR98-0004-SM-AG5ZT', 'GTEX-1HBPM-1726-SM-A8N9W', 'GTEX-1IDJH-0002-SM-G8OXI', 'GTEX-T5JC-0002-SM-325HQ', 'GTEX-OOBJ-0008-SM-DO3X5', 'GTEX-18A66-0002-SM-DLIP1', 'GTEX-1GMR3-0001-SM-7J3AM', 'GTEX-15RIE-5019-SM-AN43Y', 'GTEX-Q2AG-0003-SM-5JK4S', 'GTEX-YEC3-0426-SM-5IFK1', 'GTEX-WVLH-0011-R10A-SM-CYKW4-SUP', 'GTEX-XUJ4-1426-SM-4BONT', 'GTEX-OHPM-0003-SM-4X5ZH', 'GTEX-WL46-0626-SM-CYKVU', 'GTEX-WHSB-0004-SM-3G3RY', 'GTEX-15CHS-0011-R2a-SM-6EU2X', 'GTEX-OHPN-0011-R5A-SM-2IZJ6', 'GTEX-12WSB-0226-SM-5GCOR', 'GTEX-13G51-0826-SM-5K7VB', 'GTEX-UPJH-0326-SM-CKJTF', 'GTEX-NPJ8-0011-R1a-SM-5SI8B', 'GTEX-WVLH-0011-R2A-SM-3MJFJ', 'GTEX-PLZ4-0006-SM-2S1NY', 'GTEX-15CHR-0002-SM-6RBJC', 'GTEX-1MGNQ-0001-SM-DE1IL', 'GTEX-PWCY-0005-SM-33HBP', 'GTEX-RVPU-1526-SM-47JXQ', 'GTEX-ZYWO-0126-SM-5GZWQ', 'GTEX-ZF28-0002-SM-4YUW3', 'GTEX-QEG4-0003-SM-2MRI5', 'GTEX-YF7O-0002-SM-53IUD', 'GTEX-12WS9-0626-SM-5FQT2', 'GTEX-15ER7-0002-SM-6RBKV', 'GTEX-T6MN-0626-SM-5S2SS', 'GTEX-13O21-0003-SM-5HJPF', 'GTEX-1AX9K-0001-SM-7J39L', 'GTEX-1LSNM-5016-SM-AN498', 'GTEX-11WQC-0003-SM-5DWTG', 'GTEX-POYW-0826-SM-2XV4Y', 'GTEX-12WSM-0011-R10a-SM-AHZ6F', 'GTEX-14PJ5-0126-SM-5ZZW1', 'GTEX-TMKS-0126-SM-3DB7S', 'GTEX-14JIY-1126-SM-CYKOI', 'GTEX-15EO6-0011-R4a-SM-7KULH', 'GTEX-1H23P-0826-SM-9WYU4', 'GTEX-1313W-0004-SM-7DRP8', 'GTEX-12WSN-0004-SM-6WSBR', 'GTEX-RM2N-0002-SM-5SOEB', 'GTEX-13FTW-0003-SM-5HJRT', 'GTEX-17MF6-0006-SM-7MXT6', 'GTEX-REY6-0004-SM-2RXEZ', 'GTEX-18A7B-0004-SM-DJWXU', 'GTEX-13OW5-0003-SM-5HJQ7', 'GTEX-RWS6-2426-SM-2XU8G', 'GTEX-12WSL-5001-SM-AN3XD', 'GTEX-WZTO-0002-SM-3PYL2', 'GTEX-19HZE-2326-SM-7189J', 'GTEX-12WSE-0011-R10a-SM-AHZMZ', 'GTEX-W5WG-0003-SM-3G3TI', 'GTEX-ZY6K-0002-SM-58Q79', 'GTEX-ZE7O-0002-SM-53UBF', 'GTEX-15CHS-0526-SM-CYKOJ', 'GTEX-1RAZS-0004-SM-DCXS3', 'GTEX-RUSQ-0526-SM-2TWCI', 'GTEX-1AX8Y-0326-SM-73KYY', 'GTEX-POYW-1226-SM-325HG', 'GTEX-13G51-0004-SM-6WSBK', 'GTEX-T5JC-0826-SM-5S2SN', 'GTEX-1M4P7-0004-SM-DE1JT', 'K-562-SM-2QU31', 'GTEX-15EO6-0011-R10a-SM-AHZMY', 'GTEX-13NYS-0003-SM-6WSB2', 'GTEX-ZP4G-0002-SM-53UBM', 'GTEX-YJ8A-0002-SM-6UH3B', 'GTEX-1LVAO-0003-SM-DJWXB', 'GTEX-14E6D-0002-SM-7DROV', 'GTEX-RNOR-2226-SM-2TWDA', 'GTEX-139UC-0008-SM-5ZZUG', 'K-562-SM-5QGOR', 'GTEX-PSDG-0926-SM-2M48V', 'GTEX-S33H-0004-SM-5JK2W', 'GTEX-S4P3-1626-SM-2XUAB', 'GTEX-T6MO-0001-SM-3UZF7', 'GTEX-13S7M-0003-SM-5DU58', 'GTEX-13QJC-0003-SM-5HJQ8', 'GTEX-YEC3-0004-SM-4YUXB', 'GTEX-14PJO-0003-SM-7DRON', 'GTEX-19HZE-0002-SM-7MD4L', 'GTEX-OIZF-0004-SM-2BWYY', 'GTEX-OIZG-0526-SM-2AXUH', 'GTEX-1IDJF-1526-SM-A8N8Z', 'GTEX-146FR-0004-SM-5DU4Q', 'GTEX-R3RS-0001-SM-2XK2Z', 'K-562-SM-4R59E', 'GTEX-ZTTD-0002-SM-53UBC', 'GTEX-12WS9-0011-R10a-SM-57WBV', 'GTEX-OHPL-0003-SM-2XJZS', 'GTEX-OIZF-1426-SM-2AXUE', 'GTEX-13NYS-0003-SM-5HJPP', 'GTEX-13JUV-0001-SM-5HJPQ', 'GTEX-1399Q-0004-SM-5ANGQ', 'GTEX-1C6VR-0003-SM-7MA6D', 'GTEX-18QFQ-0003-SM-DINAU', 'GTEX-1GN1U-0002-SM-7J3A7', 'GTEX-R55F-0326-SM-3G3SE', 'GTEX-ZF2S-1026-SM-7KFSP', 'GTEX-VUSG-0003-SM-5S2S5', 'GTEX-15CHS-0626-SM-6AJA3', 'GTEX-17JCI-0004-SM-6RBKB', 'GTEX-V1D1-0001-SM-4X62Q', 'GTEX-19HZE-0002-SM-7J39O', 'GTEX-1AX8Z-1326-SM-7DHMO', 'GTEX-ZQG8-0008-SM-4YCE7', 'K-562-SM-2IZHW', 'GTEX-148VJ-2526-SM-5TDEI', 'GTEX-139UW-0003-SM-6WSC8', 'GTEX-14A6H-0003-SM-5GYBV', 'GTEX-1J8Q2-2826-SM-A8N94', 'GTEX-13NYS-3126-SM-CYKVR', 'GTEX-1497J-5019-SM-AN41W', 'GTEX-OHPM-2626-SM-5SI7I', 'GTEX-P44H-0011-R10a-SM-AHZM7', 'GTEX-ZYFD-0001-SM-5DWS6', 'GTEX-14PJ3-0626-SM-7KFRZ', 'GTEX-ZYWO-0226-SM-5GZX3', 'GTEX-1314G-0003-SM-7DRPN', 'GTEX-1GN1W-0001-SM-7MA7D', 'GTEX-POMQ-2326-SM-5S2VS', 'GTEX-V955-0001-SM-5SOEE', 'GTEX-PLZ5-0006-SM-2S1NZ', 'GTEX-OXRO-0011-R9A-SM-5S2W3', 'GTEX-14PJM-0011-R1a-SM-6AJAG', 'GTEX-U8XE-0004-SM-3USQS', 'GTEX-U412-0001-SM-5SOEO', 'GTEX-1MUQO-0004-SM-DJWY6', 'GTEX-14PJN-0002-SM-7DRP3', 'GTEX-Y3I4-0726-SM-4TT7M', 'GTEX-15RIE-5013-SM-AN43X', 'GTEX-OXRK-0004-SM-2H2WZ', 'GTEX-SUCS-0001-SM-5URDQ', 'GTEX-X62O-0001-SM-3PYL1', 'K-562-SM-3DB9M', 'GTEX-17HGU-0001-SM-6UH1W', 'GTEX-P4PQ-2626-SM-2D445', 'GTEX-PLZ6-1426-SM-5S2TR', 'GTEX-VUSG-0326-SM-5S2ST', 'GTEX-QV44-0326-SM-4R1KD', 'GTEX-14BIL-0004-SM-5DU4U', 'GTEX-OXRO-0011-R5A-SM-2D45J', 'GTEX-1PWST-2926-SM-DKTQ4', 'GTEX-1S82Y-0004-SM-DE1IX', 'GTEX-131XF-5010-SM-AN3XQ', 'GTEX-146FR-1326-SM-5SIAJ', 'GTEX-1H3NZ-0011-R3b-SM-CMKH2', 'GTEX-OXRL-1626-SM-2M47M', 'GTEX-1JMQL-0004-SM-DJWX7', 'GTEX-YEC3-2526-101809-SM-5CVLT', 'GTEX-1AX8Y-2026-SM-73KUA', 'GTEX-ZYWO-1626-SM-5E44H', 'GTEX-13W3W-0004-SM-5HJPI', 'GTEX-XBED-0004-SM-3PYKD', 'GTEX-P4QR-0326-SM-5SI8P', 'K-562-SM-2XU9E', 'GTEX-PLZ5-1726-SM-2M48L', 'GTEX-WYBS-0126-SM-3PZ7C', 'GTEX-14A6H-5006-SM-7EWE9', 'GTEX-QDT8-0926-SM-2XV59', 'GTEX-N7MS-2326-SM-2AXU4', 'GTEX-QCQG-2026-SM-2QU3K', 'GTEX-1EWIQ-0011-R8a-SM-9OSX8', 'GTEX-XUJ4-0005-SM-4BOQ6', 'GTEX-13N2G-0001-SM-5DU64', 'GTEX-UPIC-0002-SM-3NMDC', 'GTEX-139T6-0226-SM-DK2JP', 'GTEX-15DCD-0626-SM-CYKUG', 'GTEX-OIZH-0003-SM-2XK32', 'GTEX-15CHS-0326-SM-6AJ9V', 'GTEX-QVUS-0226-SM-2TWBX', 'GTEX-1IE54-0004-SM-G8OXA', 'GTEX-SNMC-0006-SM-2XV4M', 'GTEX-OIZG-1526-SM-2YUNG', 'GTEX-1HKZK-2226-SM-A8NA8', 'GTEX-WK11-0002-SM-5SOET', 'GTEX-1ICLZ-0003-SM-DC69S', 'GTEX-PWOO-1326-SM-48TCJ', 'GTEX-X88G-0003-SM-4YUXA_1', 'GTEX-12696-3026-SM-5FQTU', 'GTEX-NL3H-0011-R10A-SM-CYKWA', 'GTEX-WWTW-0426-SM-DMRPS', 'GTEX-14PJ2-0002-SM-5GYE2', 'GTEX-1EX96-0004-SM-7MA72', 'GTEX-POMQ-0006-SM-2M48J', 'GTEX-Q2AH-0004-SM-5JK52', 'GTEX-P44G-0526-SM-DMRPR', 'GTEX-SIU7-1126-SM-2XV72', 'GTEX-13OVI-1826-SM-5IFFT', 'GTEX-QV31-1626-SM-2TO5A', 'GTEX-S4UY-0526-SM-3K2AN', 'GTEX-SIU7-1126-SM-2XCDW', 'GTEX-X4XX-0004-SM-3PYL3', 'GTEX-13IVO-0004-SM-5DU54', 'GTEX-XV7Q-0004-SM-53IV4', 'GTEX-1H11D-0003-SM-AG5YR', 'GTEX-15TU5-1126-SM-6M47J', 'GTEX-ZZ64-0004-SM-6WBT6', 'GTEX-RVPU-2226-SM-2XCAQ', 'K-562-SM-7938F', 'GTEX-P4PQ-1626-SM-2D43W', 'GTEX-WRHU-0003-SM-3PYKC', 'GTEX-13FTZ-0226-SM-CYKR9', 'GTEX-P4PP-0526-SM-2M49R', 'K-562-SM-5O98J', 'GTEX-WOFL-0003-SM-6WBS3', 'GTEX-1A3MW-2526-SM-72D5P', 'GTEX-148VJ-1526-SM-CYKOS', 'GTEX-16A39-5001-SM-AN44B', 'GTEX-12WSB-0011-R11b-SM-57WBJ', 'GTEX-XYKS-2926-SM-4E3IA', 'GTEX-1N2EE-0002-SM-DE1IR', 'GTEX-1N5O9-0004-SM-DE1KM', 'GTEX-17MF6-0002-SM-6UH21', 'GTEX-1JMPY-0003-SM-DJWYO', 'GTEX-OIZF-1526-SM-2AXUI', 'GTEX-U8T8-2026-SM-7DRPP', 'GTEX-15F5U-5003-SM-AN43O', 'GTEX-RVPU-0011-R2A-SM-2XCAG', 'GTEX-1192X-1026-SM-5H12P', 'GTEX-1269C-1826-SM-DO3X2', 'GTEX-OXRL-0003-SM-2H2X2', 'GTEX-13O3Q-2826-SM-5KM2N', 'GTEX-WHSE-0926-SM-CYKVI-SUP', 'GTEX-OXRL-0003-SM-4X611', 'GTEX-1M4P7-0011-R3a-SM-EYYX2', 'K-562-SM-4WKF1', 'K-562-SM-2TC5E', 'GTEX-13X6I-0002-SM-5HJQC', 'GTEX-15CHQ-0011-R1a-SM-6AJAH', 'GTEX-OHPJ-0006-SM-2D459', 'GTEX-ZLWG-2226-SM-5DUWO', 'GTEX-11EMC-0004-SM-5DWTM', 'GTEX-S4Q7-0001-SM-5JK3A', 'GTEX-1S82U-0004-SM-DJWY5', 'GTEX-ZVT2-1526-SM-51MQZ', 'GTEX-117YW-0003-SM-58Q99', 'GTEX-1339X-1226-SM-5PNVO', 'GTEX-1J8QM-1426-SM-A8N9B', 'GTEX-ZTSS-1826-SM-57WEK', 'GTEX-Y5V5-0002-SM-4YUV2', 'GTEX-XUJ4-0004-SM-4BOQE', 'GTEX-13N11-5028-SM-AN3Z7', 'GTEX-1GF9U-0004-SM-7MA6K', 'GTEX-P4QR-0004-SM-3USRX', 'GTEX-13FLW-0003-SM-6WSC2', 'GTEX-1PWST-2926-SM-DLT2H', 'GTEX-N7MS-0011-R1a-SM-2AXVJ', 'GTEX-13NZ8-3026-SM-5L3D4', 'GTEX-UPK5-0002-SM-3G3TM', 'GTEX-YJ8A-0002-SM-53UB6', 'GTEX-RVPV-0011-R3A-SM-2XUAE', 'GTEX-12126-0001-SM-58Q8L', 'GTEX-12WS9-0126-SM-5EGL7', 'GTEX-13RTK-0003-SM-5DU57', 'GTEX-ZXES-0001-SM-58Q76', 'K-562-SM-5Q5CL', 'GTEX-RVPV-0006-SM-2XUA6', 'GTEX-PWN1-1826-SM-2QU3G', 'GTEX-XXEK-0001-SM-53IV5', 'GTEX-16NPV-0003-SM-6RBJY', 'GTEX-145MI-3126-SM-5Q5CF', 'GTEX-1GMR3-5019-SM-AN46K', 'GTEX-13CIG-0004-SM-5HJRO', 'GTEX-O5YW-0003-SM-2XK1J', 'GTEX-ZT9W-0726-SM-4YCDC', 'GTEX-1122O-0004-SM-6WBTE', 'GTEX-OXRN-0003-SM-2XK1N', 'GTEX-P4PP-1826-SM-2M47G', 'GTEX-PLZ6-1426-SM-2S1OQ', 'GTEX-R55G-0003-SM-3USRS', 'GTEX-11XUK-0003-SM-6WBU7', 'GTEX-XXEK-0001-SM-5JK35', 'GTEX-RVPU-2426-SM-2XU9M', 'GTEX-WRHU-0003-SM-5JK4F', 'K-562-SM-5J2LX', 'GTEX-14A6H-5010-SM-79OOV', 'GTEX-14PJP-0001-SM-7DLEZ', 'GTEX-15ER7-0926-SM-CYKTS-INPUT', 'GTEX-P4PP-0226-SM-5S2VR', 'GTEX-1EKGG-0004-SM-DIND5', 'GTEX-OIZI-0226-SM-2XV4P', 'GTEX-NPJ7-0011-R5a-SM-2I5GY', 'GTEX-PWCY-1826-SM-5SI7U', 'GTEX-12BJ1-0001-SM-7DRPO', 'GTEX-S4Q7-0006-SM-2XUA8', 'GTEX-S4Z8-1426-SM-4AD5G', 'GTEX-1ICG6-0004-SM-DC69L', 'GTEX-NPJ7-0011-R4a-SM-2AXW1', 'GTEX-SNMC-1526-SM-2XV4X', 'GTEX-14JIY-1326-SM-CYKT3', 'GTEX-1PFEY-0003-SM-DE1IB', 'GTEX-ZVZQ-0011-R10a-SM-AHZNV', 'GTEX-WHPG-0002-SM-3PYK5', 'GTEX-15TU5-0011-R10b-SM-793BL', 'GTEX-QESD-0526-SM-2M498', 'GTEX-1LGOU-5007-SM-AN49H', 'GTEX-ZYFD-3126-SM-5E44O', 'GTEX-OXRO-0003-SM-5JK2R', 'GTEX-RTLS-0004-SM-2RXEU', 'GTEX-1B8KE-5001-SM-AN461', 'GTEX-POMQ-2026-SM-2QU3S', 'GTEX-13CF3-0004-SM-6WSD5', 'GTEX-PW2O-0004-SM-4X61N', 'GTEX-13FTZ-0004-SM-5DU3W', 'GTEX-OXRN-0005-SM-2I5EU', 'GTEX-15D79-0004-SM-7DLH2', 'GTEX-S4UY-1026-SM-4AD53', 'GTEX-WZTO-0011-R10a-SM-AHZM6', 'GTEX-VUSG-2426-SM-5S2S2', 'GTEX-15EOM-5010-SM-AN43D', 'GTEX-RWSA-0003-SM-2XK2E', 'GTEX-Y8E5-0001-SM-53IVA', 'GTEX-16Z82-1026-SM-CYKSW', 'GTEX-WY7C-0002-SM-3PYKE', 'GTEX-ZYFD-0001-SM-58Q7A', 'GTEX-16MT8-0002-SM-6RBLH', 'GTEX-R55F-0005-SM-2TWC8', 'GTEX-12584-0004-SM-5DWTJ', 'GTEX-OHPN-0011-R8A-SM-2IZJ9', 'GTEX-14LZ3-0011-R8b-SM-69LOZ', 'GTEX-15CHS-0011-R6a-SM-6AJC1', 'GTEX-ZE7O-0002-SM-6UH3I', 'GTEX-14PHY-5001-SM-AN43I', 'GTEX-1PPH7-0004-SM-DLIPD', 'GTEX-OIZG-0003-SM-4X5ZK', 'GTEX-T8EM-0001-SM-4SVPM', 'GTEX-T5JC-0626-SM-5S2RW', 'GTEX-12WS9-3026-SM-5EGIE', 'GTEX-S7SE-2526-SM-2XV6R', 'GTEX-PWOO-2426-SM-2S1OV', 'GTEX-XOTO-2826-SM-4B65I', 'GTEX-14E6E-0001-SM-7DRNZ', 'GTEX-YB5E-0003-SM-AYKVK', 'GTEX-Y114-1126-SM-CKJTJ', 'GTEX-WWYW-0011-R10a-SM-AHZNT', 'GTEX-U3ZM-0004-SM-3G3TG', 'GTEX-ZDYS-2126-SM-5S2OD', 'GTEX-OOBJ-0326-SM-2I3F8', 'GTEX-PX3G-0226-SM-2S1OU', 'GTEX-131XG-0003-SM-5DU5J', 'GTEX-NPJ8-0011-R2a-SM-2AXV6', 'GTEX-11EI6-0826-SM-CYKV8', 'GTEX-S7SE-0011-R10A-SM-CYKW9', 'GTEX-RU72-0011-R11A-SM-2TWD7', 'GTEX-QVUS-0006-SM-2TWBV', 'GTEX-XMK1-0003-SM-4YUWQ', 'GTEX-QV31-0003-SM-3USS6', 'GTEX-S4Z8-1026-SM-72D6N', 'GTEX-XUJ4-2726-SM-4BOQ1', 'GTEX-1RDX4-2926-SM-E8VOF', 'GTEX-OXRP-2426-SM-2M47D', 'GTEX-1QP67-0004-SM-DLIP9', 'GTEX-NL3H-0011-R6a-SM-2AXVQ', 'GTEX-YB5K-0001-SM-4YUVM', 'GTEX-1I1GV-1026-SM-A8N8D', 'GTEX-P44G-0626-SM-2D44I', 'GTEX-12WSM-0001-SM-5ANFP', 'GTEX-VJWN-0003-SM-3G3S4', 'GTEX-139TU-0926-SM-DK2K2', 'GTEX-QEG4-2226-SM-4R1JM', 'GTEX-17HII-0004-SM-6RBK8', 'GTEX-12WS9-0011-R2a-SM-5CVNE', 'GTEX-1R9K5-0002-SM-DCXS2', 'GTEX-P4QR-0426-SM-2S1NV', 'GTEX-15SDE-5013-SM-AN443', 'GTEX-PWOO-0726-SM-2I3EB', 'GTEX-WFON-0626-SM-5S2SX', 'GTEX-S3XE-0426-SM-2XUAK', 'GTEX-12BJ1-5022-SM-AN3XA', 'GTEX-RWS6-0001-SM-3NMAL', 'GTEX-1LNCM-0004-SM-DE1JR', 'GTEX-UPIC-1626-SM-4IHKT', 'GTEX-X585-0003-SM-3USRA', 'GTEX-12WSK-0003-SM-5DWRQ', 'GTEX-WYVS-0002-SM-3PYKP', 'GTEX-S4P3-0004-SM-2VCTW', 'GTEX-13SLW-0003-SM-5HJRH', 'GTEX-OHPK-1626-SM-2M47T', 'GTEX-1PPH8-0003-SM-DE1JH', 'GTEX-ZGAY-0002-SM-53UB9', 'GTEX-1AX8Y-0226-SM-73KX1', 'GTEX-Q2AH-0426-SM-2IZIE', 'GTEX-15EU6-0003-SM-6RBJQ', 'GTEX-11P82-0003-SM-5DWRC', 'GTEX-13G51-0004-SM-5HJPG', 'GTEX-QV31-0526-SM-447C3', 'GTEX-QVJO-1726-SM-2TO4L', 'GTEX-WFG8-0004-SM-5URCY', 'GTEX-P4PQ-0005-SM-2D45X', 'GTEX-132QS-5001-SM-AN3Y5', 'GTEX-13PVR-5001-SM-AN416', 'GTEX-11UD2-0001-SM-58Q8A', 'GTEX-S95S-1526-SM-2XV56', 'GTEX-QDT8-0126-SM-CKJTD', 'GTEX-16NFA-0003-SM-6UH2Q', 'GTEX-1HBPM-0011-R4a-SM-CJI3L', 'GTEX-RTLS-0626-SM-2TF6G', 'GTEX-WHPG-0002-SM-3PZ77', 'GTEX-17F9Y-0004-SM-6UH1O', 'GTEX-U3ZN-0001-SM-3G3SR', 'GTEX-11TTK-0002-SM-5DWT4', 'GTEX-PX3G-0004-SM-2IJGE', 'GTEX-145LT-5013-SM-AN41U', 'GTEX-S4Q7-1526-SM-3K2AG', 'GTEX-WFON-2326-SM-5S2SF', 'GTEX-1NV5F-0004-SM-DE1IO', 'GTEX-11XUK-5016-SM-AN3X4', 'GTEX-Q2AH-1826-SM-2QU55', 'GTEX-POMQ-2126-SM-2QU4E', 'GTEX-14PJ4-0004-SM-7DROM', 'GTEX-PSDG-0726-SM-2I5FN', 'GTEX-19HZE-2126-SM-72D4X', 'GTEX-14PJM-1126-SM-CYKSX', 'GTEX-15D79-0004-SM-6RBJR', 'GTEX-X62O-0001-SM-3PZ7O', 'GTEX-R55F-0011-R5A-SM-2TWCL', 'GTEX-RM2N-0002-SM-2RXEV', 'GTEX-PLZ6-0006-SM-2I5FQ', 'GTEX-13NZA-0003-SM-5DU5O', 'GTEX-13FTW-0003-SM-5CV8M', 'K-562-SM-9HWP9', 'GTEX-14C39-0326-SM-CYKOQ', 'GTEX-QV44-0003-SM-2MRI4', 'GTEX-OHPJ-0226-SM-2D45E', 'K-562-SM-9HWP4', 'K-562-SM-9HWP7', 'GTEX-RUSQ-0001-SM-3UZFR', 'GTEX-ZDTT-2226-SM-4WKFT', 'K-562-SM-9HWPM', 'K-562-SM-9HWPN', 'GTEX-ZVTK-0002-SM-4YUW7', 'K-562-SM-9HWPF', 'K-562-SM-9HWPY', 'GTEX-14PJO-0426-SM-6LLJQ', 'K-562-SM-9HWPZ', 'GTEX-1IKOH-0003-SM-G8OXK', 'GTEX-PSDG-0626-SM-2QU42', 'K-562-SM-9HWPW', 'K-562-SM-9HWPV', 'GTEX-WFON-0003-SM-3G3S8', 'GTEX-OXRP-0003-SM-2XJZW', 'GTEX-QVJO-0011-R5A-SM-2TO63', 'GTEX-QLQ7-0626-SM-4R1JT', 'GTEX-1497J-0004-SM-5JVBR', 'K-562-SM-3K2BF', 'GTEX-1S5VW-0001-SM-DE1IW', 'GTEX-QDVJ-2126-SM-5S2U2', 'GTEX-1HBPN-1826-SM-A8N9R', 'GTEX-OHPN-0011-R10A-SM-2IZJA', 'GTEX-16XZZ-3026-SM-CYKT9', 'GTEX-T6MN-0011-R11A-SM-5S2TH', 'GTEX-OXRK-0004-SM-5JK32', 'GTEX-13NYS-0003-SM-5DU5X', 'GTEX-12ZZW-0003-SM-6WSBT', 'GTEX-P4QS-2626-SM-2D43V', 'GTEX-1I1GQ-2926-SM-A8N83', 'GTEX-11P7K-0002-SM-58Q9D', 'GTEX-1HBPN-0003-SM-AG5Y8', 'GTEX-PX3G-0226-SM-3NB2C', 'GTEX-1IDJD-5001-SM-AN47T', 'GTEX-11DXZ-0003-SM-6WBTM', 'GTEX-1GZHY-0001-SM-7MA6Z', 'GTEX-WY7C-0002-SM-3USRD', 'GTEX-1K2DU-0003-SM-DJWWE', 'GTEX-QEG4-0006-SM-2M493', 'GTEX-13N2G-0011-R1b-SM-4RGJU-hHP1', 'GTEX-13N2G-0011-R1b-SM-4RGJU-hHP2', 'GTEX-15SZO-0004-SM-6UH2X', 'GTEX-T2IS-0002-SM-3259Y', 'GTEX-1AMFI-0001-SM-7J39V', 'GTEX-QV44-1226-SM-447CJ', 'GTEX-QEL4-0226-SM-2TWAW', 'GTEX-WHSE-1726-SM-4X61H', 'GTEX-WHPG-1326-SM-4M1ZJ', 'GTEX-15SHU-1226-SM-7KFSO', 'GTEX-14BMU-5001-SM-AN427', 'GTEX-OIZH-2626-SM-2D44X', 'GTEX-12WS9-0526-SM-5FQT5', 'GTEX-15CHS-0011-R11b-SM-68711', 'GTEX-1JMQL-0526-SM-CM2RG', 'GTEX-1LKK1-0626-SM-DKTRX', 'GTEX-1LVA9-0004-SM-DJWYS', 'GTEX-11PRG-0004-SM-5DWRB', 'GTEX-1JMI6-0002-SM-DE1KA', 'GTEX-13NYS-0011-R10a-SM-AHZ6V', 'GTEX-XXEK-1326-SM-5S2SJ', 'GTEX-1LSNL-0004-SM-DJWYH', 'GTEX-1CB4I-5016-SM-AN45Z', 'GTEX-XUJ4-2826-SM-4BOQ2', 'GTEX-15DYW-0002-SM-7DLFW', 'GTEX-12WSL-0003-SM-5ANFZ', 'GTEX-XUJ4-0526-SM-4BOON', 'GTEX-1EWIQ-0011-R11a-SM-9OSX6', 'GTEX-VUSG-2826-SM-5S2RZ', 'GTEX-YEC3-2026-SM-5CVLQ', 'GTEX-OXRN-0003-SM-5JK51', 'GTEX-T2IS-0002-SM-5SOEK', 'GTEX-WVLH-0001-SM-3PYKY', 'GTEX-WYVS-0002-SM-3USR9', 'GTEX-RN64-1226-SM-2TO5D', 'GTEX-SIU7-0002-SM-4SVPJ', 'GTEX-1S831-0002-SM-DE1J5', 'GTEX-ZYY3-0002-SM-5DWTT', 'GTEX-13O3P-0003-SM-5GYAO', 'GTEX-1H3VY-5007-SM-AN46L', 'GTEX-S341-0326-SM-2XUAR', 'GTEX-1C6VR-0003-SM-7J39D', 'GTEX-QMRM-0226-SM-4R1K7', 'GTEX-16A39-0004-SM-6UH2P', 'K-562-SM-57WAW', 'GTEX-12WSE-0011-R10b-SM-CYKPY', 'GTEX-1C6VS-0003-SM-DJWWB', 'GTEX-13CZU-5001-SM-AN3YV', 'GTEX-T6MN-0005-SM-5S2T2', 'GTEX-1117F-2226-SM-5N9CH', 'GTEX-13FTX-0004-SM-6WSC4', 'GTEX-RVPU-0011-R1A-SM-2XUA7', 'GTEX-1399U-1326-SM-DK2JO', 'GTEX-11GSO-0011-R10a-SM-AHZ7H', 'GTEX-14JIY-2926-SM-CYKO5', 'GTEX-11NUK-0004-SM-58Q87', 'GTEX-1HBPH-0011-R11b-SM-A8N7U', 'GTEX-1F48J-0003-SM-7J3AS', 'GTEX-YEC3-0826-SM-5P9JF', 'GTEX-XUJ4-1526-SM-4BONU', 'GTEX-1HFI6-2826-SM-A8N9T', 'GTEX-11ZTT-0003-SM-5DWTB', 'GTEX-14753-0003-SM-5GYBX', 'GTEX-RWSA-0005-SM-2XU8W', 'GTEX-1GN73-0002-SM-7J3AZ', 'GTEX-YEC3-2326-SM-5IFJF', 'GTEX-ZT9W-0002-SM-6WBSS', 'GTEX-N7MS-0011-R3a-SM-2AXVU', 'GTEX-11EI6-0004-SM-5DWSG', 'GTEX-OXRP-0003-SM-2H2X8', 'GTEX-139TT-0004-SM-5DU4N', 'GTEX-QVJO-3126-SM-4R1KC', 'GTEX-ZP4G-1026-SM-4YCE1', 'GTEX-1MGNQ-0001-SM-DJWXI', 'GTEX-UPIC-0004-SM-4X615', 'GTEX-OOBK-0004-SM-3USRV', 'GTEX-WYJK-0001-SM-3UZEW', 'GTEX-PLZ6-0626-SM-5J1P1', 'GTEX-S341-1626-SM-3K2B8', 'GTEX-13FH7-2026-SM-5IJCR', 'GTEX-RU72-0006-SM-2TWCA', 'GTEX-1M5QR-0003-SM-DE1IS', 'GTEX-RU1J-0006-SM-2TWC6', 'GTEX-1HBPH-2926-SM-DK2JY', 'GTEX-148VJ-0011-R10a-SM-AHZ6U', 'GTEX-QMRM-0005-SM-2S1QR', 'GTEX-XBED-0004-SM-3PZ7G', 'GTEX-VUSG-0002-SM-DLIPX', 'GTEX-ZVZO-0001-SM-58Q78', 'GTEX-X62O-0001-SM-58Q8X', 'GTEX-1212Z-0004-SM-58Q8P', 'GTEX-145MN-0001-SM-5GYC2', 'GTEX-12WSK-5015-SM-AN3Y2', 'GTEX-QV44-1926-SM-2TO4T', 'GTEX-ZDYS-0005-SM-4YUWR', 'GTEX-13FLV-0011-R10b-SM-CYKS9', 'GTEX-14JG1-0002-SM-5HX8M', 'K-562-SM-E9EZI', 'GTEX-11OC5-0526-SM-EZ6XW', 'K-562-SM-E9EZO', 'GTEX-QVJO-0226-SM-2TO4P', 'K-562-SM-E9EZC', 'GTEX-PLZ5-0626-SM-2M48N', 'K-562-SM-E9EZZ', 'GTEX-12WS9-0226-SM-5EQ4X', 'GTEX-PLZ4-0006-SM-2M47Z', 'K-562-SM-E9EZT', 'GTEX-11OF3-3126-SM-5GU5E', 'GTEX-18A67-0003-SM-DJWXP', 'GTEX-QDVJ-0926-SM-2M48Y', 'GTEX-13D11-0003-SM-5HJQR', 'GTEX-1AYD5-0003-SM-7MD4M', 'GTEX-N7MT-0011-R2a-SM-2AXW5', 'GTEX-15DCD-0001-SM-7DLGT', 'GTEX-OXRL-0003-SM-5JK3X', 'GTEX-1CAMR-0004-SM-7J3B6', 'GTEX-QDVN-0003-SM-5URBU', 'GTEX-R55E-0426-SM-2TO4J', 'GTEX-16GPK-0003-SM-6UH2S', 'GTEX-1HB9E-0004-SM-AG5YT', 'GTEX-139UC-0011-R11a-SM-5KM3J', 'GTEX-POMQ-2226-SM-2M48S', 'GTEX-PLZ5-0826-SM-4E3JR', 'GTEX-ZYWO-1526-SM-5SIAL', 'GTEX-15D1Q-0004-SM-6RBJB', 'GTEX-P4QT-1826-SM-2M478', 'K-562-SM-79OJI', 'GTEX-1192X-0011-R11b-SM-DO935', 'GTEX-15RIE-0004-SM-6RBKY', 'GTEX-RNOR-0001-SM-5URCF', 'GTEX-P4QT-1626-SM-2S1NP', 'GTEX-SIU7-1526-SM-4BRWQ', 'GTEX-19HZE-2926-SM-718BQ', 'GTEX-T5JC-0011-R6A-SM-32PMI', 'GTEX-13PL7-5010-SM-AN3ZR', 'GTEX-S4UY-0526-SM-2XUA3', 'GTEX-O5YT-0010-SM-325HV', 'GTEX-13N2G-1226-SM-5IJCI', 'GTEX-ZAB4-0001-SM-6UH36', 'GTEX-S33H-0004-SM-2XK2J', 'GTEX-NL4W-0011-R5a-SM-2AXVZ', 'GTEX-OIZI-1026-SM-CLTGE', 'GTEX-ZYW4-0002-SM-6WBT8', 'GTEX-ZUA1-0001-SM-4YUVH', 'GTEX-15UF6-0011-R10a-SM-AHZLZ', 'GTEX-12WSB-2926-SM-59HKL', 'GTEX-SSA3-0001-SM-325A5', 'GTEX-O5YV-2026-SM-26GN2', 'K-562-SM-5RQHM', 'GTEX-OIZH-0003-SM-2BWZ6', 'GTEX-1IKOE-1226-SM-A8N98', 'GTEX-V1D1-0001-SM-3G3SG', 'GTEX-16YQH-1926-SM-6M47L', 'GTEX-111YS-0004-SM-58Q7Y', 'GTEX-R53T-2026-SM-2TO4X', 'GTEX-1F5PK-0003-SM-7J3AQ', 'GTEX-15DZA-0004-SM-6RBJI', 'GTEX-ZC5H-0003-SM-4YUW1', 'GTEX-131XH-0003-SM-5DU5M', 'GTEX-NPJ8-0426-SM-2AXU1', 'GTEX-YEC3-1926-SM-5IFIT', 'GTEX-146FQ-0003-SM-6WSCH', 'GTEX-145MN-5019-SM-AN41O', 'GTEX-ZV68-0001-SM-6WBS8', 'GTEX-1GF9U-5001-SM-AN46D', 'GTEX-1PPGY-0004-SM-DE1J9', 'GTEX-ZF28-0002-SM-53UC2', 'K-562-SM-4W1Y5', 'GTEX-13111-0003-SM-6WSCF', 'GTEX-13S86-0003-SM-5HJPD', 'GTEX-15RJ7-5001-SM-AN43W', 'GTEX-S4Q7-0001-SM-2XK2L', 'GTEX-X3Y1-0002-SM-3PZ85', 'GTEX-11EQ8-0001-SM-5DWSU', 'GTEX-QDVN-0003-SM-2XK1Z', 'GTEX-YECK-2526-SM-EXUR2', 'GTEX-ZVTK-0002-SM-6WBSA', 'GTEX-ZF3C-0002-SM-6UH3H', 'GTEX-1AX9I-0002-SM-7J39K', 'GTEX-N7MS-0426-SM-2QU34', 'GTEX-XAJ8-0001-SM-3PYK8', 'GTEX-ZQUD-1526-SM-7EWD9', 'GTEX-145LU-0004-SM-DINCL', 'GTEX-XOTO-0126-SM-4B66N', 'GTEX-P4QR-0426-SM-2M47O', 'GTEX-S4UY-0226-SM-3K2AP', 'GTEX-XQ8I-0001-SM-4YUVE', 'GTEX-13OW7-0926-SM-CYKRR', 'GTEX-NL3H-0009-SM-2BWYN', 'GTEX-X8HC-0003-SM-3PYKU', 'GTEX-T5JC-0226-SM-5S2SM', 'GTEX-WYVS-0002-SM-5URCN', 'GTEX-139T6-0004-SM-5HJQN', 'GTEX-N7MT-1126-SM-5SI7V', 'GTEX-11WQC-0003-SM-6WBU8', 'GTEX-13O3Q-0003-SM-6WSCX', 'GTEX-O5YW-0326-SM-2D45D', 'GTEX-13OVL-0003-SM-6WSD4', 'GTEX-R55G-2126-SM-2TWB4', 'GTEX-1211K-0003-SM-5DWTL', 'GTEX-1J8Q3-0826-SM-A8N8W', 'GTEX-1C64O-5007-SM-AN46B', 'GTEX-R53T-0926-SM-2TO5U', 'GTEX-146FQ-5004-SM-AN41I', 'GTEX-OIZG-0005-SM-2AXUC', 'GTEX-OXRN-0526-SM-2D44N', 'GTEX-1IDJU-1426-SM-CNPQA', 'GTEX-R45C-0326-SM-2TWB2', 'GTEX-PVOW-1026-SM-CLTG9', 'GTEX-WOFM-0003-SM-3PZ7W', 'GTEX-11TUW-0011-R5b-SM-5BC6Q', 'GTEX-P44H-1126-SM-2VCU3', 'GTEX-S4Z8-0326-SM-3K2AU', 'GTEX-PSDG-0004-SM-2IJGH', 'GTEX-1F48J-2126-SM-7PBYC', 'GTEX-1192W-0001-SM-6WBTI', 'GTEX-1AX8Y-0526-SM-72D5K', 'GTEX-14PQA-0011-R10a-SM-AHZ74', 'GTEX-RTLS-0826-SM-2TWCX', 'GTEX-OHPN-0011-R10A-SM-2I5FL', 'GTEX-P78B-1626-SM-2M489', 'GTEX-S4Q7-0326-SM-2XUAU', 'GTEX-R55C-0003-SM-3USSH', 'GTEX-QCQG-2126-SM-2QU35', 'GTEX-ZAB5-2626-SM-5KM3Y', 'GTEX-13N2G-0011-R10a-SM-AHZ6N', 'GTEX-WFG8-0004-SM-3USQZ', 'GTEX-1MA7W-0004-SM-DJWYG', 'GTEX-12KS4-0003-SM-5DWRE', 'GTEX-ZLFU-0004-SM-53UBZ', 'GTEX-YF7O-0002-SM-6UH3Q', 'GTEX-18A7B-0004-SM-DINCC', 'GTEX-PWCY-0526-SM-2I3ER', 'GTEX-1EX96-0004-SM-7J3AP', 'GTEX-XGQ4-0001-SM-5SOEG', 'GTEX-11P82-0003-SM-58Q7E', 'GTEX-NL4W-0006-SM-2I3GH', 'GTEX-T5JW-0002-SM-5URDG', 'GTEX-1I1GQ-2826-SM-A8N82', 'GTEX-WFG7-0326-SM-3GILI', 'GTEX-P44G-0011-R6A-SM-2XV5M', 'GTEX-YEC4-1026-101814-SM-5P9FX', 'GTEX-UPK5-0002-SM-5SOE3', 'GTEX-16NFA-5003-SM-AN44K', 'GTEX-11H98-0426-SM-EZ6XS', 'GTEX-PWCY-0001-SM-4X5ZP', 'GTEX-Z93T-0002-SM-58Q8Z', 'K-562-SM-4M1XD', 'GTEX-R55F-0326-SM-4X616', 'GTEX-1RQED-0126-SM-E9U6M', 'GTEX-14DAQ-0004-SM-5CV9E', 'GTEX-XV7Q-0004-SM-4YUWC', 'GTEX-OIZI-0005-SM-2XV5F', 'GTEX-UJHI-0001-SM-3L287', 'GTEX-ZA64-0003-SM-53UAS', 'GTEX-1FIGZ-0003-SM-7J3AX', 'GTEX-WK11-0002-SM-CLTGB', 'GTEX-1LGRB-0003-SM-AYROG', 'GTEX-1LC46-5019-SM-AN497', 'GTEX-17GQL-5025-SM-AN455', 'GTEX-ZTX8-0004-SM-4YUXI', 'GTEX-S4UY-0003-SM-5JK3Y', 'GTEX-1IL2U-0004-SM-G8OXE', 'GTEX-S95S-0004-SM-325A4', 'GTEX-OHPN-0005-SM-2AXVX', 'GTEX-PWCY-2226-SM-2QU4X', 'GTEX-S4Q7-1426-SM-2XU8I', 'GTEX-13CF3-0004-SM-5JVC6', 'GTEX-11UD2-0001-SM-6WBTW', 'GTEX-13QBU-0003-SM-5DU3V', 'GTEX-1F75W-0004-SM-7J3AH', 'GTEX-17MFQ-0003-SM-6RBKC', 'GTEX-15DDE-0011-R10a-SM-AHZN2', 'GTEX-11GS4-0004-SM-58Q7W', 'GTEX-14BIN-0011-R10a-SM-AHZ79', 'GTEX-1K2DU-5001-SM-AN48D', 'GTEX-14JIY-0626-SM-62LF9', 'GTEX-15RIE-0004-SM-6UH2C', 'GTEX-S3XE-1726-SM-3K2AM', 'GTEX-14LZ3-0626-SM-CYKQF', 'K-562-SM-7RHFN', 'GTEX-14H4A-0003-SM-7DRO5', 'GTEX-PWN1-0126-SM-2IZIM', 'K-562-SM-7DHKI', 'GTEX-1EH9U-0126-SM-7SB87', 'GTEX-1128S-1426-SM-5N9EF', 'GTEX-PWO3-0011-R3A-SM-2IZIX', 'GTEX-S7SE-0126-SM-2XV4Z', 'GTEX-OXRP-0326-SM-2D444', 'GTEX-14E7W-1726-SM-5ZZWB', 'GTEX-R55E-0011-R5A-SM-2TO4Q', 'GTEX-13FLV-0004-SM-5DU4T', 'GTEX-1C6VQ-2326-SM-DLL3O', 'GTEX-15RIF-0004-SM-6UH2F', 'GTEX-RU72-0004-SM-2XK2S', 'GTEX-U4B1-0001-SM-3G3TT', 'GTEX-UTHO-0003-SM-3G3TQ', 'GTEX-1AMEY-0004-SM-7MD4T', 'GTEX-QV31-0003-SM-2MRI2', 'GTEX-SIU8-0226-SM-2XV4Q', 'GTEX-1I6K6-0003-SM-DC69H', 'GTEX-1IDJH-1526-SM-A8N96', 'GTEX-1A3MV-0002-SM-7J39R', 'GTEX-12ZZW-0003-SM-5ANGB', 'GTEX-PLZ6-1726-SM-2QU2Z', 'GTEX-14PJ5-1426-SM-68718', 'GTEX-PWO3-1026-SM-2IZJ1', 'GTEX-X62O-0001-SM-3USR7', 'GTEX-YEC4-2126-101817-SM-EYYVS', 'GTEX-1A8G6-0004-SM-7MD4S', 'GTEX-P4QR-0226-SM-3NMCH', 'GTEX-11ZVC-0004-SM-58Q8O', 'GTEX-TKQ1-0002-SM-3UZF2', 'GTEX-11DXW-0726-SM-CYKR5', 'GTEX-XBEW-0001-SM-4YUVV', 'GTEX-15UF7-5013-SM-AN449', 'GTEX-16Z82-0011-R10a-SM-AHZME', 'GTEX-13OW6-0002-SM-5GYAF', 'GTEX-14LZ3-0011-R6a-SM-686ZO', 'GTEX-15UF6-3026-SM-CYKTB', 'GTEX-X261-0002-SM-3PYKS', 'GTEX-14PHY-5019-SM-AN43N', 'GTEX-13FXS-0004-SM-5HJPR', 'GTEX-1OJC4-0001-SM-DJWWU', 'GTEX-NPJ8-1526-SM-2D43H', 'GTEX-Q2AG-0011-R10A-SM-2IZK9', 'GTEX-QEG4-0003-SM-5URCG', 'GTEX-16NFA-2426-SM-7KUL5', 'GTEX-1LC47-2026-SM-EV7AH', 'GTEX-1P4AB-0001-SM-DE1JX', 'GTEX-RVPU-0011-R9A-SM-3NM8E', 'GTEX-1HSGN-5007-SM-AN47A', 'GTEX-XUJ4-0926-SM-4BOPA', 'GTEX-1HFI7-1226-SM-A8N7Z', 'K-562-SM-69LO5', 'K-562-SM-5O9BA', 'GTEX-1GN1U-0011-R3a-SM-A8N9M', 'GTEX-PWOO-0526-SM-2QU5E', 'GTEX-YEC3-0526-SM-4YCF2', 'GTEX-ZYWO-0004-SM-6WBUE', 'GTEX-QVJO-1325-SM-2TO6N', 'GTEX-1339X-0226-SM-DK2K3', 'GTEX-1J8QM-5004-SM-AN49A', 'GTEX-1I1GR-2826-SM-A8N8F', 'GTEX-S341-1926-SM-2XU91', 'GTEX-139D8-0001-SM-7DROC', 'GTEX-S4UY-0006-SM-3K2A7', 'GTEX-1QMI2-0004-SM-DE1JO', 'GTEX-13PLJ-0003-SM-5HJPK', 'GTEX-QMRM-0526-SM-2M49C', 'GTEX-Q2AH-1726-SM-2S1OT', 'GTEX-1K2DA-0002-SM-ANILQ', 'GTEX-14ASI-0001-SM-5CV9M', 'GTEX-WCDI-0001-SM-5SOE7', 'GTEX-RTLS-0526-SM-2TWBW', 'GTEX-15SB6-5016-SM-AN44J', 'GTEX-1EMGI-0004-SM-DLIOK', 'GTEX-13S86-0003-SM-6WSCT', 'GTEX-16XZZ-0011-R7b-SM-7MGVT', 'GTEX-1JMPZ-0004-SM-DE1JU', 'GTEX-145LT-5001-SM-AN41S', 'GTEX-QESD-0726-SM-4R1KQ', 'GTEX-15SHU-1126-SM-CYKTR', 'GTEX-WHSE-1726-SM-7DRPJ', 'GTEX-17HHY-0002-SM-6UH1X', 'K-562-SM-4JBIQ', 'GTEX-131XE-5013-SM-AN3XX', 'GTEX-11UD2-0001-SM-5DWT3', 'K-562-SM-2D439', 'GTEX-WFON-0003-SM-3UZES', 'GTEX-1122O-0926-SM-5N9C9', 'GTEX-S7SF-0006-SM-2XUAD', 'GTEX-1KANC-0004-SM-DJWYC', 'GTEX-Q734-0001-SM-2XK1A', 'GTEX-1B98T-0726-SM-7939J', 'GTEX-PLZ6-0003-SM-2IJGD', 'GTEX-13NZA-1726-SM-5J1NA', 'GTEX-1IDJC-0004-SM-DHP12', 'GTEX-139YR-5001-SM-AN3YP', 'GTEX-ZDTS-0002-SM-6UH3E', 'GTEX-1AYCT-5004-SM-AN45J', 'GTEX-WWYW-0004-SM-3PZ7I', 'GTEX-UPK5-0326-SM-3GAF3', 'GTEX-11GS4-0011-R10a-SM-AHZ7I', 'GTEX-111FC-0011-R10a-SM-AHZ7K', 'GTEX-1EX96-0826-SM-7MGXK', 'GTEX-1JN1M-1826-SM-ARU92', 'GTEX-Q2AG-2926-SM-2IZJR', 'GTEX-14PKU-0004-SM-7DLF7', 'GTEX-OXRP-0006-SM-2IZIO', 'GTEX-P44H-1126-SM-5JK39', 'GTEX-1O97I-1126-SM-E9U69', 'GTEX-17KNJ-0003-SM-6RBJX', 'GTEX-T2IS-0002-SM-325HF', 'GTEX-SIU8-0002-SM-4SVQ6', 'GTEX-QEG5-0126-SM-4R1JR', 'GTEX-13OVI-5007-SM-AN3ZJ', 'GTEX-111FC-0001-SM-5DWSO', 'GTEX-13FXS-0004-SM-5DU61', 'GTEX-OHPM-1626-SM-2AXUZ', 'GTEX-12ZZW-0126-SM-5DUVB', 'GTEX-OXRK-0006-SM-2D43M', 'K-562-SM-731D6', 'GTEX-RVPU-0011-R10A-SM-2XU9G', 'K-562-SM-7LG4B', 'GTEX-UTHO-0003-SM-3G3SP', 'K-562-SM-2XCBH', 'GTEX-1GPI6-0004-SM-DLION', 'GTEX-P4QT-0526-SM-2D443', 'GTEX-1H23P-1826-SM-A8NA5', 'GTEX-VJWN-2926-SM-DMZ2G', 'GTEX-ZV7C-0002-SM-4YUX7', 'GTEX-SIU8-0626-SM-2XV5R', 'GTEX-1KD5A-0001-SM-DJWXF', 'GTEX-S4P3-0004-SM-4X5ZQ', 'GTEX-1GZ4I-0002-SM-DLIP8', 'GTEX-1RDX4-2826-SM-E6CRA', 'GTEX-S7SF-1626-SM-3K2AY', 'GTEX-12126-0001-SM-6WBU6', 'GTEX-12WSA-0001-SM-7DRQ8', 'GTEX-1LC46-0001-SM-DE1K7', 'GTEX-1117F-0003-SM-58Q7G', 'GTEX-1KAFJ-0003-SM-DJWYT', 'GTEX-1GZ4I-0006-SM-ARL6Z', 'GTEX-15TU5-2326-SM-7KUDT', 'GTEX-PSDG-0004-SM-2XK1T', 'GTEX-WL46-0002-SM-3PZ7T', 'GTEX-O5YW-0003-SM-5JK2Y', 'GTEX-PSDG-0526-SM-2QU4G', 'GTEX-RU72-0011-R2A-SM-2TWDC', 'GTEX-RU72-0011-R11A-SM-2TF6J', 'GTEX-16BQI-0003-SM-6UH2O', 'GTEX-N7MT-0011-R11A-SM-2IZHP', 'GTEX-R55D-1026-SM-2TWC3', 'GTEX-N7MT-0011-R8a-SM-2AXV2', 'GTEX-15DCZ-0626-SM-6AJ9I', 'GTEX-QDT8-0011-R9A-SM-32PKH', 'GTEX-1H1ZS-0006-SM-A8N7P', 'GTEX-1399S-2226-SM-DLIPM', 'GTEX-ZF29-0004-SM-6UH18', 'GTEX-15EU6-5007-SM-AN43F', 'GTEX-P78B-0004-SM-4X5ZL', 'GTEX-XK95-0726-SM-DMRPT', 'K-562-SM-4V81D', 'GTEX-RNOR-0011-R3A-SM-2TWD1', 'GTEX-13CZU-0003-SM-6WSC3', 'GTEX-OXRP-0326-SM-5SI7J', 'K-562-SM-5PNWU', 'GTEX-PVOW-1026-SM-5NQ7M', 'GTEX-NFK9-0004-SM-3UZFC', 'GTEX-15RJ7-0001-SM-6UH2H', 'GTEX-1211K-0003-SM-7DRPU', 'GTEX-13SLX-0003-SM-5DU5R', 'GTEX-ZYWO-0004-SM-58Q9F', 'GTEX-11ZTT-5001-SM-AN3YD', 'GTEX-18A67-0003-SM-DINAR', 'GTEX-S33H-2226-SM-5S2UF', 'GTEX-POYW-0526-SM-5KM2H', 'GTEX-15CHR-0002-SM-7DLH5', 'GTEX-15RIF-1926-SM-7KUDW', 'GTEX-1LVAM-0004-SM-DJWWR', 'GTEX-NPJ8-0011-R6a-SM-2AXVV', 'GTEX-1GF9W-0426-SM-9MQKK', 'GTEX-QCQG-0003-SM-2IJGO', 'GTEX-1HFI6-0001-SM-AG5Y7', 'GTEX-15FZZ-1026-SM-DK2JM', 'GTEX-14E6D-0002-SM-5CV9L', 'GTEX-15SDE-2426-SM-72D6L', 'GTEX-S7SF-2026-SM-2XU9J', 'GTEX-PVOW-0011-R1A-SM-2XV5W', 'GTEX-XUJ4-0001-SM-53IUY', 'GTEX-RU72-0011-R5A-SM-2TWCN', 'GTEX-PW2O-1826-SM-2QU3A', 'GTEX-11ILO-2026-SM-5N9CQ', 'GTEX-1IDJF-0002-SM-DC69O', 'GTEX-15CHS-1226-SM-5ZZVR', 'GTEX-13NYC-0326-SM-5K7WP', 'GTEX-1117F-0011-R10b-SM-CYKQ8', 'GTEX-ZVZQ-0002-SM-6WBSK', 'GTEX-1LVA9-3026-SM-E9U5M', 'K-562-SM-793B6', 'GTEX-X638-0004-SM-4YUV8_1', 'GTEX-13FH7-0004-SM-5HJPL', 'GTEX-RU1J-1926-SM-2TWBP', 'GTEX-QLQ7-0005-SM-2M486', 'GTEX-YB5K-0001-SM-53IUP', 'GTEX-VUSH-0002-SM-4X5ZZ', 'GTEX-1MJK3-0003-SM-DE1IP', 'GTEX-R53T-0626-SM-2TO5Y', 'GTEX-14753-0011-R10a-SM-AHZ7C', 'GTEX-13NZ8-0004-SM-5HJPC', 'GTEX-QEL4-0826-SM-325AI', 'GTEX-R55D-0926-SM-CLTGD', 'GTEX-S32W-0001-SM-2VCTS', 'GTEX-145MI-0011-R3b-SM-5PNWK', 'GTEX-N7MS-0011-R10A-SM-2IZJW', 'GTEX-Q2AH-0526-SM-2IZI3', 'GTEX-RNOR-0526-SM-2TWB9', 'GTEX-146FH-1826-SM-5QGQ7', 'GTEX-183FY-0003-SM-6RBJZ', 'GTEX-QEG4-0003-SM-3USS4', 'GTEX-13O61-0003-SM-6WSAT', 'GTEX-PLZ4-0226-SM-2S1NW', 'GTEX-N7MT-0011-R3a-SM-2AXVY', 'GTEX-YJ8O-1026-SM-DMRPU', 'GTEX-ZV7C-0926-SM-59HL1', 'GTEX-R3RS-0001-SM-5URBE', 'GTEX-1QP6S-0326-SM-DTXF7', 'GTEX-S32W-0326-SM-2XUAN', 'GTEX-1A8G7-0126-SM-7DUGH', 'GTEX-12WSK-0003-SM-5ANG3', 'GTEX-S4Z8-0003-SM-2XK2M', 'GTEX-11TUW-0011-R11b-SM-DNZZX', 'GTEX-1ICG6-0004-SM-DCCZF', 'GTEX-147JS-0003-SM-7DRO2', 'GTEX-11ZUS-0011-R10a-SM-AHZMK', 'GTEX-T6MO-0001-SM-325AL', 'GTEX-1B98T-0004-SM-7J38T', 'GTEX-14PQA-0003-SM-7DLH4', 'GTEX-Y5V5-1026-SM-5LUAH', 'GTEX-1HCU6-1926-SM-A8NA6', 'GTEX-QMRM-0002-SM-2XK25', 'GTEX-139UC-0011-R9a-SM-5K7W7', 'GTEX-ZDYS-0005-SM-53UBJ', 'GTEX-13JUV-0526-SM-CYKUW', 'GTEX-139TS-0426-SM-CYKPU', 'GTEX-S4Z8-1826-SM-3K2BH', 'GTEX-15DCE-5004-SM-AN459', 'GTEX-12WSG-5001-SM-AN3X9', 'GTEX-14ICK-0004-SM-5DU5N', 'GTEX-1RQED-0001-SM-DE1NW', 'GTEX-13PL7-5016-SM-AN3ZS', 'GTEX-14PJM-0003-SM-5GYE8', 'GTEX-16NFA-5018-SM-AN44N', 'GTEX-1LH75-0626-SM-DKTRU', 'GTEX-11WQK-0726-SM-CYKV6', 'GTEX-RWS6-2026-SM-2XU8X', 'GTEX-1F52S-0011-R1a-SM-9OSX9', 'GTEX-PWN1-0004-SM-2XK15', 'GTEX-NL3G-0008-SM-4E3JX', 'GTEX-OXRK-0926-SM-2D44V', 'GTEX-131XE-0004-SM-7DRP4', 'GTEX-ZVP2-0002-SM-7FG6C', 'GTEX-Q2AI-1526-SM-2QU49', 'GTEX-15UF6-0001-SM-6UH2E', 'GTEX-12WSB-0011-R1b-SM-5P9EY', 'GTEX-ZZPT-0002-SM-6WBUG', 'GTEX-1CB4J-0004-SM-7J38M', 'GTEX-1IDFM-0003-SM-DC69R', 'GTEX-13PL6-0002-SM-6WSCU', 'GTEX-SE5C-0003-SM-325HT', 'GTEX-XXEK-0001-SM-4YUWD', 'GTEX-1J1R8-0726-SM-A8N92', 'GTEX-1HCU6-2026-SM-A8NA7', 'GTEX-12WSN-0004-SM-5ANGE', 'GTEX-18D9A-0126-SM-DLL4R', 'GTEX-12WSI-0004-SM-5DWRW', 'GTEX-NL4W-0009-SM-2BWZB', 'GTEX-QESD-0006-SM-2M499', 'K-562-SM-2D454', 'GTEX-1QL29-0002-SM-DE1JQ', 'GTEX-131XF-0926-SM-5BC69', 'GTEX-RTLS-0004-SM-7DLGV', 'GTEX-15EU6-5004-SM-AN43H', 'GTEX-1H3VY-0004-SM-AG5YM', 'GTEX-13NZ8-0004-SM-6WSBJ', 'GTEX-117XS-0001-SM-58Q7J', 'GTEX-TMMY-0326-SM-33HBF', 'GTEX-ZYFG-0004-SM-58Q7C', 'GTEX-17GQL-0926-SM-731E7', 'K-562-SM-9HWNW', 'K-562-SM-9HWNV', 'K-562-SM-9HWNU', 'K-562-SM-9HWNT', 'K-562-SM-9HWNQ', 'K-562-SM-9HWNP', 'GTEX-13O3P-0003-SM-6WSAJ', 'GTEX-11EQ8-2826-SM-5N9C1', 'GTEX-YEC4-0006-SM-6UH3M', 'GTEX-N7MS-0011-R8a-SM-2AXVD', 'GTEX-PLZ4-0006-SM-5SI8L', 'GTEX-1JN6P-2526-SM-E9IK1', 'K-562-SM-2D45Y', 'GTEX-1GF9V-0004-SM-7MA6Q', 'GTEX-QLQ7-0003-SM-3USS2', 'GTEX-1RNSC-0002-SM-DLIQK', 'GTEX-1QL29-0002-SM-DLIPA', 'GTEX-Q734-1426-SM-48TZQ', 'GTEX-T2YK-0003-SM-5URCD', 'GTEX-1B932-5013-SM-AN45S', 'GTEX-13PLJ-0003-SM-6WSCN', 'GTEX-SIU7-1726-SM-2XV4K', 'GTEX-UJMC-0002-SM-3G3ST', 'GTEX-15CHC-5004-SM-AN437', 'GTEX-OHPJ-0008-SM-4E3JG', 'GTEX-PX3G-1626-SM-2QU4K', 'GTEX-QESD-0001-SM-3USS1', 'GTEX-1GMR2-0003-SM-7J3B2', 'GTEX-1B98T-0626-SM-731FN', 'GTEX-N7MS-0626-SM-2QU2Q', 'GTEX-WYVS-0002-SM-3PZ7R', 'GTEX-1H3O1-0003-SM-AG5YL', 'GTEX-S7SE-0011-R2A-SM-2XV69', 'GTEX-QDT8-0926-SM-7DRQ9', 'GTEX-P44H-0011-R3A-SM-2XCEQ', 'GTEX-Q734-0006-SM-2IZIL', 'GTEX-S4P3-0226-SM-3K2BD', 'GTEX-QDVJ-0003-SM-2XK1Y', 'GTEX-11UD1-0001-SM-5DWT6', 'GTEX-XOTO-0726-SM-4B659', 'GTEX-XOTO-3026-SM-4B65M', 'GTEX-XUZC-0002-SM-4YUX1', 'GTEX-13N2G-0826-SM-CYKVY', 'GTEX-QEG5-0006-SM-2IZJE', 'GTEX-YB5K-0002-SM-AYRPK', 'GTEX-1I6K6-1126-SM-A8N8O', 'GTEX-Y8DK-0011-R10a-SM-AHZ7M', 'GTEX-PLZ4-0126-SM-2QU4D', 'GTEX-1JMLX-0826-SM-DLT2I', 'GTEX-X4XY-0002-SM-5URBG', 'GTEX-11ZU8-0011-R3a-SM-5BC6U', 'GTEX-WHSE-0926-SM-CYKVI-ELUATE', 'GTEX-R55E-0011-R8A-SM-2TO6E', 'GTEX-15DDE-0001-SM-6RBJO', 'GTEX-RU72-1126-SM-2TWAV', 'K-562-SM-5IFEL', 'GTEX-145MI-0011-R5a-SM-5Q5B4', 'K-562-SM-4RH3A', 'K-562-SM-5PNU4', 'GTEX-1PPH7-0004-SM-DE1J8', 'GTEX-QMR6-1926-SM-4SVPX', 'GTEX-QDT8-0011-R9A-SM-5SI8J', 'GTEX-P4PP-0326-SM-2D43Z', 'GTEX-RN64-0004-SM-5URBC', 'GTEX-OXRN-0005-SM-2D43O', 'GTEX-XUYS-0001-SM-5JK2T', 'GTEX-Q2AI-1626-SM-2S1P9', 'GTEX-1QP6S-0001-SM-DLIQM', 'GTEX-WOFM-0003-SM-4X61U', 'GTEX-UPIC-0626-SM-4IHK2', 'GTEX-ZYVF-0004-SM-6WBSC', 'GTEX-PWN1-0004-SM-5JK3S', 'GTEX-QXCU-0826-SM-2TO5Q', 'GTEX-QDVJ-0005-SM-2QU5C', 'GTEX-WHSB-0626-SM-4M1XP', 'GTEX-TML8-0004-SM-5SOEP', 'GTEX-OIZG-1526-SM-2QU3R', 'GTEX-XXEK-1726-SM-5S2SR', 'GTEX-S341-1726-SM-2XU8C', 'GTEX-144GO-0126-SM-EZ6XU', 'GTEX-P4QR-0326-SM-2M47P', 'GTEX-SJXC-0003-SM-4SVPY', 'GTEX-139UC-0011-R4a-SM-5K7XM', 'GTEX-15SHV-5004-SM-AN444', 'GTEX-WHSB-0004-SM-3USQY', 'GTEX-WL46-0011-R2A-SM-3LK6O', 'GTEX-14ICL-0004-SM-5CV9H', 'GTEX-VUSG-0002-SM-4X61J', 'GTEX-1F5PL-0006-SM-A8N7M', 'GTEX-OXRP-0126-SM-48TDI', 'GTEX-XOTO-0011-R4B-SM-EZ6LL', 'GTEX-P44G-0226-SM-2D44P', 'GTEX-PWO3-0011-R6A-SM-2IZJ2', 'GTEX-1H3VE-1326-SM-9WYTB', 'GTEX-ZLWG-0001-SM-4YUVG', 'GTEX-12WSA-0001-SM-5DWRG', 'GTEX-12WS9-0826-SM-59HJX', 'GTEX-S4Z8-0006-SM-2XU8E', 'GTEX-WY7C-0002-SM-3PZ81', 'GTEX-S4Q7-0426-SM-2XUAZ', 'GTEX-1QW4Y-0003-SM-DCXR5', 'GTEX-1ICLY-0003-SM-DCCZE', 'GTEX-13G51-0011-R10b-SM-CYKS7', 'GTEX-POMQ-0003-SM-2XK11', 'GTEX-SNOS-0004-SM-5SOEL', 'GTEX-XOTO-0011-R9A-SM-4GICI', 'GTEX-O5YV-0004-SM-2BWZ5', 'GTEX-X5EB-0001-SM-3USRB', 'GTEX-RVPU-0011-R4A-SM-5SI8K', 'GTEX-1JJEA-0003-SM-DE1IJ', 'GTEX-R55C-0326-SM-2TO5H', 'GTEX-QLQ7-1826-SM-2QU3E', 'GTEX-Y9LG-0003-SM-4YUVB', 'GTEX-SIU7-1326-SM-2XK3D', 'GTEX-1LC46-0001-SM-DJWX9', 'GTEX-1192W-0001-SM-58Q7T', 'GTEX-1QP28-0004-SM-DLIQJ', 'GTEX-ZVP2-0002-SM-53UAJ', 'GTEX-XUYS-0001-SM-DLIPV', 'GTEX-X4XY-0002-SM-3UZEV', 'GTEX-Y8DK-0002-SM-53IV6', 'GTEX-1QP2A-0626-SM-DKTRW', 'GTEX-YECK-0002-SM-53IUQ', 'GTEX-13RTL-0826-SM-DO3WZ', 'GTEX-15DYW-0011-R10a-SM-AHZNE', 'GTEX-1HUB1-0002-SM-AG5ZX', 'GTEX-ZVZQ-0011-R10b-SM-CYKRF', 'GTEX-117YW-0003-SM-5DWRD', 'GTEX-OIZI-1026-SM-2VCU2', 'GTEX-POYW-1226-SM-5JK3K', 'GTEX-17F9E-5007-SM-AN44S', 'GTEX-P78B-0626-SM-5S2W4', 'GTEX-1399T-2426-SM-CYKW2', 'GTEX-139TS-1526-SM-DK2JL', 'GTEX-OHPL-0126-SM-2AXUS', 'GTEX-ZYT6-0002-SM-5DWS9', 'GTEX-15DDE-3026-SM-6AJ9C', 'GTEX-WFJO-0003-SM-3UZER', 'GTEX-1AX8Z-5007-SM-AN45W', 'GTEX-145LT-0126-SM-E9WQX', 'GTEX-1I6K7-0003-SM-DC69I', 'GTEX-S33H-0005-SM-2XU96', 'GTEX-1F75I-2326-SM-A8N7V', 'GTEX-P4PP-0004-SM-5JK2S', 'GTEX-ZYWO-1026-SM-5E45U', 'GTEX-11WQK-2326-SM-5GU4Z', 'GTEX-1A3MW-0004-SM-7MD4N', 'GTEX-139T4-0426-SM-DLIOF', 'GTEX-SIU7-0002-SM-5JK4G', 'GTEX-1C64O-5004-SM-AN46A', 'GTEX-N7MT-0126-SM-26GMS', 'GTEX-11WQC-0003-SM-58Q8N', 'GTEX-1JMQK-0004-SM-DE1NE', 'GTEX-N7MT-0126-SM-26GMB', 'GTEX-13G51-2726-SM-CYKUZ', 'GTEX-P44H-0011-R3A-SM-5SI8A', 'GTEX-1K9T9-0004-SM-DJWYM', 'GTEX-OIZF-0626-SM-2AXUP', 'GTEX-12WSJ-0003-SM-6WSBN', 'GTEX-OHPL-0726-SM-3TW8U', 'GTEX-Y5V5-0002-SM-6UH3T', 'GTEX-1HBPM-2726-SM-DK2K1', 'GTEX-133LE-0001-SM-5ANGK', 'GTEX-1KXAM-0004-SM-DE1K5', 'GTEX-1IKK5-2426-SM-CE6SV', 'GTEX-QMRM-0005-SM-2M48F', 'GTEX-QLQW-1526-SM-2TO4Z', 'K-562-SM-6AJ95', 'GTEX-QMRM-0826-SM-2M49A', 'GTEX-OIZG-1126-SM-2AXUQ', 'GTEX-SIU7-0526-SM-3NM8I', 'GTEX-13FH7-5019-SM-AN3Z3', 'GTEX-QCQG-1926-SM-2QU3U', 'GTEX-15TU5-0226-SM-6PALR', 'GTEX-OXRL-1826-SM-2M47K', 'GTEX-OOBK-3025-SM-48TBP', 'GTEX-ZTX8-0004-SM-6WBSE', 'GTEX-OHPL-0326-SM-2HMKS', 'GTEX-ZDTS-0002-SM-53UAQ', 'GTEX-U8XE-0004-SM-3G3TV', 'GTEX-RUSQ-1026-SM-2TWD9', 'GTEX-UPJH-0002-SM-3G3T3', 'GTEX-PVOW-1026-SM-325A8', 'GTEX-OHPN-0011-R1A-SM-2M49D', 'GTEX-Y5V6-0001-SM-4YUXD', 'GTEX-117XS-0001-SM-5DWSE', 'GTEX-1A32A-0001-SM-7MD4W', 'GTEX-TMZS-0002-SM-5URDR', 'GTEX-P4QT-0226-SM-2D446', 'GTEX-1LGOU-0003-SM-DJWYI', 'GTEX-ZTSS-0003-SM-4YUW6', 'GTEX-T5JC-0426-SM-5S2SL', 'GTEX-X4XX-0004-SM-3USR8', 'GTEX-1AYD5-0003-SM-7J39P', 'GTEX-14E6D-1026-SM-CYKRZ', 'GTEX-PVOW-0011-R3A-SM-2XV66', 'GTEX-11DZ1-0002-SM-CLTG7', 'GTEX-11WQK-0003-SM-5DWTN', 'GTEX-13S7M-0003-SM-6WSAH', 'GTEX-14BIL-0326-SM-CYKR6', 'GTEX-X5EB-0826-SM-46MVS', 'GTEX-13N2G-0011-R10a-SM-4RGKC-hCf', 'K-562-SM-4AD4F', 'GTEX-13N2G-0011-R10a-SM-4RGKC-hCe', 'GTEX-U412-0326-SM-5DUVP', 'GTEX-1B996-0011-R11b-SM-7MXTM', 'GTEX-TMMY-0003-SM-3G3SV', 'GTEX-1KANA-0003-SM-DJWY3', 'K-562-SM-4BRWK', 'GTEX-13OW7-0001-SM-5HJRV', 'GTEX-WL46-1026-SM-CYKVS', 'GTEX-1F6IF-0004-SM-7J3AJ', 'GTEX-1AX8Z-5004-SM-AN45U', 'GTEX-13JVG-0003-SM-6WSBH', 'GTEX-147F4-0011-R10a-SM-AHZ7E', 'GTEX-N7MT-0011-R5a-SM-2AXVM', 'GTEX-PW2O-1526-SM-2I5FT', 'GTEX-14LZ3-0011-R8b-SM-69LP6', 'GTEX-P44H-1126-SM-4SVQD', 'GTEX-QDT8-1426-SM-EXUR4', 'GTEX-S33H-0526-SM-2XUAH', 'GTEX-QEG4-0326-SM-5S2UQ', 'GTEX-1S82P-0002-SM-DE1IY', 'GTEX-OHPJ-0003-SM-2XK1G', 'K-562-SM-2XCFL', 'GTEX-1QAET-1426-SM-DLT2N', 'GTEX-12WS9-0006-SM-5O99W', 'K-562-SM-6ETZI', 'GTEX-PW2O-1726-SM-2QU4Y', 'GTEX-RVPU-0011-R8A-SM-2TF74', 'GTEX-T6MN-0003-SM-325AF', 'GTEX-15EOM-5007-SM-AN43E', 'GTEX-14E6C-0011-R10a-SM-AHZMW', 'GTEX-N7MS-0011-R7a-SM-2AXV5', 'GTEX-WFON-2026-SM-5S2TE', 'GTEX-WRHK-0003-SM-4X61X', 'GTEX-NL4W-0011-R10A-SM-2IZHO', 'GTEX-X638-0004-SM-DLIPT', 'GTEX-14PK6-0003-SM-7DRPF', 'GTEX-1NV8Z-0003-SM-DE1IN', 'GTEX-ZPU1-0002-SM-4YUW5', 'GTEX-15SDE-5019-SM-AN445', 'GTEX-S7PM-0926-SM-3PYK3', 'GTEX-Q2AG-0005-SM-33HBX', 'GTEX-P44G-2326-SM-2XV5B', 'GTEX-12WSG-0326-SM-5FQU3', 'GTEX-Q2AI-0003-SM-5JK3F', 'GTEX-13X6H-0001-SM-5DU55', 'GTEX-S3LF-0526-SM-EZ6LK', 'GTEX-13CIG-0004-SM-5DU44', 'GTEX-PWO3-0011-R2A-SM-2IZJO', 'K-562-SM-731AF', 'GTEX-QDT8-0926-SM-4SVPN', 'GTEX-14PKU-0004-SM-6RBLN', 'GTEX-17GQL-5007-SM-AN454', 'GTEX-NPJ8-0011-R9a-SM-2QU2O', 'GTEX-RTLS-0426-SM-2TWBI', 'GTEX-TSE9-0002-SM-3USSJ', 'GTEX-TMKS-0002-SM-DLIOM', 'GTEX-REY6-0004-SM-2XK3B', 'GTEX-R53T-0003-SM-3UZFQ', 'GTEX-15FZZ-5001-SM-AN44G', 'GTEX-1H11D-5010-SM-AN46Y', 'GTEX-12WSH-0011-R10a-SM-AHZ6J', 'GTEX-WFG7-0002-SM-3G3TJ', 'GTEX-Y114-2726-SM-4TT9B', 'GTEX-RVPU-0011-R4A-SM-2XU9Y', 'GTEX-ZF2S-0002-SM-4YUWS', 'GTEX-PWN1-2626-SM-2IZIJ', 'GTEX-1I1GQ-0002-SM-AG5YY', 'GTEX-13NZA-0011-R10b-SM-CYKRC', 'GTEX-1AX8Y-0726-SM-731CX', 'GTEX-14PJ3-0004-SM-5GYE1', 'GTEX-RVPU-0001-SM-2XK36', 'GTEX-1B8L1-0004-SM-7J3AV', 'GTEX-14PN4-0011-R10b-SM-CYKO4', 'GTEX-S4P3-1626-SM-3K2AZ', 'GTEX-1AYCT-0004-SM-7J39Y', 'GTEX-15RIF-0004-SM-6RBL2', 'GTEX-WK11-0002-SM-3PYKK', 'GTEX-WOFM-0003-SM-5JK3H', 'GTEX-PSDG-0226-SM-2I5FR', 'GTEX-ZYWO-2226-SM-5GZXR', 'GTEX-P4PQ-0326-SM-2D43Y', 'GTEX-YJ89-0002-SM-53IUO', 'GTEX-ZYWO-2026-SM-5GIDA', 'GTEX-U8T8-2026-SM-3L286', 'GTEX-15DDE-0011-R5b-SM-7KULJ', 'GTEX-YEC4-0006-SM-4YUWY', 'GTEX-ZXES-1226-SM-5NQ8G', 'GTEX-SNMC-0126-SM-2XV6G', 'GTEX-12C56-0003-SM-7DRPT', 'GTEX-12WS9-0011-R3a-SM-5CVNF', 'GTEX-1ICLY-2026-SM-A8N9D', 'GTEX-1RQEC-0004-SM-DE1NX', 'GTEX-ZYW4-0002-SM-58Q7H', 'GTEX-13NZ9-0004-SM-5HJQY', 'GTEX-12ZZZ-0011-R10a-SM-AHZ6M', 'GTEX-139UC-0004-SM-5HJQ3', 'GTEX-15DCD-0011-R10b-SM-CYKO7', 'GTEX-R55G-2326-SM-2TWC5', 'GTEX-15UF7-0003-SM-6RBLL', 'GTEX-1BAJH-0003-SM-7J39G', 'GTEX-PWCY-1826-SM-2QU4P', 'GTEX-ZZPT-0002-SM-5DWTU', 'GTEX-15ER7-0926-SM-CYKTS-SUP', 'GTEX-R55D-0006-SM-2TWBJ', 'GTEX-VJYA-0003-SM-3G3TA', 'GTEX-12696-0326-SM-DK2K4', 'GTEX-15ER7-0226-SM-CYKUA', 'GTEX-XOTO-0526-SM-4B662', 'GTEX-17EVP-0004-SM-6RBJT', 'GTEX-15SHU-0626-SM-CYKUC', 'GTEX-P4QR-0726-SM-2M49P', 'GTEX-13O61-0003-SM-5HJRK', 'GTEX-PWOO-1226-SM-48TCO', 'GTEX-1LH75-0626-SM-DLT2J', 'GTEX-ZAJG-0001-SM-6UH3R', 'GTEX-TKQ2-0002-SM-5URCO', 'GTEX-1JMOU-0004-SM-DJWY8', 'GTEX-15DCE-0003-SM-7DLFB', 'GTEX-ZVT3-1426-SM-59HLP', 'GTEX-15CHS-0426-SM-6LLJ8', 'GTEX-Y8LW-0004-SM-6UH3N', 'GTEX-PWOO-0004-SM-5JK4A', 'GTEX-12WSB-2426-SM-5EGJC', 'GTEX-RVPU-0011-R5A-SM-2XU9Z', 'GTEX-1IKK5-0003-SM-G8OX3', 'GTEX-1E1VI-0003-SM-7J38L', 'GTEX-OOBK-0004-SM-5JK4J', 'GTEX-14A5I-0004-SM-5DU5S', 'GTEX-P4QS-1626-SM-2M474', 'GTEX-11XUK-0003-SM-58Q8M', 'GTEX-13O3Q-0011-R1b-SM-4W9GN-hHP2a', 'GTEX-13O3Q-0011-R1b-SM-4W9GN-hHP2c', 'GTEX-13O3Q-0011-R1b-SM-4W9GN-hHP2b', 'GTEX-S4Z8-2026-SM-2XU8K', 'GTEX-14753-0526-SM-CYKNW', 'GTEX-13FLV-0004-SM-6WSBZ', 'GTEX-1N2EF-0002-SM-DE1NA', 'GTEX-XUW1-0002-SM-4YUXF', 'GTEX-1PWST-2526-SM-E76QW', 'GTEX-OIZI-0526-SM-2XV55', 'GTEX-RVPV-0003-SM-2XK35', 'GTEX-N7MT-0009-SM-2BWYA', 'GTEX-15TU5-2426-SM-7KUDU', 'GTEX-QEG5-0003-SM-2XK26', 'GTEX-WYBS-0126-SM-3PYKA', 'GTEX-Z9EW-0002-SM-4YUV9', 'GTEX-PSDG-0005-SM-2M481', 'GTEX-PWCY-2326-SM-2IZIF', 'GTEX-T6MN-0126-SM-5S2TD', 'GTEX-WL46-0926-SM-CYKVT', 'GTEX-NPJ8-0011-R1a-SM-2HMLC', 'GTEX-PLZ4-0126-SM-2S1OG', 'GTEX-PLZ6-0006-SM-2M48W', 'GTEX-15SHW-0004-SM-6UH2L', 'GTEX-VJWN-0003-SM-7DRQA', 'GTEX-13PVR-0003-SM-7DRNW', 'GTEX-Z93S-0002-SM-6UH3P', 'GTEX-P4QR-0004-SM-5JK2Q', 'GTEX-1J1OQ-0826-SM-CXZJG', 'GTEX-13N1W-0002-SM-5HJR2', 'GTEX-1ICG6-5019-SM-AN47J', 'GTEX-147F3-0002-SM-6WSBE', 'GTEX-12WSF-0011-R10a-SM-AHZNQ', 'GTEX-12WSI-0004-SM-6WSCD', 'GTEX-XXEK-0626-SM-5S2SZ', 'GTEX-13NZA-0003-SM-6WSBM', 'GTEX-R55G-0526-SM-2TO6S', 'GTEX-YEC3-0226-SM-5IFJO', 'GTEX-1RLM8-2626-SM-E8VOM', 'GTEX-TMKS-0226-SM-3DB7X', 'GTEX-R55E-0826-SM-2TO6B', 'GTEX-15EO6-0011-R10a-SM-CYKSK', 'GTEX-1GPI6-0004-SM-DINCR', 'GTEX-ZZPT-0002-SM-58Q9E', 'GTEX-12WSF-3126-SM-CYKSU', 'GTEX-12WSB-0626-SM-59HJO', 'K-562-SM-2AXTU', 'GTEX-S341-0226-SM-2XCAX', 'GTEX-PVOW-0726-SM-2XV5G', 'GTEX-18464-0004-SM-DINAW', 'GTEX-15TU5-0011-R8b-SM-793BM', 'GTEX-QEG4-0426-SM-2I5GK', 'GTEX-1HBPM-0011-R5b-SM-CJI3I', 'GTEX-QESD-1726-SM-2QU3X', 'GTEX-S4UY-0126-SM-2XU9R', 'GTEX-REY6-0726-SM-2TO4O', 'GTEX-SJXC-0126-SM-2XV4I', 'GTEX-ZTTD-0002-SM-6WBSH', 'GTEX-18A6Q-0003-SM-DJWXQ', 'GTEX-ZAB5-0004-SM-53UBI', 'GTEX-117YX-0001-SM-7DRQ6', 'GTEX-145LT-0226-SM-EZ6Y2', 'GTEX-1POEN-0002-SM-DE1JF', 'GTEX-NPJ8-0011-R4a-SM-2AXW8', 'GTEX-19HZE-0426-SM-731C8', 'GTEX-ZVT4-0126-SM-51MS1', 'GTEX-13OVI-0003-SM-5GY9X', 'GTEX-WHSE-0011-R5A-SM-3P5ZO', 'GTEX-PSDG-0226-SM-33HC1', 'GTEX-15DCZ-0526-SM-686Z3', 'GTEX-13FTX-0004-SM-5HJQL', 'GTEX-X4EO-0003-SM-3PZ83', 'GTEX-12WSE-0002-SM-5ANG8', 'GTEX-P4QT-2626-SM-2IZIN', 'GTEX-13OW8-0002-SM-5CV8J', 'GTEX-13QBU-5001-SM-AN419', 'GTEX-PLZ6-0003-SM-2XK1U', 'GTEX-13W46-0003-SM-5HJPN', 'GTEX-PX3G-0006-SM-2IZHT', 'GTEX-169BO-5013-SM-AN44D', 'GTEX-15RJE-2326-SM-6LPIZ', 'GTEX-13NZB-0004-SM-5HJRG', 'GTEX-16XZZ-0011-R2a-SM-7IGPG', 'GTEX-13FH7-0004-SM-6WSB3', 'GTEX-OHPM-0526-SM-2AXUY', 'GTEX-NPJ8-1726-SM-2QU3J', 'GTEX-X3Y1-0002-SM-5URCV', 'GTEX-Q2AG-0226-SM-2QU2T', 'GTEX-RUSQ-0001-SM-2XK38', 'GTEX-12WSN-5022-SM-7DHN6', 'GTEX-12WSG-2526-SM-5EQ4O', 'GTEX-12WSJ-0003-SM-5DWRN', 'GTEX-15SZO-0004-SM-6RBLK', 'GTEX-S4Q7-0001-SM-2VCU1', 'GTEX-139TS-0003-SM-5HJR1', 'GTEX-1HGF4-2326-SM-A8NA2', 'GTEX-15TU5-1226-SM-6PAMI', 'GTEX-R55F-0011-R5A-SM-2TF5L', 'GTEX-145ME-5010-SM-793AR', 'GTEX-OXRP-0226-SM-48TDH', 'GTEX-V955-0001-SM-3G3TY', 'GTEX-XMD1-0011-R10a-SM-AHZNY', 'GTEX-SNMC-0426-SM-DMRPY', 'GTEX-QDVJ-0003-SM-2MRI3', 'GTEX-Q2AG-0011-R1A-SM-2IZJV', 'GTEX-R55E-0003-SM-5URBX', 'GTEX-13NZA-0011-R10a-SM-AHZ6W', 'GTEX-W5X1-0002-SM-3USSL', 'GTEX-1CB4I-0002-SM-DLIOJ', 'GTEX-11WQK-0011-R10a-SM-AHZ6L', 'GTEX-S33H-2226-SM-2XCB7', 'GTEX-14C39-0011-R6b-SM-5YYAZ', 'GTEX-QVJO-0011-R7A-SM-2TO6W', 'GTEX-Y8DK-0326-SM-CYKR1', 'GTEX-ZDTS-0002-SM-4YUVC', 'GTEX-Q2AH-0005-SM-2IZIK', 'GTEX-NPJ8-0326-SM-26GMG', 'GTEX-NPJ8-0326-SM-26GMX', 'GTEX-QVUS-2826-SM-2TWCH', 'GTEX-XPT6-0004-SM-53IVP', 'GTEX-TKQ2-0002-SM-325HS', 'GTEX-1LSVX-0003-SM-DE1K4', 'GTEX-147GR-0003-SM-6WSBC', 'GTEX-QDT8-0006-SM-32PL3', 'GTEX-18D9U-0001-SM-DINAT', 'GTEX-Z93S-0002-SM-53IUJ', 'GTEX-T2YK-0226-SM-4DM5O', 'GTEX-13NYS-0326-SM-CYKVV', 'GTEX-QXCU-0004-SM-2XK33', 'GTEX-OIZH-0326-SM-2AXUF', 'GTEX-12WSJ-0003-SM-5ANG5', 'GTEX-18D9B-1026-SM-72D6A', 'GTEX-NPJ8-0011-R8a-SM-2AXVK', 'GTEX-Q734-0626-SM-2IZI5', 'GTEX-PW2O-0006-SM-2IZHL', 'GTEX-1LSNL-0004-SM-DE1KG', 'GTEX-PWOO-0004-SM-2IJGC', 'GTEX-14B4R-0004-SM-7DRQ2', 'GTEX-Q2AG-0126-SM-2IZJX', 'GTEX-TMZS-0002-SM-3UZF4', 'GTEX-1399S-2226-SM-DKTR7', 'GTEX-XXEK-0005-SM-5S2TG', 'GTEX-S4Z8-0006-SM-3K2AQ', 'GTEX-PWOO-2226-SM-2QU2S', 'GTEX-SIU7-1326-SM-2VCU7', 'GTEX-1399T-3026-SM-CYKW1', 'GTEX-S341-0226-SM-2XU9O', 'GTEX-1C2JI-5001-SM-AN45Q', 'GTEX-PSDG-0004-SM-5JK3L', 'GTEX-QVJO-0011-R2A-SM-2S1QK', 'GTEX-RUSQ-0001-SM-2RXER', 'GTEX-NFK9-0726-SM-2AXU7', 'GTEX-X4LF-0001-SM-7DRQ5', 'GTEX-13JUV-0001-SM-5DU62', 'GTEX-1JJ6O-0001-SM-DLIOO', 'GTEX-11DXZ-0003-SM-58Q7X', 'GTEX-P4PP-0326-SM-5S2TQ', 'GTEX-V1D1-0001-SM-5SOE2', 'GTEX-ZPU1-0002-SM-53UAZ', 'GTEX-QV44-0003-SM-5SOEM', 'GTEX-1I1GR-2726-SM-A8N8L', 'GTEX-N7MT-0011-R1a-SM-2AXWA', 'GTEX-QDVN-0726-SM-2I3FO', 'GTEX-11ZUS-0003-SM-5DWTA', 'GTEX-XAJ8-0008-SM-46MU7', 'GTEX-QMR6-1926-SM-2XK2D', 'GTEX-NL4W-0011-R4a-SM-2AXW6', 'GTEX-QVJO-1426-SM-2TO5V', 'GTEX-ZYT6-0002-SM-58Q7D', 'GTEX-139UC-0011-R8b-SM-5IJCL', 'GTEX-1GMR3-0001-SM-7MA6Y', 'GTEX-P4QT-0003-SM-2H2X3', 'GTEX-1JMQK-0004-SM-DJWWF', 'GTEX-11P81-0002-SM-6WBTV', 'GTEX-QLQ7-1126-SM-4R1KN', 'GTEX-QDT8-0926-SM-CLTFX', 'GTEX-139TS-1526-SM-DLIO9', 'GTEX-1OZHM-0003-SM-DCXSA', 'GTEX-12696-2926-SM-5FQTG', 'GTEX-1128S-0001-SM-6WBUH', 'GTEX-14E6C-0002-SM-5CV9B', 'GTEX-NPJ8-0004-SM-2BWYM', 'GTEX-13NZB-0004-SM-5DU5H', 'GTEX-SN8G-0002-SM-325AH', 'GTEX-P44G-0011-R7A-SM-2D45K', 'GTEX-OXRK-0326-SM-2YUMQ', 'GTEX-P4QT-1726-SM-2M47C', 'GTEX-ZPCL-0001-SM-6WBS6', 'GTEX-WYVS-2026-SM-4PQZ4', 'GTEX-P44H-0526-SM-2XV4U', 'GTEX-1211K-0003-SM-58Q98', 'GTEX-15G19-0726-SM-CYKSQ', 'GTEX-1I6K7-0003-SM-DCCZI', 'GTEX-QDT8-0226-SM-2XV5J', 'GTEX-R55E-1026-SM-2TO6T', 'GTEX-ZYFG-0004-SM-5DWS8', 'GTEX-14E7W-0011-R10a-SM-CYKPN', 'GTEX-1B8KZ-0003-SM-7J39Z', 'GTEX-13FHO-1026-SM-CYKRK', 'K-562-SM-5MR32', 'GTEX-OIZI-0626-SM-2XV6K', 'GTEX-13SLW-0003-SM-5DU4Y', 'GTEX-P4QS-1826-SM-2M475', 'GTEX-14PHW-0002-SM-5GYEU', 'GTEX-NL3H-0006-SM-2AXUR', 'GTEX-QEL4-1926-SM-4R1JI', 'GTEX-148VI-5004-SM-AN41P', 'GTEX-147GR-0011-R10a-SM-AHZ6Y', 'K-562-SM-5LU2Y', 'GTEX-13OW6-0002-SM-6WSAM', 'GTEX-QVUS-0011-R8A-SM-2TWCG', 'GTEX-1IKOE-0001-SM-G8OXL', 'GTEX-U8T8-1226-SM-4E3IH', 'GTEX-ZQUD-0002-SM-4YUWT', 'GTEX-12ZZX-0001-SM-6WSBV', 'GTEX-UJHI-0001-SM-3G3TR', 'GTEX-NL3G-0526-SM-4RTWW', 'GTEX-QDVN-2026-SM-2QU4I', 'GTEX-ZQUD-0002-SM-53UBT', 'GTEX-RVPV-1726-SM-5SI8C', 'GTEX-15UKP-1526-SM-6PAL9', 'GTEX-NL3H-0011-R8a-SM-5S2VF', 'GTEX-139T6-5016-SM-AN3YQ', 'GTEX-P4QT-0003-SM-2XJZX', 'GTEX-1IDJI-2526-SM-ACKWY', 'K-562-SM-7EWCW', 'GTEX-QV31-0005-SM-2S1Q6', 'GTEX-XLM4-1526-SM-5S2S1', 'GTEX-15TU5-0011-R1a-SM-7938S', 'GTEX-XOTO-2126-SM-4B64U', 'GTEX-1F52S-0004-SM-7J3AY', 'GTEX-11GSP-0004-SM-5DWSV', 'GTEX-NPJ7-2826-SM-2AXTW', 'GTEX-14DAQ-0926-SM-CYKRD', 'GTEX-11TUW-0004-SM-5DWT8', 'GTEX-PWO3-0011-R8A-SM-2M49F', 'GTEX-1IKOE-5007-SM-AN48A', 'GTEX-UJMC-0002-SM-5SOEI', 'GTEX-15CHQ-0011-R8b-SM-6AJAI', 'GTEX-RN64-0826-SM-2TO5W', 'GTEX-15CHS-0011-R1a-SM-68712', 'GTEX-13O1R-0002-SM-5DU3N', 'GTEX-X4EO-0003-SM-5URCE', 'GTEX-S7SE-0926-SM-CYKVJ', 'GTEX-131XG-5016-SM-AN3XZ', 'K-562-SM-2AXVE', 'GTEX-XV7Q-0005-SM-4BRWI', 'GTEX-RU72-3026-SM-2TWCD', 'GTEX-1HBPH-2026-SM-CMKGL', 'GTEX-RU72-0011-R6A-SM-2TWCU', 'GTEX-14PJM-0011-R10a-SM-CYKSM', 'GTEX-QDT8-0526-SM-32PL5', 'GTEX-16AAH-0003-SM-6RBLI', 'GTEX-13VXU-0011-R10a-SM-CYKSP', 'GTEX-OHPN-0011-R10A-SM-5S2U3', 'GTEX-O5YV-0526-SM-2M49G', 'GTEX-ZWKS-0002-SM-6WBUM', 'K-562-SM-73KTR', 'GTEX-13FTW-5001-SM-AN3ZN', 'GTEX-13OW5-0003-SM-6WSCR', 'GTEX-Q734-1826-SM-2IZIA', 'GTEX-RVPU-2326-SM-2TF6R', 'GTEX-WFG7-0002-SM-5SOEQ', 'GTEX-13PDP-1026-SM-DLIOB', 'GTEX-WEY5-0003-SM-3UZET', 'K-562-SM-7LT88', 'GTEX-14PJ6-5019-SM-AN42V', 'GTEX-13N1W-0002-SM-5DU4S', 'GTEX-13NYS-1626-SM-CYKVW', 'GTEX-12WSD-0002-SM-6WSCG', 'GTEX-OXRN-0011-R8A-SM-2D45Q', 'GTEX-WY7C-0002-SM-7FG6L', 'GTEX-1ICLZ-0003-SM-DCCZP', 'GTEX-PW2O-0004-SM-2IJGJ', 'GTEX-1KD4Q-0002-SM-DE1NB', 'GTEX-117XS-2526-SM-CYKR4', 'GTEX-18D9A-0126-SM-DMZ2W', 'GTEX-ZZPU-0003-SM-5DWTO', 'GTEX-X5EB-0001-SM-3PYKT', 'GTEX-16MT8-0002-SM-6UH2U', 'GTEX-1LG7Z-0002-SM-DJWY1', 'GTEX-N7MS-2425-SM-26GMD', 'GTEX-N7MS-0011-R5a-SM-2AXW7', 'GTEX-1A3MW-3026-SM-72D5E', 'GTEX-ZDTS-0526-SM-6AJB8', 'GTEX-PWOO-2326-SM-2S1PQ', 'GTEX-1QP66-0002-SM-DLIP7', 'GTEX-WYJK-0001-SM-5URBR', 'GTEX-11ZTT-0003-SM-58Q8I', 'GTEX-RTLS-0004-SM-3USSE', 'GTEX-11I78-0002-SM-5DWT9', 'GTEX-RWSA-1126-SM-2XU92', 'GTEX-139YR-0004-SM-5HJRB', 'GTEX-P44G-0011-R5A-SM-2D45M', 'GTEX-17F9Y-0004-SM-6RBJW', 'GTEX-ZP4G-0002-SM-4YUW4', 'GTEX-1GN1V-0002-SM-7MA7H', 'GTEX-PLZ5-0006-SM-2M47Y', 'GTEX-1H23P-0002-SM-AG5YQ', 'GTEX-139UC-0005-SM-5PNU7', 'GTEX-PSDG-0426-SM-2S1OF', 'GTEX-1J1R8-0003-SM-G8OXS', 'GTEX-Q2AI-1626-SM-2QU33', 'GTEX-YB5E-0003-SM-CLTGF', 'GTEX-XUJ4-0126-SM-4BOP7', 'GTEX-14BMV-0826-SM-CYKTX-INPUT', 'GTEX-UTHO-0426-SM-5SI7N', 'GTEX-ZT9W-0002-SM-4YUVT', 'GTEX-13FH7-5007-SM-AN3Z1', 'GTEX-12WSH-0001-SM-6WSBP', 'GTEX-WFG7-0002-SM-3G3SI', 'GTEX-QLQW-1626-SM-4R1K1', 'GTEX-WFJO-0003-SM-3G3T4', 'GTEX-OIZG-0726-SM-2AXUL', 'GTEX-1O9I2-0004-SM-DE1IA', 'GTEX-1IE54-2126-SM-A8N97', 'K-562-SM-43ISS', 'GTEX-1LGOU-0003-SM-DE1KS', 'GTEX-1NV88-0926-SM-DKTS7', 'GTEX-R55F-1526-SM-2TWBA', 'GTEX-RVPV-0003-SM-6UH3S', 'GTEX-1AX8Y-0426-SM-7189O', 'GTEX-YEC3-1426-101806-SM-5PNXX', 'GTEX-13QJC-0003-SM-5GYCG', 'GTEX-14PN4-0011-R10a-SM-AHZ72', 'GTEX-NPJ7-0011-R9a-SM-2AXV3', 'GTEX-1S82U-0004-SM-DE1J4', 'GTEX-1RB15-0926-SM-E9U5Y', 'GTEX-S4Z8-0003-SM-2VCTZ', 'GTEX-1EMGI-0004-SM-DINCE', 'GTEX-ZEX8-0326-SM-4WKGS', 'GTEX-YFC4-0001-SM-6UH16', 'GTEX-R45C-2826-SM-EZ6XR', 'GTEX-1H1DF-0001-SM-AG5YI', 'GTEX-ZYFC-0001-SM-6WBT4', 'GTEX-145MG-0011-R10a-SM-CYKO9', 'GTEX-QDVJ-0226-SM-2M48Z', 'GTEX-13O3O-0011-R10a-SM-AHZMQ', 'K-562-SM-7IGNY', 'GTEX-R55F-0011-R6A-SM-2TF4L', 'GTEX-146FR-5007-SM-AN41G', 'GTEX-1LVA9-0004-SM-DE1KL', 'GTEX-11DYG-0004-SM-5DWSF', 'GTEX-Q2AG-0126-SM-2HMLB', 'GTEX-14B4R-0004-SM-5CV95', 'GTEX-QV44-0926-SM-2TO6O', 'GTEX-12WSG-0001-SM-7DRPL', 'GTEX-OXRN-0003-SM-2H2X4', 'GTEX-16Z82-0011-R10a-SM-CYKT5', 'GTEX-NPJ7-2726-SM-2AXTV', 'GTEX-13VXU-0004-SM-5DU5Q', 'GTEX-QDVJ-0426-SM-2M491', 'GTEX-RN64-0004-SM-3USS7', 'GTEX-ZPU1-0002-SM-6WBSJ', 'GTEX-SE5C-0526-SM-CKJTH', 'GTEX-11ONC-0003-SM-5DWT5', 'GTEX-1399U-1926-SM-DLIQQ', 'GTEX-S341-0001-SM-5JK33', 'GTEX-145MO-0001-SM-7DROL', 'GTEX-S7SF-0003-SM-3USSM', 'GTEX-1C2JI-5013-SM-AN45P', 'GTEX-11UD2-2826-SM-5GU6X', 'GTEX-11UD1-0001-SM-58Q8D', 'GTEX-WZTO-1326-SM-CYKUQ-INPUT', 'K-562-SM-4PQYN', 'GTEX-12WSB-3026-SM-59HKM', 'GTEX-1EU9M-0003-SM-7MA71', 'GTEX-Q2AG-0003-SM-2XK18', 'GTEX-OIZG-0003-SM-2XK2R', 'GTEX-13NYB-0004-SM-5HJQP', 'GTEX-RUSQ-0626-SM-2TWD6', 'GTEX-1RB15-0004-SM-DLIQB', 'GTEX-1GTWX-0001-SM-7MA6I', 'GTEX-XOT4-0001-SM-5URCM', 'GTEX-QDVJ-2026-SM-2QU3H', 'GTEX-1I1GR-0002-SM-AG5YZ', 'GTEX-1S5VW-1326-SM-E9TKA', 'GTEX-ZV6S-0002-SM-6WBSV', 'GTEX-OIZH-1626-SM-2D451', 'GTEX-ZAB5-0004-SM-6UH3A', 'GTEX-13FHP-0004-SM-5HJPM', 'GTEX-S7SF-1626-SM-2XU9K', 'GTEX-16XZZ-0011-R10a-SM-AHZM2', 'K-562-SM-4LMI2', 'GTEX-14BIL-0004-SM-7DRO6', 'GTEX-XAJ8-0001-SM-3PZ7B', 'GTEX-Q2AG-0426-SM-2QU4J', 'GTEX-OHPK-0003-SM-5JK4Z', 'GTEX-12WSI-0004-SM-5ANGP', 'GTEX-1JN1M-0004-SM-DJWWG', 'GTEX-Q2AH-2026-SM-2QU4Q', 'GTEX-XOTO-0011-R3A-SM-4B64W', 'GTEX-1NV5F-0004-SM-DJWX3', 'GTEX-1E1VI-0003-SM-7MA61', 'GTEX-14BIN-0004-SM-7DROI', 'GTEX-ZDTS-0526-SM-62LFA', 'GTEX-S7SF-0001-SM-2XV74', 'GTEX-13FLW-0003-SM-5CV8N', 'GTEX-17HHE-0002-SM-6RBJV', 'GTEX-14LZ3-0003-SM-7DLGY', 'GTEX-1ICLZ-2726-SM-A8N8R', 'GTEX-WRHU-0003-SM-4X5ZS', 'GTEX-S4UY-0003-SM-2XK2Q', 'GTEX-QVUS-0011-R6A-SM-2TWBT', 'GTEX-R3RS-0526-SM-2TO59', 'GTEX-1HKZK-0004-SM-AG5ZY', 'GTEX-1IDJD-5010-SM-AN47R', 'GTEX-RWSA-0003-SM-4X61Z', 'GTEX-13OVI-0003-SM-6WSCW', 'GTEX-WXYG-0001-SM-5SOE5', 'GTEX-RU72-0526-SM-2TWBS', 'GTEX-YEC3-1426-SM-4WWEM', 'GTEX-14PKU-2126-SM-7LT8A', 'GTEX-O5YU-0004-SM-53IT8', 'GTEX-1OJC4-1826-SM-EXOJI', 'GTEX-144FL-3126-SM-5TDCX', 'GTEX-N7MS-0011-R4a-SM-2AXW2', 'GTEX-145MF-0004-SM-5HJQO', 'GTEX-S7SF-0426-SM-2XUAV', 'GTEX-11DXY-0004-SM-5DWST', 'GTEX-1GTWX-3026-SM-9MQL4', 'GTEX-1JMOU-0004-SM-DE1KO', 'GTEX-VUSH-0002-SM-3PYKM', 'GTEX-U3ZH-0003-SM-5URBK', 'GTEX-11EI6-0004-SM-6WBTC', 'GTEX-P78B-0426-SM-2M48K', 'GTEX-1LKK1-0626-SM-DLT2M', 'GTEX-11GSO-2526-SM-CYKO1', 'GTEX-14PJP-0001-SM-5GYEA', 'GTEX-YEC3-0626-SM-5IFHZ', 'GTEX-QVJO-0011-R6A-SM-2TO5X', 'GTEX-15EOM-5004-SM-79ONJ', 'GTEX-146FR-5016-SM-AN41E', 'GTEX-N7MT-0011-R10A-SM-2IZHQ', 'GTEX-O5YV-0526-SM-26GN1', 'GTEX-14BMV-0011-R10b-SM-CYKPO', 'GTEX-R3RS-0005-SM-2TO73', 'GTEX-SIU8-0926-SM-2XV6Q', 'GTEX-1GL5R-2226-SM-7SB97', 'GTEX-13OVJ-0001-SM-5GYAL', 'GTEX-1477Z-0011-R10a-SM-AHZ6Z', 'GTEX-13FHP-1026-SM-CYKPW', 'GTEX-QLQ7-0003-SM-2XK22', 'GTEX-ZVT3-0001-SM-6WBS5', 'GTEX-WHWD-0002-SM-3G3SY', 'GTEX-NL3G-0007-SM-4SOIF', 'GTEX-12WSB-0426-SM-5BC6H', 'GTEX-11P7K-0002-SM-5DWTW', 'GTEX-VJYA-0003-SM-5SOED', 'GTEX-REY6-0005-SM-2TO6X', 'GTEX-YFC4-0001-SM-4YUWA', 'GTEX-RUSQ-1526-SM-2TWBQ', 'GTEX-139UC-0426-SM-5K7UR', 'GTEX-QDVJ-2126-SM-2QU44', 'GTEX-1399S-5001-SM-AN3YL', 'GTEX-11WQK-2526-SM-5CVL8', 'GTEX-X3Y1-0002-SM-3PYKJ', 'GTEX-PWOO-0004-SM-3UZFA', 'GTEX-1IDJI-2326-SM-A8N9I', 'GTEX-1C6WA-0003-SM-7MA6H', 'GTEX-15DYW-0002-SM-6RBJG', 'GTEX-N7MS-0826-SM-2AXU2', 'GTEX-S4P3-0004-SM-5JK4R', 'GTEX-S32W-0726-SM-2XUAY', 'GTEX-WVLH-0011-R10a-SM-AHZNZ', 'GTEX-WVLH-0011-R10A-SM-CYKW4-ELUATE', 'GTEX-12WS9-0011-R4b-SM-5P9EU', 'GTEX-P44G-0011-R4A-SM-2D45O', 'GTEX-Q2AI-1326-SM-2QU41', 'GTEX-1AX8Y-1926-SM-73KWX', 'GTEX-ZQUD-0126-SM-4YCDY', 'GTEX-1122O-2026-SM-5NQ91', 'GTEX-1S83E-0004-SM-DE1J1', 'GTEX-145MH-0626-SM-CYKUL', 'GTEX-X585-0003-SM-58Q8W', 'GTEX-X4EP-0002-SM-3PYKV', 'GTEX-14PJ4-5019-SM-AN42S', 'GTEX-11TUW-3126-SM-5GU4Y', 'GTEX-WK11-0002-SM-3PZ7M', 'GTEX-QEG5-0526-SM-2QU38', 'GTEX-XLM4-0011-R11A-SM-4AT46', 'GTEX-1FIGZ-0003-SM-7MA7A', 'GTEX-ZLFU-2126-SM-4WWEV', 'GTEX-RU72-0011-R7A-SM-2TWD8', 'GTEX-1C64N-0003-SM-7J3A2', 'GTEX-1B8L1-0004-SM-7MA5X', 'GTEX-U412-0001-SM-3G3T2', 'GTEX-PX3G-2626-SM-2IZI6', 'GTEX-1192X-0004-SM-5DWSK', 'GTEX-QVUS-0011-R1A-SM-2TWC2', 'GTEX-15ETS-5004-SM-AN43L', 'GTEX-S33H-2426-SM-2XU8L', 'GTEX-QV44-0126-SM-4R1KH', 'GTEX-TMZS-0002-SM-3G3S5', 'GTEX-13OVK-0001-SM-5GYAH', 'GTEX-R55F-0011-R8A-SM-2TWCT', 'GTEX-14PN4-0001-SM-6RBKN', 'GTEX-1GF9W-0004-SM-7MA6X', 'GTEX-R55D-0926-SM-2TWDB', 'GTEX-14C5O-0002-SM-5CV8D', 'K-562-SM-3NB3I', 'GTEX-P4PQ-2626-SM-2HMKM', 'GTEX-15EOM-0626-SM-6LPKI', 'GTEX-1C6VS-0003-SM-7MA5Y', 'GTEX-PWO3-0004-SM-5JK3T', 'GTEX-P4PP-0004-SM-3UZF8', 'GTEX-QEG5-0003-SM-5URCP', 'GTEX-OXRN-0011-R5A-SM-2D45L', 'GTEX-S32W-0001-SM-5JK4N', 'GTEX-11P82-0003-SM-6WBSX', 'GTEX-1399R-2226-SM-DLIQ5', 'GTEX-ZV7C-0002-SM-6WBSM', 'GTEX-1C64O-1526-SM-7EWES', 'GTEX-15CHQ-0011-R7b-SM-69LP1', 'GTEX-12WSG-5005-SM-AN3XB', 'GTEX-15G19-0001-SM-7J38Q', 'GTEX-13X6I-0011-R1b-SM-DG7EQ-HP3_B', 'GTEX-1HFI6-3126-SM-A8N8J', 'GTEX-131YS-0003-SM-7DRPE', 'GTEX-11ILO-0626-SM-5A5LO', 'K-562-SM-7IGL8', 'GTEX-ZDTS-0526-SM-6EU38', 'GTEX-ZVT2-0003-SM-4YUVI', 'GTEX-12WSD-0002-SM-5DWRK', 'GTEX-YFCO-0001-SM-6UH34', 'GTEX-1S3DN-0001-SM-DE1IZ', 'GTEX-1B8SF-0003-SM-7MA6B', 'GTEX-13S7M-0003-SM-5HJRR', 'GTEX-WRHK-0003-SM-6WBUL', 'GTEX-139D8-0001-SM-5GYBZ', 'GTEX-NPJ8-2526-SM-2HML8', 'GTEX-PWOO-2526-SM-2S1PG', 'GTEX-1PBJI-0003-SM-DE1IF', 'GTEX-ZF2S-0426-SM-4WKGP', 'GTEX-1PBJI-0003-SM-DJWWV', 'GTEX-1IDJF-0002-SM-DCCZD', 'GTEX-RWS6-0004-SM-3USSB', 'GTEX-15ER7-0926-SM-CYKTS-ELUATE', 'GTEX-U3ZM-0426-SM-57WG1', 'GTEX-1L5NE-0002-SM-DJWXX', 'GTEX-13SLX-0326-SM-CYKVN', 'GTEX-1JN1M-0004-SM-DE1NF', 'GTEX-N7MT-1226-SM-2D7W4', 'GTEX-12WSN-0626-SM-5BC61', 'GTEX-148VJ-1626-SM-5QGPH', 'GTEX-SUCS-0001-SM-325A7', 'GTEX-SIU8-0826-SM-2XV65', 'GTEX-R55G-0726-SM-2TWCM', 'GTEX-111FC-3026-SM-5GZYJ', 'K-562-SM-9HWQF', 'GTEX-1RQED-0001-SM-DLIP3', 'GTEX-11WQK-0011-R11a-SM-5BC6W', 'K-562-SM-3NMDG', 'GTEX-OHPJ-0526-SM-2HMJR', 'GTEX-15EOM-0004-SM-7DLFA', 'GTEX-OHPM-2626-SM-2AXUV', 'GTEX-14E6C-0002-SM-7DROW', 'GTEX-OHPL-1826-SM-2QU3B', 'GTEX-1JMQJ-0003-SM-DJWY9', 'GTEX-X585-0003-SM-3PZ7U', 'GTEX-OOBJ-0003-SM-5JK45', 'GTEX-ZYW4-1026-SM-5SI8W', 'GTEX-PWN1-0226-SM-2S1OZ', 'GTEX-117YW-0003-SM-6WBSY', 'GTEX-13NZ9-0004-SM-6WSC6', 'GTEX-17EVP-0004-SM-6UH1L', 'GTEX-S3LF-0226-SM-EZ6LJ', 'GTEX-14PHY-0002-SM-5GYE7', 'GTEX-1S83E-0004-SM-DJWXM', 'GTEX-OHPK-0326-SM-2D456', 'GTEX-REY6-0526-SM-2TO71', 'GTEX-S7SE-0004-SM-325A3', 'GTEX-T2YK-0003-SM-325AB', 'GTEX-RU72-0004-SM-4X623', 'GTEX-1AX9K-0001-SM-7MD4I', 'GTEX-11EI6-0004-SM-58Q7L', 'GTEX-13OW7-0001-SM-5GYA6', 'GTEX-O5YW-1826-SM-2M47V', 'GTEX-PX3G-0006-SM-33HBQ', 'GTEX-12WSM-0326-SM-CYKW3', 'GTEX-1B8SG-0003-SM-7MD4Y', 'GTEX-ZLWG-0226-SM-4WWC6', 'GTEX-XOTO-0002-SM-5URC7', 'GTEX-1HCVE-0726-SM-DK2JZ', 'GTEX-13NYC-2626-SM-5K7VN', 'GTEX-11P81-0002-SM-5DWT1', 'GTEX-11LCK-0002-SM-58Q7P', 'GTEX-N7MS-0225-SM-4E3HO', 'GTEX-1AYCT-0004-SM-7MD4U', 'GTEX-13VXU-0004-SM-5HJQS', 'GTEX-OXRL-0326-SM-4MVNY', 'GTEX-ZPU1-1526-SM-4YCFF', 'GTEX-ZVZQ-0426-SM-59HLT', 'GTEX-R55E-0011-R4A-SM-2TO6H', 'GTEX-S4Z8-1526-SM-731D2', 'GTEX-14PJM-0003-SM-7DLG8', 'GTEX-POMQ-2326-SM-2S1O8', 'GTEX-QCQG-0006-SM-2S1OW', 'GTEX-1IDJD-0003-SM-DC69J', 'GTEX-XLM4-0005-SM-5S2RY', 'GTEX-1PIIG-2026-SM-DKTQ3', 'GTEX-R55C-1926-SM-2TO4Y', 'GTEX-15DCD-0011-R3b-SM-6EU1I', 'GTEX-15DCZ-1126-SM-6871O', 'GTEX-O5YT-0010-SM-5JK31', 'GTEX-13RTL-0826-SM-DOKJO', 'GTEX-WCDI-0001-SM-4X62J', 'GTEX-1JMLX-0826-SM-DKTR9', 'GTEX-11I78-1626-SM-5A5M2', 'K-562-SM-5GZWN', 'GTEX-14ASI-0011-R10a-SM-AHZNF', 'GTEX-13G51-0011-R10a-SM-AHZ6X', 'GTEX-N7MS-1626-SM-3LK5F', 'GTEX-OOBJ-1626-SM-2D44Y', 'GTEX-P4QR-1026-SM-2I5GP', 'GTEX-YFC4-0001-SM-53ITN', 'GTEX-S341-1926-SM-3K2BA', 'GTEX-15D1Q-0004-SM-7DLFO', 'GTEX-1AX8Y-0826-SM-731EV', 'GTEX-12WSA-1026-SM-CYKQ3', 'GTEX-T5JC-0005-SM-4DM7B', 'GTEX-QV44-0003-SM-4SVQB', 'GTEX-QMRM-0826-SM-2I5G7', 'GTEX-P44G-0011-R1A-SM-2D45S', 'GTEX-P4QT-1626-SM-5S2UR', 'GTEX-16A39-5016-SM-AN44C', 'GTEX-11ILO-0126-SM-5A5LN', 'GTEX-X4EO-0003-SM-3USRE', 'GTEX-QVJO-0002-SM-5URD6', 'GTEX-RWS6-2126-SM-2XU95', 'GTEX-S4Z8-1926-SM-3K2AR', 'GTEX-ZF29-1326-SM-4WWB6', 'GTEX-PVOW-1026-SM-2XV5D', 'GTEX-XQ8I-0001-SM-53IUL', 'GTEX-X585-0011-R10A-SM-CYKUN', 'GTEX-Q2AG-0011-R11A-SM-2IZK6', 'GTEX-15DZA-0004-SM-7DLFX', 'GTEX-15DCZ-1726-SM-69LPX', 'GTEX-WOFL-0003-SM-4SVQY', 'GTEX-17EUY-0003-SM-6UH27', 'GTEX-13O3O-1526-SM-CYKUS', 'GTEX-S4P3-1526-SM-3K2AV', 'GTEX-1C2JI-0006-SM-7MKFD', 'GTEX-TSE9-0002-SM-3G3SA', 'GTEX-PWOO-1526-SM-48TCK', 'GTEX-15SHW-0004-SM-6RBKO', 'GTEX-145ME-1926-SM-5SI9S', 'GTEX-Q734-2026-SM-2QU4S', 'GTEX-SIU7-0002-SM-2VCU5', 'GTEX-XBEC-0002-SM-5SOEV', 'GTEX-1RMOY-0003-SM-DLIQE', 'GTEX-1IDFM-0003-SM-DCCZO', 'GTEX-WYJK-0001-SM-3PZ7Z', 'GTEX-144GM-0226-SM-DK2JT', 'GTEX-12584-0004-SM-7DRPH', 'GTEX-NL3H-0011-R4a-SM-2AXW4', 'GTEX-XBEC-0002-SM-53IUU', 'GTEX-P4QR-0726-SM-2I5GO', 'GTEX-145MF-0004-SM-7DRP1', 'GTEX-R45C-0003-SM-2XK31', 'GTEX-13W46-0003-SM-6WSAK', 'GTEX-S4Q7-1526-SM-2XU9C', 'GTEX-VUSH-0326-SM-3NB2I', 'GTEX-14JG1-0011-R11a-SM-62LEE', 'GTEX-ZLV1-0826-SM-4WWEQ', 'GTEX-19HZE-0626-SM-73KXJ', 'GTEX-1MCC2-0004-SM-DE1JS', 'GTEX-13S86-5019-SM-AN3ZV', 'GTEX-QEG5-1826-SM-4R1JP', 'GTEX-131XF-0004-SM-7DRPB', 'GTEX-1B8SG-0003-SM-7J3A3', 'GTEX-PWOO-0626-SM-48TZH', 'GTEX-O5YW-0003-SM-2H2WT', 'GTEX-PWOO-0726-SM-2IZI1', 'GTEX-146FH-0002-SM-6WSBB', 'GTEX-RWS6-0626-SM-2XUAT', 'GTEX-X15G-0002-SM-3PYKW', 'K-562-SM-32QPQ', 'GTEX-1HCU8-0008-SM-DMZ2E', 'GTEX-111FC-0826-SM-CYKRT', 'GTEX-ZDTT-0002-SM-53UAP', 'GTEX-OXRK-0826-SM-2D44K', 'GTEX-1IDJV-0001-SM-G8OXC', 'GTEX-13CZV-0003-SM-6WSB7', 'GTEX-S32W-2426-SM-2XU8T', 'GTEX-W5X1-0002-SM-3G3S7', 'GTEX-ZVE2-0002-SM-6WBSQ', 'GTEX-W5WG-0003-SM-5URC3', 'GTEX-ZLV1-1526-SM-4WWBC', 'GTEX-13O3O-0126-SM-5KM4E', 'GTEX-1GN1U-3126-SM-9KNV6', 'GTEX-1KD4Q-0002-SM-DJWYV', 'GTEX-1MA7W-0004-SM-DE1K6', 'GTEX-19HZE-0011-R11b-SM-79ONN', 'GTEX-14PJ4-5004-SM-AN42T', 'GTEX-15EO6-0004-SM-7DLGJ', 'K-562-SM-5E43B', 'GTEX-15SDE-0001-SM-6UH2N', 'GTEX-XOTO-0011-R6B-SM-4B65X', 'GTEX-12126-1026-SM-CYKSE', 'GTEX-Y3IK-0004-SM-6UH1D', 'GTEX-132AR-1626-SM-5EGHP', 'GTEX-N7MS-0007-SM-2D43E', 'GTEX-XOTO-0326-SM-4B66K', 'GTEX-1P4AB-0001-SM-DJWXJ', 'GTEX-N7MT-1326-SM-2AXUA', 'GTEX-SIU8-0926-SM-5SI87', 'GTEX-S7PM-0926-SM-4X617', 'GTEX-Q2AI-0003-SM-2IJGK', 'GTEX-1GPI7-0002-SM-DLIOU', 'GTEX-SNMC-0004-SM-4X61G', 'GTEX-1EH9U-0011-R11b-SM-CE6R1', 'GTEX-1R9PM-0001-SM-DLIQD', 'GTEX-UPIC-1526-SM-4IHLU', 'GTEX-UTHO-2426-SM-CKJTI', 'GTEX-RU72-0004-SM-5SOEJ', 'GTEX-PVOW-0006-SM-2XV5H', 'GTEX-V955-0826-SM-3GAF9', 'GTEX-145ME-5016-SM-AN41F', 'GTEX-QLQ7-0003-SM-5URCS', 'GTEX-1QP28-0004-SM-DE1JK', 'GTEX-S3LF-0001-SM-DLIOS', 'GTEX-ZYWO-0004-SM-5DWTS', 'GTEX-QV44-0006-SM-2S1Q7', 'GTEX-X261-0002-SM-3USR3', 'GTEX-WFG7-2026-SM-3GIL7', 'GTEX-14PN3-5001-SM-AN434', 'GTEX-RTLS-0006-SM-2TWAQ', 'GTEX-15ETS-0003-SM-6RBJE', 'GTEX-QXCU-0004-SM-2MRI8', 'GTEX-Z93T-0002-SM-DLIPS', 'GTEX-1339X-0226-SM-DLIOH', 'GTEX-13QIC-0004-SM-5DU5F', 'GTEX-QDVN-2226-SM-2QU3V', 'GTEX-NL3H-0006-SM-2I3FW', 'GTEX-ZF29-2026-SM-4WWB7', 'K-562-SM-7P8RB', 'GTEX-18A6Q-0003-SM-DINAS', 'GTEX-17EVQ-0001-SM-6RBK4', 'GTEX-QV31-0003-SM-2XK27', 'GTEX-XUJ4-2126-SM-4BOOX', 'GTEX-14C39-0011-R10b-SM-CYKPP', 'GTEX-145MO-0001-SM-5GY9Z', 'GTEX-1IDJU-0426-SM-A8N91', 'GTEX-1RDX4-0008-SM-G8OXP', 'GTEX-15DCZ-0726-SM-69LOV', 'GTEX-Q2AG-0005-SM-2IZK2', 'GTEX-QLQW-1326-SM-2M48H', 'GTEX-17KNJ-5019-SM-AN44V', 'GTEX-NL3H-0009-SM-5JK37', 'GTEX-P44G-2426-SM-2D44E', 'GTEX-145LU-0004-SM-DLIOV', 'GTEX-OXRP-0003-SM-5JK3P', 'GTEX-XOTO-0826-SM-4B65O', 'GTEX-NPJ8-0326-SM-2D43G', 'GTEX-14ABY-0011-R10a-SM-AHZ73', 'GTEX-1F6I4-0004-SM-7MA6S', 'GTEX-14JIY-0011-R11b-SM-62LE8', 'GTEX-1399T-0426-SM-CYKV1', 'GTEX-1LB8K-3126-SM-E9U57', 'GTEX-14PJ5-0001-SM-5GYE4', 'GTEX-S7SE-0011-R10a-SM-AHZNW', 'GTEX-1F5PK-0003-SM-7MA73', 'GTEX-RVPU-0001-SM-7DLFD', 'GTEX-1PIEJ-0003-SM-DE1IH', 'GTEX-11EQ9-0003-SM-58Q7V', 'GTEX-ZYWO-0826-SM-5E45W', 'K-562-SM-5K7WK', 'GTEX-13O3O-0726-SM-CYKVH', 'GTEX-1A8G7-0003-SM-7MA6A', 'GTEX-WFG8-0004-SM-3G3SO', 'GTEX-14AS3-0003-SM-5DU52', 'GTEX-11GSO-0003-SM-5DWSW', 'GTEX-SJXC-0005-SM-2XV6A', 'GTEX-1GZ2Q-1826-SM-9JGHQ', 'GTEX-YJ8A-0002-SM-4YUUZ', 'GTEX-144GM-0226-SM-E9IK3', 'GTEX-1JN6P-0001-SM-DE1N7', 'GTEX-11WQK-0003-SM-58Q8U', 'GTEX-1GN2E-0003-SM-7MA6W', 'GTEX-YB5K-1426-SM-5IFIB', 'GTEX-QXCU-0826-SM-2TC6F', 'GTEX-U8T8-0226-SM-3DB97', 'GTEX-O5YT-0010-SM-3UZF5', 'GTEX-RU72-0011-R11A-SM-5SI8G', 'GTEX-ZVT2-0003-SM-6WBS4', 'GTEX-RU1J-1726-SM-2TWCK', 'GTEX-132Q8-0004-SM-5DU3I', 'GTEX-Q2AH-1926-SM-2QU43', 'K-562-SM-7KUDR', 'GTEX-13FHO-0011-R10a-SM-AHZMN', 'GTEX-13FTW-0003-SM-6WSBY', 'GTEX-RWSA-0003-SM-2VCTT', 'GTEX-1RNTQ-0003-SM-DLIQF', 'GTEX-1H3NZ-2026-SM-A8N9P', 'GTEX-1KXAM-5019-SM-AN493', 'GTEX-183WM-0004-SM-6UH1T', 'GTEX-14PN3-0004-SM-7DLGK', 'K-562-SM-5MR5S', 'GTEX-YF7O-2326-101833-SM-5CVN9', 'GTEX-1192X-0004-SM-58Q7Q', 'GTEX-OOBK-0005-SM-2D43L', 'GTEX-WHSB-0326-SM-3LK6K', 'GTEX-OXRO-1926-SM-2M488', 'GTEX-WL46-0002-SM-4X61I', 'GTEX-11XUK-0003-SM-5DWTF', 'GTEX-RVPU-0011-R5A-SM-2XCAD', 'GTEX-OOBJ-0226-SM-2D453', 'GTEX-T8EM-0001-SM-3G3SK', 'GTEX-O5YV-0004-SM-2XJZR', 'GTEX-ZYWO-1926-SM-5E456', 'GTEX-OOBJ-0003-SM-2BWZC', 'GTEX-XOTO-0011-R5A-SM-4B657', 'GTEX-13QIC-0004-SM-6WSCY', 'GTEX-15ETS-0003-SM-7DLGI', 'GTEX-139UC-3126-SM-5J1OY', 'GTEX-S4Z8-2026-SM-3K2A9', 'GTEX-1GMR3-5001-SM-AN46H', 'GTEX-WVLH-0001-SM-4X624', 'GTEX-QVJO-0011-R4A-SM-2TO67', 'GTEX-QDVJ-2126-SM-2S1PO', 'GTEX-R55E-0011-R9A-SM-2TO6R', 'GTEX-XMD3-0003-SM-53IVI', 'GTEX-P4QR-1026-SM-2M49Q', 'GTEX-P44G-0011-R2A-SM-2XV51', 'GTEX-1MJK3-1226-SM-E9J3G', 'GTEX-S7SF-2126-SM-3K2B2', 'GTEX-15CHS-0001-SM-6RBJP', 'GTEX-X261-0002-SM-3PZ7E', 'GTEX-131XE-0004-SM-5GYA4', 'GTEX-1J1R8-0926-SM-A8N93', 'GTEX-X88G-0003-SM-DLIPW', 'GTEX-13FLW-1426-SM-CYKSG', 'GTEX-QDT8-0326-SM-2XV4W', 'GTEX-15CHR-5019-SM-AN43C', 'GTEX-S4UY-0003-SM-4X61Q', 'GTEX-QLQ7-1626-SM-2QU45', 'GTEX-1LVAN-0004-SM-ANILG', 'GTEX-13OVH-0011-R10a-SM-AHZMU', 'GTEX-12WSN-1426-SM-5GCO6', 'GTEX-15TU5-0326-SM-6PALS', 'GTEX-15DCZ-2326-SM-6AJB1', 'GTEX-12WSM-0001-SM-6WSBS', 'GTEX-PWO3-0005-SM-2IZJC', 'GTEX-1N5O9-0004-SM-DJWYY', 'GTEX-15DCZ-0426-SM-DLL4Q', 'GTEX-ZQG8-1926-SM-4YCEO', 'GTEX-1H3VY-5004-SM-AN46M', 'GTEX-U8T8-2026-SM-4SVQM', 'GTEX-16NPX-0002-SM-6UH1Y', 'GTEX-T6MN-0003-SM-5URB7', 'GTEX-1CB4J-1626-SM-79OOK', 'GTEX-1K2DU-0004-SM-E6UGM', 'GTEX-POYW-1226-SM-325A1', 'GTEX-OXRL-0005-SM-2D43N', 'K-562-SM-3GIKB', 'GTEX-WHWD-0002-SM-4X61F', 'GTEX-Q734-1926-SM-2IZIV', 'GTEX-1AX8Z-0002-SM-7MD5D', 'GTEX-13OW5-0003-SM-5GYAG', 'GTEX-14PJM-0011-R10a-SM-AHZNK', 'GTEX-1IL2V-0003-SM-G8OXR', 'GTEX-13OW8-0002-SM-6WSAW', 'GTEX-WOFL-0003-SM-3PZ7H', 'GTEX-131XW-0002-SM-7DROR', 'GTEX-V955-0826-SM-5SI82', 'GTEX-Y3IK-0004-SM-4YUX2', 'GTEX-1LGOU-5013-SM-AN49I', 'GTEX-13FTZ-0011-R10a-SM-AHZ6S', 'GTEX-Q2AG-0826-SM-2IZK5', 'GTEX-PWN1-0006-SM-5SI7T', 'GTEX-1AX9K-5010-SM-AN45D', 'GTEX-11GSO-2926-SM-CYKQT', 'GTEX-S7PM-0926-SM-3PZ76', 'GTEX-1PIGE-0001-SM-DJWWW', 'GTEX-P78B-0626-SM-2M48C', 'GTEX-WI4N-2026-SM-4OOS7', 'GTEX-OIZF-1326-SM-7SB8E', 'GTEX-T5JW-0002-SM-3G3TO', 'GTEX-1AMFI-0001-SM-7MD4R', 'GTEX-1J8QM-5019-SM-AN499', 'GTEX-1QEPI-0001-SM-DE1JM', 'GTEX-13N11-0003-SM-6WSCJ', 'GTEX-YFCO-1626-SM-4W1Z3', 'GTEX-RN64-0626-SM-2TO65', 'GTEX-OOBK-2626-SM-2HMKY', 'GTEX-1J8EW-0003-SM-G8OX9', 'GTEX-OHPJ-0006-SM-3LK6G', 'K-562-SM-CJI1Q', 'GTEX-RU1J-0002-SM-5URDF'])

In [19]:
len(mismatch_ids)

5569

In [20]:
print(sample_ids.shape)
print(len(mismatch_ids))
print('There are ', len(sample_ids) - len(mismatch_ids) ,'samples with expression measured.')

(22951,)
5569
('There are ', 17382, 'samples with expression measured.')


In [21]:
sample_ids = pd.Series(sample_ids)

In [22]:
leftover_sample_ids = sample_ids[~sample_ids.isin(mismatch_ids)]

### Check previous metadata and how the leftover_sample_ids match
There are 17382 samples in both datasets

In [23]:
metadf[metadf.SAMPID.isin(leftover_sample_ids)].shape

(17382, 63)

In [24]:
metadf[metadf.SAMPID.isin(leftover_sample_ids)].loc[:, 'SMTS'].value_counts()

Brain              2642
Skin               1809
Esophagus          1445
Blood Vessel       1335
Adipose Tissue     1204
Blood               929
Heart               861
Muscle              803
Colon               779
Thyroid             653
Nerve               619
Lung                578
Breast              459
Testis              361
Stomach             359
Pancreas            328
Pituitary           283
Adrenal Gland       258
Prostate            245
Spleen              241
Liver               226
Small Intestine     187
Ovary               180
Salivary Gland      162
Vagina              156
Uterus              142
Kidney               89
Bladder              21
Cervix Uteri         19
Fallopian Tube        9
Name: SMTS, dtype: int64

In [25]:
metadf[metadf.SAMPID.isin(leftover_sample_ids)].loc[:, 'SMTSD'].value_counts()#.head()

Muscle - Skeletal                            803
Whole Blood                                  755
Skin - Sun Exposed (Lower leg)               701
Artery - Tibial                              663
Adipose - Subcutaneous                       663
Thyroid                                      653
Nerve - Tibial                               619
Skin - Not Sun Exposed (Suprapubic)          604
Lung                                         578
Esophagus - Mucosa                           555
Adipose - Visceral (Omentum)                 541
Esophagus - Muscularis                       515
Cells - Cultured fibroblasts                 504
Breast - Mammary Tissue                      459
Heart - Left Ventricle                       432
Artery - Aorta                               432
Heart - Atrial Appendage                     429
Colon - Transverse                           406
Esophagus - Gastroesophageal Junction        375
Colon - Sigmoid                              373
Testis              

In [26]:
%%time
gtex = parse(file_path='./data/raw/GTEx_v1.9/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gctx', 
             cid=list(leftover_sample_ids))

CPU times: user 5.96 s, sys: 2.36 s, total: 8.33 s
Wall time: 8.42 s


In [27]:
# 56k genes * 17382 samples
gtex.data_df.shape

(56200, 17382)

In [28]:
print(gtex.col_metadata_df.shape)
print(gtex.row_metadata_df.shape)

(17382, 0)
(56200, 1)


In [29]:
gtex.row_metadata_df

rhd,Description
rid,
ENSG00000223972.5,DDX11L1
ENSG00000227232.5,WASH7P
ENSG00000278267.1,MIR6859-1
ENSG00000243485.5,MIR1302-2HG
ENSG00000237613.2,FAM138A
ENSG00000268020.3,OR4G4P
ENSG00000240361.1,OR4G11P
ENSG00000186092.4,OR4F5
ENSG00000238009.6,RP11-34P13.7


In [30]:
# universe of protein coding genes
gene_universe = pd.read_csv('./data/gene_universe.txt', header=None)
gene_universe = gene_universe.iloc[:, 0]

In [31]:
# df with genes in rows and selected tissue samples in columns
gtex.data_df.head()

cid,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,...,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
rid,,,,,,,,,,,,,,,,,,,,,
ENSG00000223972.5,0.00000,0.000,0.000,0.00000,0.000,0.000,0.00,0.000000,0.00,0.00000,...,0.00000,0.03629,0.00000,0.00,0.00000,0.0000,0.000,0.00000,0.01965,0.02522
ENSG00000227232.5,8.76400,3.861,7.349,11.07000,3.306,5.389,11.99,16.950001,10.04,12.50000,...,1.60600,2.26800,5.38600,2.31,2.45600,4.0230,1.922,2.85700,0.86960,2.16700
ENSG00000278267.1,0.00000,0.000,1.004,0.00000,0.000,0.000,0.00,0.000000,0.00,0.00000,...,0.00000,0.00000,0.00000,0.00,0.00000,0.0000,0.000,0.00000,0.00000,0.00000
ENSG00000243485.5,0.07187,0.000,0.000,0.06761,0.000,0.000,0.00,0.000000,0.00,0.06265,...,0.00000,0.00000,0.06073,0.00,0.08464,0.1435,0.000,0.05216,0.00000,0.00000
ENSG00000237613.2,0.00000,0.000,0.000,0.00000,0.000,0.000,0.00,0.039040,0.00,0.00000,...,0.02429,0.00000,0.00000,0.00,0.00000,0.0000,0.000,0.00000,0.00000,0.00000


In [32]:
df = gtex.data_df.merge(gtex.row_metadata_df, left_index=True, right_index=True)

In [33]:
df = df.rename(columns={'Description': 'gene_symbol'}).set_index('gene_symbol')

In [34]:
# select only genes from protein coding gene universe
df = df[df.index.isin(gene_universe)]

In [35]:
df.head()

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,...,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
gene_symbol,,,,,,,,,,,,,,,,,,,,,
OR4F5,0.000000,0.102500,0.074340,0.000000,0.04233,0.056090,0.174300,0.000000,0.000000,0.059100,...,0.129000,0.136700,0.057290,0.12690,0.119800,0.06767,0.113500,0.000000,0.148000,0.000000
OR4F29,0.000000,0.050090,0.000000,0.000000,0.00000,0.054830,0.085190,0.050680,0.198600,0.000000,...,0.000000,0.033410,0.112000,0.00000,0.117100,0.00000,0.055480,0.192400,0.144700,0.000000
OR4F16,0.066280,0.000000,0.000000,0.000000,0.04138,0.000000,0.000000,0.000000,0.049650,0.000000,...,0.031540,0.033410,0.000000,0.04136,0.078060,0.19850,0.166400,0.000000,0.000000,0.046440
SAMD11,0.359800,0.476000,0.276200,12.030000,11.39000,1.712000,25.120001,2.091000,0.673900,0.611700,...,6.652000,4.761000,1.247000,1.76300,1.028000,0.80820,0.451900,6.346000,1.051000,0.163900
NOC2L,90.739998,93.559998,94.870003,67.599998,26.52000,33.669998,85.620003,93.059998,64.540001,80.949997,...,34.880001,42.610001,60.419998,41.23000,52.209999,55.73000,71.209999,58.759998,75.220001,39.299999


In [36]:
df.shape

(19128, 17382)

In [37]:
# total number of donors 
df.columns.str.split('-').str[:2].str.join('-').nunique()

948

In [38]:
df.to_csv('./data/processed/gtex_sample_expression.csv')